A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
testUSDCMSSpreadAndEquityDualRAPutAtMat_20201125_LGM2F_2023.m


clc
clear
valueDate = '2020-11-25';

maxIdx = 5;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);

%% CD-SNP500 Dual Range Accrual Pricing start

%RA info
rangeAccrualParams.payCcy = 'USD';
rangeAccrualParams.startDate = H_Date('2020-11-25');
rangeAccrualParams.maturity = 15;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.0555;
couponFreq = 0.25;

% Long: USD CMS 30y
% Short: USD CMS 5Y
tenorL = 30;

tenorS = 5;

lowerS = -0.001;
upperS = 999;

lowerEQ = 50;
upperEQ = 999999;

putAtMatStrike = 40;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20201125	20201125	20210225	20210225
20210225	20210225	20210525	20210525
20210525	20210525	20210825	20210825
20210825	20210825	20211125	20211125
20211125	20211125	20220225	20220225
20220225	20220225	20220525	20220525
20220525	20220525	20220825	20220825
20220825	20220825	20221125	20221125
20221125	20221125	20230225	20230225
20230225	20230225	20230525	20230525
20230525	20230525	20230825	20230825
20230825	20230825	20231125	20231125
20231125	20231125	20240225	20240225
20240225	20240225	20240525	20240525
20240525	20240525	20240825	20240825
20240825	20240825	20241125	20241125
20241125	20241125	20250225	20250225
20250225	20250225	20250525	20250525
20250525	20250525	20250825	20250825
20250825	20250825	20251125	20251125
20251125	20251125	20260225	20260225
20260225	20260225	20260525	20260525
20260525	20260525	20260825	20260825
20260825	20260825	20261125	20261125
20261125	20261125	20270225	20270225
20270225	20270225	20270525	20270525
20270525	20270525	20270825	20270825
20270825	20270825	20271125	20271125
20271125	20271125	20280225	20280225
20280225	20280225	20280525	20280525
20280525	20280525	20280825	20280825
20280825	20280825	20281125	20281125
20281125	20281125	20290225	20290225
20290225	20290225	20290525	20290525
20290525	20290525	20290825	20290825
20290825	20290825	20291125	20291125
20291125	20291125	20300225	20300225
20300225	20300225	20300525	20300525
20300525	20300525	20300825	20300825
20300825	20300825	20301125	20301125
20301125	20301125	20310225	20310225
20310225	20310225	20310525	20310525
20310525	20310525	20310825	20310825
20310825	20310825	20311125	20311125
20311125	20311125	20320225	20320225
20320225	20320225	20320525	20320525
20320525	20320525	20320825	20320825
20320825	20320825	20321125	20321125
20321125	20321125	20330225	20330225
20330225	20330225	20330525	20330525
20330525	20330525	20330825	20330825
20330825	20330825	20331125	20331125
20331125	20331125	20340225	20340225
20340225	20340225	20340525	20340525
20340525	20340525	20340825	20340825
20340825	20340825	20341125	20341125
20341125	20341125	20350225	20350225
20350225	20350225	20350525	20350525
20350525	20350525	20350825	20350825
20350825	20350825	20351125	20351125
];

endDate = H_Date(couponScheduleInt(size(couponScheduleInt,1),4));

% %% CK1F model param matu setting start
% % matu = 15 + 1.0/365.0;
% matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% 
% matu = 20.0;
% 
% % matu = matu -10.0;
% 
% %% end

 rangeDays = zeros(60,1);

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) >= 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,1))) + 1;
    end
end


%resetDate startDate endDate paymentDate
callScheduleInt = [20201125	20201125	20210225	20201125
20210225	20210225	20210525	20210225
20210525	20210525	20210825	20210525
20210825	20210825	20211125	20210825
20211125	20211125	20220225	20211125
20220225	20220225	20220525	20220225
20220525	20220525	20220825	20220525
20220825	20220825	20221125	20220825
20221125	20221125	20230225	20221125
20230225	20230225	20230525	20230225
20230525	20230525	20230825	20230525
20230825	20230825	20231125	20230825
20231125	20231125	20240225	20231125
20240225	20240225	20240525	20240225
20240525	20240525	20240825	20240525
20240825	20240825	20241125	20240825
20241125	20241125	20250225	20241125
20250225	20250225	20250525	20250225
20250525	20250525	20250825	20250525
20250825	20250825	20251125	20250825
20251125	20251125	20260225	20251125
20260225	20260225	20260525	20260225
20260525	20260525	20260825	20260525
20260825	20260825	20261125	20260825
20261125	20261125	20270225	20261125
20270225	20270225	20270525	20270225
20270525	20270525	20270825	20270525
20270825	20270825	20271125	20270825
20271125	20271125	20280225	20271125
20280225	20280225	20280525	20280225
20280525	20280525	20280825	20280525
20280825	20280825	20281125	20280825
20281125	20281125	20290225	20281125
20290225	20290225	20290525	20290225
20290525	20290525	20290825	20290525
20290825	20290825	20291125	20290825
20291125	20291125	20300225	20291125
20300225	20300225	20300525	20300225
20300525	20300525	20300825	20300525
20300825	20300825	20301125	20300825
20301125	20301125	20310225	20301125
20310225	20310225	20310525	20310225
20310525	20310525	20310825	20310525
20310825	20310825	20311125	20310825
20311125	20311125	20320225	20311125
20320225	20320225	20320525	20320225
20320525	20320525	20320825	20320525
20320825	20320825	20321125	20320825
20321125	20321125	20330225	20321125
20330225	20330225	20330525	20330225
20330525	20330525	20330825	20330525
20330825	20330825	20331125	20330825
20331125	20331125	20340225	20331125
20340225	20340225	20340525	20340225
20340525	20340525	20340825	20340525
20340825	20340825	20341125	20340825
20341125	20341125	20350225	20341125
20350225	20350225	20350525	20350225
20350525	20350525	20350825	20350525
20350825	20350825	20351125	20350825
];

callYN = ones(60,1);
callYN(1) = 0;
callYN(2) = 0;


manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','putAtMatStrike','tenorL', ...
                                'tenorS','lowerS','upperS','lowerEQ','upperEQ','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,putAtMatStrike,tenorL,tenorS,lowerS,upperS,lowerEQ,upperEQ,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});
                    
eventDatesInfo={rangeAccrualParams.startDate,endDate};
    
scheduleInt = rangeAccrualInfo('couponScheduleInt');
scheduleSize = size(scheduleInt,1);
autoCallScheduleEventEates={};
for i=1:scheduleSize
    eventDatesInfo{end+1} = H_Date(scheduleInt(i,1));
end

rangeAccrualInfo('eventDates') = eventDatesInfo; 
rangeAccrualInfo('valueDateH')= valueDateH;
% 
% tenorL = 0.25;
% tenorS = 0.25;
% 
% tenor = 0.25;
% 
% lowerS = 0.0;
% upperS = 999;
% 
% 
% lower = 0.0;
% upper = 0.05;

rangeAccrualInfo('lowerS') =lowerS;
rangeAccrualInfo('upperS') =upperS;
% rangeAccrualInfo('tenor') = 0.25;

rangeAccrualParams.params = rangeAccrualInfo;

% rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);
% rangeAccrual = CDCMSSpreadDualRangeAccrual(rangeAccrualParams);
rangeAccrual = CMSSpreadAndEquityDualRAPutAtMat(rangeAccrualParams);

% maxIdx =2.5;

maxIdx = 2;

rmseSimul = zeros(100,1);
for idx2=1:1
    %% USD Model building
    %% LGM2F model param matu setting
%     % matu = 15 + 1.0/365.0;
%     % matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
%     matu = 20.0;

    
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.00081111
0.083333333	0.001290309
0.166666667	0.001661613
0.25	0.00218909
0.5	0.002567648
0.75	0.002394088
1	0.002189996
1.5	0.002358275
2	0.002542794
2.5	0.002761162
3	0.002980484
3.5	0.00331556
4	0.003653897
4.5	0.004095603
5	0.004542049
5.5	0.005022558
6	0.005504425
6.5	0.005986293
7	0.006468161
7.5	0.006871943
8	0.00727307
8.5	0.007674198
9	0.008075325
9.5	0.008476453
10	0.00887758
10.5	0.009187056
11	0.009491776
11.5	0.009796497
12	0.010101218
12.5	0.010312606
13	0.010519716
13.5	0.010726826
14	0.010933936
14.5	0.011141045
15	0.011348155
15.5	0.011474448
16	0.011597036
16.5	0.011719625
17	0.011842213
17.5	0.011964801
18	0.012087389
18.5	0.012209977
19	0.012332565
19.5	0.012455154
20	0.012577742
20.5	0.012633118
21	0.012684598
21.5	0.012736078
22	0.012787557
22.5	0.012839037
23	0.012890517
23.5	0.012941997
24	0.012993477
24.5	0.013044957
25	0.013096437
25.5	0.013119813
26	0.013141386
26.5	0.01316296
27	0.013184533
27.5	0.013206106
28	0.01322768
28.5	0.013249253
29	0.013270827
29.5	0.0132924
30	0.013313974
30.5	0.013279634
31	0.013241359
31.5	0.013203084
32	0.013164809
32.5	0.013126534
33	0.013088259
33.5	0.013049984
34	0.01301171
34.5	0.012973435
35	0.01293516
35.5	0.012896885
36	0.01285861
36.5	0.012820335
37	0.01278206
37.5	0.012743785
38	0.01270551
38.5	0.012667235
39	0.01262896
39.5	0.012590685
40	0.01255241
40.5	0.012514135
41	0.01247586
41.5	0.012437586
42	0.012399311
42.5	0.012361036
43	0.012322761
43.5	0.012284486
44	0.012246211
44.5	0.012207936
45	0.012169661
45.5	0.012131386
46	0.012093111
46.5	0.012054836
47	0.012016561
47.5	0.011978286
    ];

%     for idx=1:size(rawData,1)
%         rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
%     end
    
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.005133762
0.083333333	0.003330533
0.166666667	0.003449309
0.25	0.003434383
0.5	0.004201366
0.75	0.00507484
1	0.005717385
1.5	0.00700626
2	0.008344079
2.5	0.009151168
3	0.009943398
3.5	0.011114025
4	0.012286013
4.5	0.013398254
5	0.014501432
5.5	0.014852448
6	0.015186586
6.5	0.01552254
7	0.015856678
7.5	0.016282307
8	0.016708648
8.5	0.017137317
9	0.017563658
9.5	0.017992327
10	0.018418668
10.5	0.018489584
11	0.018575434
11.5	0.018686093
12	0.018814873
12.5	0.018869596
    ];

%     for idx=1:size(rawData,1)
%         rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
%     end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal

% 15 x 15
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00152	0.0020235	0.002707	0.0033985	0.003975	0.004875	0.00566	0.006315	0.00585275
2	0.002476	0.0031875	0.0037475	0.0042815	0.00468	0.0053155	0.0058435	0.0061515	0.00593875
3	0.0039595	0.0042195	0.0045605	0.0048845	0.0051295	0.0055535	0.005932	0.006051	0.005967
4	0.0046305	0.004927	0.0051045	0.00532	0.0054875	0.0057605	0.006007	0.005983	0.006007
5	0.0052745	0.005435	0.0055615	0.005673	0.00577	0.005908	0.0060255	0.005919	0.0060005
7	0.0058265	0.0058325	0.005865	0.005896	0.0059245	0.0059805	0.006011	0.005796	0.005956
10	0.00601	0.005982	0.0059605	0.0059455	0.005939	0.005949	0.005925	0.0055765	0.00580725
15	0.0057795	0.0057245	0.0056915	0.0056665	0.0056495	0.0056555	0.0056355	0.005237	0.0055145
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a USDBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = USDBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= USDBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
  %% create a LGM2FModel with initial modelParams
% rawDataSwaption = importdata('C:\Data\USDSwaptionVol20160215_15_15.txts');
tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
dH2.tenor = dH1.tenor;

% %KRW
% dHTest = [0.058982775	0.543001377
% 0.061234997	0.56742444
% 0.062961086	0.622386628
% 0.067490351	0.649674014
% 0.075410597	0.63934757
% 0.087663903	0.593591285
% 0.112669112	0.501377655
% 0.117220704	0.537782529
% 0.140692951	0.444873509
% 0.168027009	0.390384507
% 0.182609655	0.410559905
% 0.184911175	0.327958872
% 0.214621342	0.297359459
% 0.213082379	0.268233279
% 0.228595305	0.243594185
% 0.220204578	0.241085585
% 0.27611566	0.202850389
% 0.244581841	0.134958255
% 0.314853582	0.120816977
% 0.317684576	0.109773131
% 0.325368543	0.100229783
% 0.348723657	0.073457815
% 0.31747133	0.033017077
% 0.359049924	0.006573421
% ];

%USD
% dHTest = [0.058982775	0.543001377
% 0.061234997	0.56742444
% 0.062961086	0.622386628
% 0.067490351	0.649674014
% 0.075410597	0.63934757
% 0.087663903	0.593591285
% 0.112669112	0.501377655
% 0.117220704	0.537782529
% 0.140692951	0.444873509
% 0.168027009	0.390384507
% 0.182609655	0.410559905
% 0.184911175	0.327958872
% 0.214621342	0.297359459
% 0.213082379	0.268233279
% 0.228595305	0.243594185
% 0.220204578	0.241085585
% 0.27611566	0.202850389
% 0.244581841	0.134958255
% 0.314853582	0.120816977
% 0.317684576	0.109773131
% 0.325368543	0.100229783
% 0.348723657	0.073457815
% 0.31747133	0.033017077
% 0.359049924	0.006573421
% ];

%YJ
dHTest = [0.05716398	0.571157594
0.031071807	0.574327717
0.051909085	0.566994036
0.084423728	0.504938505
0.110981441	0.505850232
0.126615877	0.512894684
0.138839882	0.520531957
0.140945313	0.476959286
0.153312384	0.490097273
0.168370724	0.528176749
0.157600035	0.399453072
0.165379986	0.40846218
0.156391437	0.371822746
0.172648355	0.324064952
0.180094943	0.268717957
0.13826959	0.218563862
0.178643616	0.167852794
0.176083126	0.155383947
0.204021377	0.129711958
0.221749672	0.114164075
0.181792237	0.103246005
0.197372328	0.072330489
0.215010522	0.033005001
0.200702673	0.006547032
];

dH1.quote = dHTest(:,1);
dH2.quote = dHTest(:,2);


dH1Orig = dH1;
dH2Orig = dH2;

% Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.tenor =  expiry;
Alpha2.tenor =  Alpha1.tenor;
correl.tenor = Alpha1.tenor;
correl.quote = zeros(length(expiry),1);

%swaption only initial parameters

%jaeweol
% AlphaTest = [0.027941111	0.006258382
% 0.021327126	0.006691243
% 0.015009253	0.006502172
% 0.012188678	0.007718057
% 0.010231971	0.009065476
% 0.006437447	0.009477921
% 0.004646654	0.012890028
% 0.004203719	0.024248438
% ];

%YJ

AlphaTest = [0.048471192	0.00036138
0.039781874	0.002321994
0.032116249	0.00382839
0.028154158	0.006146119
0.02589521	0.00807697
0.02743618	0.007265099
0.023870795	0.011836718
0.016056121	0.028747909
];


Alpha1.quote = AlphaTest(:,1);
Alpha2.quote = AlphaTest(:,2);

smoothdH = 1.0;
smoothAlpha = 1.0;
smoothTermCorr =0.1;
targetTermCorr = 0.92;

freq = 4;

modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                        {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% import irModels.*;
% import targetFunctions.*;
                    
USDLGM = LGM2F(IRModel(Model(USDAllMktData,modelParams)));


tic
% out = USDLGM.CalibrateToATMSwaptionSurface(USDBlack,'global',200);
out = USDLGM.CalibrateToATMSwaptionSurfaceNormal(USDBlack,'global',200);
% out = USDLGM.CalibrateToATMSwaptionSurface(USDBlack,'dummy');

dH1Calib = USDLGM.dH1;
dH2Calib = USDLGM.dH2;
Alpha1Calib = USDLGM.Alpha1;
Alpha2Calib = USDLGM.Alpha2;

modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                        {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

USDLGM2FRisky = LGM2FRisky(IRModelRisky(Model(USDAllMktData,modelParamsCalib)));
    

toc
    %% LGM2FRisky model building end
    %% USD model building end

    %% Equity(SNP500) eq model buidling start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.00081111
0.083333333	0.001290309
0.166666667	0.001661613
0.25	0.00218909
0.5	0.002567648
0.75	0.002394088
1	0.002189996
1.5	0.002358275
2	0.002542794
2.5	0.002761162
3	0.002980484
3.5	0.00331556
4	0.003653897
4.5	0.004095603
5	0.004542049
5.5	0.005022558
6	0.005504425
6.5	0.005986293
7	0.006468161
7.5	0.006871943
8	0.00727307
8.5	0.007674198
9	0.008075325
9.5	0.008476453
10	0.00887758
10.5	0.009187056
11	0.009491776
11.5	0.009796497
12	0.010101218
12.5	0.010312606
13	0.010519716
13.5	0.010726826
14	0.010933936
14.5	0.011141045
15	0.011348155
15.5	0.011474448
16	0.011597036
16.5	0.011719625
17	0.011842213
17.5	0.011964801
18	0.012087389
18.5	0.012209977
19	0.012332565
19.5	0.012455154
20	0.012577742
20.5	0.012633118
21	0.012684598
21.5	0.012736078
22	0.012787557
22.5	0.012839037
23	0.012890517
23.5	0.012941997
24	0.012993477
24.5	0.013044957
25	0.013096437
25.5	0.013119813
26	0.013141386
26.5	0.01316296
27	0.013184533
27.5	0.013206106
28	0.01322768
28.5	0.013249253
29	0.013270827
29.5	0.0132924
30	0.013313974
30.5	0.013279634
31	0.013241359
31.5	0.013203084
32	0.013164809
32.5	0.013126534
33	0.013088259
33.5	0.013049984
34	0.01301171
34.5	0.012973435
35	0.01293516
35.5	0.012896885
36	0.01285861
36.5	0.012820335
37	0.01278206
37.5	0.012743785
38	0.01270551
38.5	0.012667235
39	0.01262896
39.5	0.012590685
40	0.01255241
40.5	0.012514135
41	0.01247586
41.5	0.012437586
42	0.012399311
42.5	0.012361036
43	0.012322761
43.5	0.012284486
44	0.012246211
44.5	0.012207936
45	0.012169661
45.5	0.012131386
46	0.012093111
46.5	0.012054836
47	0.012016561
47.5	0.011978286
    ];

%     for idx=1:size(rawData,1)
%         rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
%     end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20201013','USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    SNP500AllMktData =containers.Map('zeroCurve',zeroCurve);
    
    %% import and construct a spot For SNP500
    rawData = 100.0;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20170905','USD','USDSpot'});
    spotMktData = MktData(mktData);
    SNP500AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor SNP500

%     %flat vol surface
%     rawData= [0	1
% 30	0.188967596
% 91	0.1914241
% 183	0.193714376
% 365	0.196564748
% 730	0.19912693
% 1095	0.199867819
% 1460	0.199987438
% 1825	0.19988816
% ];
    
%     rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 30	0.563598379	0.473533935	0.392869833	0.318521146	0.249036108	0.188967596	0.168364957	0.190201906	0.22032859	0.249871726	0.277485257
% 91	0.448827144	0.38485828	0.328059861	0.276592028	0.230079673	0.1914241	0.170801136	0.173774339	0.188367595	0.205877668	0.223500859
% 183	0.381204894	0.333208143	0.291075142	0.253599082	0.220656024	0.193714376	0.176378594	0.171390162	0.175838553	0.184844235	0.195509664
% 365	0.326633182	0.292255908	0.262579089	0.236768379	0.214631495	0.196564748	0.183416818	0.175879389	0.173607419	0.175231997	0.179241998
% 730	0.286104662	0.262500793	0.242505866	0.225456295	0.211032297	0.19912693	0.18974179	0.18287225	0.178398342	0.176038104	0.175392684
% 1095	0.267793961	0.249175185	0.233554125	0.22033538	0.209174324	0.199867819	0.192286064	0.186322712	0.181858841	0.178744306	0.176798491
% 1460	0.256793431	0.241157503	0.228109636	0.217104114	0.207803659	0.199987438	0.193499095	0.188214352	0.184020479	0.180804894	0.178451336
% 1825	0.249272463	0.23565635	0.2243307	0.214790843	0.206715852	0.19988816	0.194150042	0.189378118	0.185467949	0.182325094	0.179860628
%     ];


    rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
53	0.6587	0.5554	0.4626	0.3765	0.2949	0.2192	0.1776	0.1926	0.2223	0.2525	0.2810
88	0.6017	0.5110	0.4297	0.3547	0.2841	0.2199	0.1804	0.1856	0.2080	0.2328	0.2568
116	0.5685	0.4852	0.4107	0.3421	0.2781	0.2203	0.1828	0.1826	0.2004	0.2219	0.2432
207	0.4990	0.4313	0.3712	0.3163	0.2660	0.2214	0.1896	0.1803	0.1878	0.2015	0.2167
298	0.4578	0.3996	0.3480	0.3014	0.2592	0.2222	0.1946	0.1821	0.1835	0.1920	0.2030
389	0.4298	0.3781	0.3325	0.2915	0.2548	0.2228	0.1983	0.1848	0.1825	0.1873	0.1952
571	0.3932	0.3502	0.3125	0.2790	0.2492	0.2236	0.2032	0.1899	0.1840	0.1840	0.1877
753	0.3696	0.3322	0.2996	0.2709	0.2456	0.2238	0.2063	0.1937	0.1865	0.1841	0.1850
935	0.3526	0.3192	0.2903	0.2649	0.2427	0.2237	0.2081	0.1964	0.1888	0.1850	0.1842
1117	0.3395	0.3092	0.2831	0.2603	0.2404	0.2233	0.2092	0.1983	0.1907	0.1862	0.1843
1488	0.3201	0.2943	0.2722	0.2531	0.2365	0.2222	0.2103	0.2007	0.1934	0.1883	0.1852
1852	0.3066	0.2839	0.2646	0.2479	0.2334	0.2210	0.2105	0.2019	0.1951	0.1899	0.1863
        ];

    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20170905','fwdMoneyNess','USD','SNP500VolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
%     impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';


    SNP500AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor SNP500
    rawData = [0    0.0149
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20170905','USD','SNP500DividendCurve'});
    
    mktData('dividendType') = 'dividendYield';
    
    dividendCurveMktData = MktData(mktData);
    
    SNP500AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For SNP500

    rawData = [0    0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20170905','USD','SNP500RepoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    SNP500AllMktData('repoRateCurve') = repoRateCurveMktData;

    %% import and construct a forward Curve For SNP500
    rawData = [30	100
91	100
183	100
365	100
730	100
1095	100
1460	100
1825	100
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20170905','USD','SNP500ForwardCurve'});

%     mktData('useInterpolatedForward') = true;
    mktData('useInterpolatedForward') = false;

    forwardCurveMktData = MktData(mktData);
    SNP500AllMktData('forwardCurve') = forwardCurveMktData;


    %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParamsEQ= containers.Map({'modelName'},{'Black'});
    SNP500Black = EQBlack(EQModel(Model(SNP500AllMktData,modelParamsEQ))); 
    maturities = SNP500Black.volSurface.maturities;
    strikes =    SNP500Black.volSurface.strikes;
    impVolSurface = SNP500Black.volSurface.vol;
    spot = SNP500Black.spot.params('rawData');
    expiry = maturities;

   %% create a DupireModel with initial modelParams
    %Initialize local volatility
    SNP500ImpliedVolSurface = SNP500Black.volSurface;
    localVolSurface = SNP500ImpliedVolSurface;

    modelParamsEQ= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurface,'Dupire'});

    %% vol calibration grid setting
    modelParamsEQ('Kmin') = 0.1;
    modelParamsEQ('Kmax') = 2.5;
    modelParamsEQ('Ns') = 241;

    dK = (modelParamsEQ('Kmax') - modelParamsEQ('Kmin'))/(modelParamsEQ('Ns')-1);
    modelParamsEQ('dK') = dK;

    Ks = zeros(modelParamsEQ('Ns'),1);
    for i=1:modelParamsEQ('Ns')
        Ks(i) = modelParamsEQ('Kmin') + modelParamsEQ('dK')*(i-1);
    end

    modelParamsEQ('Ks') = Ks;

    % restricted interval for calibration target

    modelParamsEQ('numOfCutoffTenor') = length(maturities);

    modelParamsEQ('lowerCutoff') = 0.0;
    modelParamsEQ('upperCutoff') = 2.0;

    lowerCutoffVector = modelParamsEQ('lowerCutoff')*ones(length(maturities),1);
    upperCutoffVector = modelParamsEQ('upperCutoff')*ones(length(maturities),1);

    % 
    % lowerCutoffVector(1) = 0.85;
    % lowerCutoffVector(2) = 0.7;
    % lowerCutoffVector(3) = 0.5;

    % 
    % upperCutoffVector(1) = 1.15;
    % upperCutoffVector(2) = 1.3;
    % upperCutoffVector(3) = 1.5;

    modelParamsEQ('lowerCutoffVector') = lowerCutoffVector;
    modelParamsEQ('upperCutoffVector') = upperCutoffVector;


    % GFType : (Calibration) Forward Scheme option
    %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
    %              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
    %              1  = one step matrix exponential , scaling and squaring with Pade approximation
    %              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
    %              3  = one step Huge Decomposition & multiplication from right RG
    %              4  = multi step Huge Decomposition & multiplication from right RG


    %%
    % % backGFType : (Pricing) backward scheme FD(PDE,GF) option
    % %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
    % %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
    % %              1 = one step matrix exponential , scaling and squaring with Pade approximation
    % %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
    % %              3 =  one step Huge Decomposition & one step Thomas
    % %              4 = multi step Huge Decomposition & multistep thomas
    % %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
    % %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
    % %              34=  one step Huge, daily Thomas(backward)
    % 
    % GFType = -1;
    % backGFType = -1;

    GFType = -1;
    backGFType = -1;

    modelParamsEQ('GFType') = GFType;
    modelParamsEQ('backGFType') = backGFType;


    % mc time steps in days 
%     modelParamsEQ('mcOneTimeStep') = 1095;
    modelParamsEQ('mcOneTimeStep') = 7;
%     modelParamsEQ('NMC') = 2^15;
    
    modelParamsEQ('NMC') = 40000;
%     modelParamsEQ('NMC') = 400;

    %pricingSchemeType = 1 for PDE oneTimesStep
    %pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
    %pricingSchemeType = 4 for one time markov chain monte carlo

    modelParamsEQ('pricingSchemeType') =1;

    modelParamsEQ('useQuasiRandomYN') = 'NO';

    modelParamsEQ('tol') = 0.0001;
    modelParamsEQ('maxIter') = 100;

    % SNP500Dupire = EQDupireSpotGF(EQModel(Model(AllMktData,modelParams)));
    SNP500Dupire = EQCOMDupireSpotGF(EQModel(Model(SNP500AllMktData,modelParamsEQ)));
    % SNP500Dupire = EQDupireSpot(EQModel(Model(AllMktData,modelParams)));
    %SNP500Dupire = EQDupire(EQModel(Model(AllMktData,modelParams)));

    out = SNP500Dupire.CalibrateToVolSurface(SNP500Black,'bootstrap');

    %% create  an EQBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});
%     USDUSDBlack = FXBlack(FXModel(Model(USDUSDAllMktData,modelParams))); 
    %% USDUSD fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'USD','SNP500'}, ...
                            {USDAllMktData, SNP500AllMktData});

    % USD,KTB                    
%     correlationMatrix = ones(2,2);
%     correlationMatrix(1,2) = 0.3;
%     correlationMatrix(2,1) = correlationMatrix(1,2);

    % USD, SNP500
    correlationMatrix = ones(2, 2);
    correlationMatrix(1,2) = 0.405144;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelNameMap = containers.Map({'USD','SNP500'},{USDLGM2FRisky, SNP500Dupire});
    modelParams = containers.Map({'modelNameMap'}, {modelNameMap});
    % correlationNames = {'USD','USD'};
    modelParams('stochasticModelNames') = {'USD','SNP500'};
    
    assetAndDealInputMap = containers.Map({'IR1','Equity1'},{'USD', 'SNP500'});
    modelParams('assetAndDealInputMap') = {assetAndDealInputMap};
    
    modelParams('correlationMatrix') = correlationMatrix;
    modelParams('refModelName') = 'USD';
    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    modelParams('eqVolCalibrated') = false;
    
    % a new thing , we need securityInfo to generate timeStep accordingly
    % so that we can calculate hybrid local vol easily
     modelParams('mcOneTimeStep') = 7;
    modelParams('securityInfo') = rangeAccrualParams;
    modelParams('useBrigoCorrYN') = 'simple_adjust';
    
%     USDCK1FRiskySNP500 = MultiAssetModelCK1FEQDupire(multiAssetMktData, modelParams);
    USDLGM2FRiskySNP500 = MultiAssetModelLGM2FEQDupire(multiAssetMktData, modelParams);
    %% MultiAssetModel building end
   
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    % weekly step calculator
    % pricer = Pricer(rangeAccrual,USDLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);
    
    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,USDLGMRisky,numMethodAMC,true);
    % pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(rangeAccrual,USDLGM1FRiskyUSDLGM1F,numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(rangeAccrual, USDCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(rangeAccrual, USDCK1FRiskyUSDCK1F, numMethodAMC, false);
%     pricer = DailyWeeklyRiskyPricer_CDAndSNP500RA(rangeAccrual, USDCK1FRiskySNP500, numMethodAMC, true);
%     pricer = DailyWeeklyRiskyPricer_CDAndSNP500RAHyb(rangeAccrual, USDCK1FRiskySNP500, numMethodAMC, true);
%       pricer = DailyWeeklyRiskyPricer_CDRA2(rangeAccrual, USDCK1FRisky, numMethodAMC, false);
%       pricer = DailyWeeklyRiskyPricer_CDAndSNP500RAHyb(rangeAccrual, USDLGM2FRiskySNP500, numMethodAMC, true);
%       pricer = DailyWeeklyRiskyPricer_CDAndSNP500RAHyb(rangeAccrual, USDLGM2FRiskySNP500, numMethodAMC, true);
      pricer = DailyWeeklyRiskyPricer_CMSSpreadAndEquityDualRAPutAtMat(rangeAccrual, USDLGM2FRiskySNP500, numMethodAMC, true);
     
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH, 'PSANo');
%     pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
    pricingOutMC1 = pricer.computePriceMCCallableGenericReg2(valueDateH,'PSANo');
%     pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3 = 1:length(pricingOutMC1.nonCall.cashflow_npv_e)
%     for idx3 = 1:length(pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx2, idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
%         closedPrice_cashflowNPV(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
%         
%         closedPrice_fwdCD(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdCD(idx3);
%         closedPrice_fwdKTB(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdKTB(idx3);
%         closedPrice_fwdSpread(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSpread(idx3);
%         closedPrice_fwdMu(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdMu(idx3);
%         closedPrice_fwdSigma(idx2, idx3) = pricingOutClosed.pricerInfo.nonCall.fwdSigma(idx3);
%                 
    end
end

aaa =1.0;


In [ ]:


testRoughHestonPade.m

clc;clear;
params0 = struct('al',0.55,'lam',0,'eta',0.8,'rho',-0.65,'H',0.05);
xi = @(s) (0.025 + 0.*s);
params0.nu = params0.eta.*sqrt(2.0.*params0.al-1).*gamma(params0.al);
kk = -0.4:0.1:0.4;
kk = 0.5:0.1:1.5;
kk = log(kk);

pade = RoughHestonPade;

dhPade = @(params,a,x) (pade.dhPade33(params,a,x));
% dhPade = pade.dhPade33;

n = 500;
% phiPadeRaw = @(u,t) (pade.phiRoughHestonDhApproxRaw(params0,xi,dhPade,n,u,t));
phiPade = @(u,t) (pade.phiRoughHestonDhApprox(params0,xi,dhPade,n,u,t));
lewis = Lewis;
impvRoughHestonPade = lewis.impvolPhi(phiPade);

impvolOut1Y = impvRoughHestonPade(kk,30/365);
aaa = 1.0;
figure
plot(kk,impvolOut1Y)

% phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
% impvHestonFun = lewis.impvolPhi(phiHestionFun);

In [ ]:

testRoughHestonAdamsAndPade.m
clc;clear;
params0 = struct('al',0.55,'lam',0,'eta',0.8,'rho',-0.65,'H',0.05);
xi = @(s) (0.025 + 0.*s);
params0.nu = params0.eta.*sqrt(2.0.*params0.al-1).*gamma(params0.al);
kk = 0.5:0.1:1.5;
% kk = 0.9;
kk = log(kk);

adams1 = RoughHestonAdams;
nSteps = 500;
volAdamsFun = adams1.impliedVolRoughHeston(params0, xi, nSteps);
volAdams = volAdamsFun(kk,30/365);
aaa = 1.0;

pade = RoughHestonPade;

dhPade = @(params,a,x) (pade.dhPade33(params,a,x));

n = 500;
phiPade = @(u,t) (pade.phiRoughHestonDhApprox(params0,xi,dhPade,n,u,t));
lewis = Lewis;
impvRoughHestonPade = lewis.impvolPhi(phiPade);

impvolOut1Y = impvRoughHestonPade(kk,30/365);
aaa = 1.0;
figure
plot(kk,impvolOut1Y)
hold on
plot(kk, volAdams)
bbb = 1.0;
% phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
% impvHestonFun = lewis.impvolPhi(phiHestionFun);

In [ ]:
testRoughHestonAdams.m


clc;clear;
params0 = struct('al',0.55,'lam',0,'eta',0.8,'rho',-0.65,'H',0.05);
xi = @(s) (0.025 + 0.*s);
params0.nu = params0.eta.*sqrt(2.0.*params0.al-1).*gamma(params0.al);
kk = 0.5:0.1:1.5;
% kk = 0.9;
kk = log(kk);

adams1 = RoughHestonAdams;
nSteps = 500;
volAdamsFun = adams1.impliedVolRoughHeston(params0, xi, nSteps);
volAdams = volAdamsFun(kk,30/365);
aaa = 1.0;

pade = RoughHestonPade;

dhPade = @(params,a,x) (pade.dhPade33(params,a,x));

n = 500;
phiPade = @(u,t) (pade.phiRoughHestonDhApprox(params0,xi,dhPade,n,u,t));
lewis = Lewis;
impvRoughHestonPade = lewis.impvolPhi(phiPade);

impvolOut1Y = impvRoughHestonPade(kk,30/365);
aaa = 1.0;
figure
plot(kk,impvolOut1Y)
hold on
plot(kk, volAdams)
bbb = 1.0;
% phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
% impvHestonFun = lewis.impvolPhi(phiHestionFun);

In [ ]:
testMariaDB.m

clc;
clear;
dbName = 'Investar';
userName = 'root';
passWord='dws4004!';
javaclasspath ...
('D:\db_jdbc\mariadb-java-client-2.7.6.jar');

props = java.util.Properties;
props.setProperty('user','root');
props.setProperty('password','dws4004!');

driver = org.mariadb.jdbc.Driver;
% dbUrl = 'jdbc:mariadb://127.0.0.1:3306/Investar';
dbUrl= strcat('jdbc:mariadb://127.0.0.1:3306/','Investar');

conn = driver.connect(dbUrl,props);
aaa = 1.0;

% conn = database(dbName,userName,passWord,driver,dbUrl);
% sql = 'SELECT min(last_update) FROM company_info';
% sql = 'SELECT * FROM company_info';
sql = sprintf('SELECT * FROM daily_price WHERE code = ''%s'' and date >= ''%s'' and date <= ''%s''','005930','20190-01-02','2022-08-09');

ps = conn.prepareStatement(sql);
rs = ps.executeQuery();
aaa  = 1.0;
count = 0;
result =struct;

while rs.next()
    count= count +1;
    result(count).var1 = char(rs.getString(1));
    result(count).var2 = char(rs.getString(2));
    result(count).var3 = char(rs.getString(3));
    result(count).var4 = char(rs.getString(4));
    result(count).var5 = char(rs.getString(5));
    result(count).var6 = char(rs.getString(6));
    result(count).var7 = char(rs.getString(7));
    result(count).var8 = char(rs.getString(8));
    
end


% while rs.next()
%     count= count +1;
%     result(count).var1 = char(rs.getString(1));
%     result(count).var2 = char(rs.getString(2));
% end
rs.close();
conn.close();
aaa = 1.0;


curs = exec(conn,'SELECT max(last_update) FROM company_info');
curs = fetch(curs);

curs.Data

close(curs);
close(conn);
aaa = 1.0;



In [ ]:
testMandelBrotVanNess.m


H = 0.75;
integrand = @(u) (((1+u).^(H-1/2) -u.^(H-1/2)).^2);

aa = integral(integrand,0,inf);
bb = integral(integrand,0,1e10);

cc = 0;

In [ ]:
testHestonCharacteristicFunction.m


clc;clear;
subHeston = struct('lambda', 0.6067,'rho', -0.7571,'eta', 0.2928,'vbar', 0.0707,'v',0.0654);

lewis = Lewis;
phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
impvHestonFun = lewis.impvolPhi(phiHestionFun);
impvHestonFunRaw = lewis.impvolPhiRaw(phiHestionFun);

K = 0.5:0.1:1.5;
% K = 0.5;


k = log(K);

if k > 0
    ddd = 1.0;
end

tic
impvolOut1Y = impvHestonFun(k,91/365);
toc

figure
plot(K,impvolOut1Y)
% tic
% impvolOut1Y2 = impvHestonFunRaw(k,1);
% toc

dd = 0.0;

function out = phiHeston(params,u,t)
    
    lambda = params.lambda;
    rho = params.rho;
    eta = params.eta;
    vbar= params.vbar;
    v = params.v;
    
    al = -u.*u./2 - 1i.*u./2;
    bet= lambda - rho.*eta.*1i.*u;
    gam = eta.^2./2;
    d = sqrt(bet.*bet - 4.*al.*gam);
    rp = (bet + d)./(2.*gam);
    rm = (bet - d)./(2.*gam);
    g  = rm./rp;
    D  = rm .* (1 - exp(-d.*t))./ (1 - g.*exp(-d.*t));
    C  = lambda .* (rm .* t - 2./eta.^2 * log( (1 - g.*exp(-(d.*t)))./(1 - g) ) );
    
    out = exp(C.*vbar + D.*v);
    
end


In [ ]:

testHeston111.m

clc;clear;
subHeston = struct('lambda', 0.6067,'rho', -0.7571,'eta', 0.2928,'vbar', 0.0707,'v',0.0654);

lewis = Lewis;
phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
impvHestonFun = lewis.impvolPhi(phiHestionFun);
impvHestonFunRaw = lewis.impvolPhiRaw(phiHestionFun);

K = 0.5:0.1:1.5;
% K = 0.5;


k = log(K);

if k > 0
    ddd = 1.0;
end

tic
impvolOut1Y = impvHestonFun(k,91/365);
toc

figure
plot(K,impvolOut1Y)
% tic
% impvolOut1Y2 = impvHestonFunRaw(k,1);
% toc

dd = 0.0;

function out = phiHeston(params,u,t)
    
    lambda = params.lambda;
    rho = params.rho;
    eta = params.eta;
    vbar= params.vbar;
    v = params.v;
    
    al = -u.*u./2 - 1i.*u./2;
    bet= lambda - rho.*eta.*1i.*u;
    gam = eta.^2./2;
    d = sqrt(bet.*bet - 4.*al.*gam);
    rp = (bet + d)./(2.*gam);
    rm = (bet - d)./(2.*gam);
    g  = rm./rp;
    D  = rm .* (1 - exp(-d.*t))./ (1 - g.*exp(-d.*t));
    C  = lambda .* (rm .* t - 2./eta.^2 * log( (1 - g.*exp(-(d.*t)))./(1 - g) ) );
    
    out = exp(C.*vbar + D.*v);
    
end


In [ ]:
testAFVSimulationRSQEAndHQEAndRoughHestonAdamsAndPade.m

clc;clear;
params0 = struct('al',0.55,'lam',0,'eta',0.8,'rho',-0.65,'H',0.05);
xi = @(s) (0.025 + 0.*s);
params0.nu = params0.eta.*sqrt(2.0.*params0.al-1).*gamma(params0.al);

afvSim = AFVSimulation;
% paths = 100000;
paths = 100000;
% paths = 1000;

steps = 500;
% steps = 100;

% expiries = 30/365;
expiries = 1;

rsqeSim = afvSim.RSQESim(params0, xi,paths, steps, expiries);
hqeSim = afvSim.HQESim(params0, xi,paths, steps, expiries);

spots100 = exp(rsqeSim.x);
spots100_hqe = exp(hqeSim.x);

figure(123)
hist(spots100,200)
figure(124)
hist(spots100_hqe,200)

blackFormula = BlackFormula;
Sf = spots100;
AK = mean(Sf)*0.9;

Sf_hqe = spots100_hqe;
AK_hqe = mean(Sf_hqe)*0.9;

smile100 = @(k) (blackFormula.ivS(spots100,expiries,mean(spots100)*exp(k)));
smile100_hqe = @(k) (blackFormula.ivS(spots100_hqe,expiries,mean(spots100_hqe)*exp(k)));

kk = 0.5:0.1:1.5;
kk = log(kk);
impVol_rsqe = smile100(kk);
impVol_hqe = smile100_hqe(kk);

figure
plot(kk,impVol_rsqe)

hold on
plot(kk,impVol_hqe)

ccc = 1.0;
aaa = 1.0;

% kk = 0.5:0.1:1.5;
% % kk = 0.9;
% kk = log(kk);
% 
adams1 = RoughHestonAdams;
nSteps = 500;
volAdamsFun = adams1.impliedVolRoughHeston(params0, xi, nSteps);
volAdams = volAdamsFun(kk,expiries);
hold on
plot(kk,volAdams);


aaa = 1.0;

pade = RoughHestonPade;

dhPade = @(params,a,x) (pade.dhPade33(params,a,x));

n = 500;
phiPade = @(u,t) (pade.phiRoughHestonDhApprox(params0,xi,dhPade,n,u,t));
lewis = Lewis;
impvRoughHestonPade = lewis.impvolPhi(phiPade);

impvolOut1Y = impvRoughHestonPade(kk,expiries);

hold on
plot(kk,impvolOut1Y)

legend('RSQE MC100000','HQE MC100000','Adams scheme','(3,3) Pade Approximation')

aaa = 1.0;

% hold on
% plot(kk, volAdams)
% bbb = 1.0;
% phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
% impvHestonFun = lewis.impvolPhi(phiHestionFun);


In [ ]:
testAFVSimulationHQEAndRiemannSumQEAndRoughHestonAdamsAndPade.m


clc;clear;
params0 = struct('al',0.55,'lam',0,'eta',0.8,'rho',-0.65,'H',0.05);
xi = @(s) (0.025 + 0.*s);
params0.nu = params0.eta.*sqrt(2.0.*params0.al-1).*gamma(params0.al);

afvSim = AFVSimulation;
% paths = 100000;
paths = 100000;

steps = 500;
% expiries = 30/365;
expiries = 1;

% rsqeSim = afvSim.RSQESim(params0, xi,paths, steps, expiries);
hqeSim = afvSim.HQESim(params0, xi,paths, steps, expiries);


spots100 = exp(rsqeSim.x);
figure(123)
hist(spots100,200)

blackFormula = BlackFormula;
Sf = spots100;
AK = mean(Sf)*0.9;
vol1 = blackFormula.ivS(Sf,expiries,AK);

smile100 = @(k) (blackFormula.ivS(spots100,expiries,mean(spots100)*exp(k)));

kk = 0.5:0.1:1.5;
kk = log(kk);
impVolRsqe = smile100(kk);

figure
plot(kk,impVolRsqe)

ccc = 1.0;
aaa = 1.0;

% kk = 0.5:0.1:1.5;
% % kk = 0.9;
% kk = log(kk);
% 
adams1 = RoughHestonAdams;
nSteps = 500;
volAdamsFun = adams1.impliedVolRoughHeston(params0, xi, nSteps);
volAdams = volAdamsFun(kk,expiries);
hold on
plot(kk,volAdams);


aaa = 1.0;

pade = RoughHestonPade;

dhPade = @(params,a,x) (pade.dhPade33(params,a,x));

n = 500;
phiPade = @(u,t) (pade.phiRoughHestonDhApprox(params0,xi,dhPade,n,u,t));
lewis = Lewis;
impvRoughHestonPade = lewis.impvolPhi(phiPade);

impvolOut1Y = impvRoughHestonPade(kk,expiries);

hold on
plot(kk,impvolOut1Y)

aaa = 1.0;

% hold on
% plot(kk, volAdams)
% bbb = 1.0;
% phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
% impvHestonFun = lewis.impvolPhi(phiHestionFun);

In [ ]:
testAFVSimulationAndRoughHestonAdamsAndPadeCrossCheck.m

clc;clear;

rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
81	0.591573	0.503071	0.417725	0.336232	0.270457	0.211908	0.168043	0.170101	0.187082	0.207676	0.229034
109	0.562976	0.477712	0.39804	0.327322	0.268269	0.21295	0.169561	0.162384	0.174212	0.191371	0.209742
200	0.48943	0.420322	0.357842	0.305338	0.259318	0.214615	0.175982	0.157034	0.157095	0.164936	0.17618
291	0.441731	0.382138	0.331833	0.289186	0.250839	0.212886	0.179106	0.158509	0.153481	0.155438	0.162007
382	0.405318	0.353314	0.310619	0.273361	0.239282	0.205502	0.176355	0.157992	0.150039	0.150269	0.15517
564	0.377121	0.333803	0.293922	0.259843	0.22989	0.202559	0.179758	0.163916	0.155801	0.153783	0.156323
753	0.350113	0.311544	0.277642	0.247328	0.220153	0.196325	0.176888	0.163524	0.157063	0.156224	0.158833
935	0.337495	0.299169	0.265051	0.23653	0.21305	0.194026	0.178288	0.167614	0.160976	0.158949	0.160193
    ];

kkRawData = [
    0.502025407669468	0.602430489203362	0.702835570737255	0.803240652271149	0.903645733805043	1.00405081533894	1.10445589687283	1.20486097840672	1.30526605994062	1.40567114147451	1.50607622300840
    0.502331445977285	0.602797735172743	0.703264024368200	0.803730313563657	0.904196602759114	1.00466289195457	1.10512918115003	1.20559547034549	1.30606175954094	1.40652804873640	1.50699433793186
    0.503722259439991	0.604466711327990	0.705211163215988	0.805955615103986	0.906700066991984	1.00744451887998	1.10818897076798	1.20893342265598	1.30967787454398	1.41042232643198	1.51116677831997
    0.504616786082473	0.605540143298967	0.706463500515462	0.807386857731956	0.908310214948451	1.00923357216495	1.11015692938144	1.21108028659793	1.31200364381443	1.41292700103092	1.51385035824742
    0.513374840342606	0.616049808411127	0.718724776479648	0.821399744548169	0.924074712616690	1.02674968068521	1.12942464875373	1.23209961682225	1.33477458489077	1.43744955295930	1.54012452102782
    0.512702795581519	0.615243354697823	0.717783913814126	0.820324472930430	0.922865032046734	1.02540559116304	1.12794615027934	1.23048670939565	1.33302726851195	1.43556782762825	1.53810838674456
    0.520098294925050	0.624117953910059	0.728137612895069	0.832157271880079	0.936176930865089	1.04019658985010	1.14421624883511	1.24823590782012	1.35225556680513	1.45627522579014	1.56029488477515
    0.518864511232013	0.622637413478416	0.726410315724818	0.830183217971221	0.933956120217623	1.03772902246403	1.14150192471043	1.24527482695683	1.34904772920323	1.45282063144964	1.55659353369604
    ];

impVolrsqeTest = zeros(size(kkRawData));
impVolhqeTest = zeros(size(kkRawData));
impVolAdamsTest = zeros(size(kkRawData));
impVolPadeTest = zeros(size(kkRawData));


expiryRawData =  rawData(2:end,1);
marketVolSurface = rawData(2:end,2:end);

expirySize = length(expiryRawData);

params0 = struct('al',0.600044293714652,'lam',0,'eta',0.458266093945180,'rho',-0.764593637956381,'H',0.100044293714652);
params0.nu = 0.458266093945180;
params0.eta = params0.nu/(sqrt(2.0.*params0.al-1).*gamma(params0.al));
xi = @(s) (0.071923356959668 + 0.*s);

afvSim = AFVSimulation;
% paths = 100000;
paths = 100000;

for idx=1:expirySize
    
    steps = expiryRawData(idx);
    expiries = expiryRawData(idx)/365;
    

    rsqeSim = afvSim.RSQESim(params0, xi,paths, steps, expiries);
    hqeSim = afvSim.HQESim(params0, xi,paths, steps, expiries);


    spots100 = exp(rsqeSim.x);
    spots100h = exp(hqeSim.x);

    blackFormula = BlackFormula;

    Sf = spots100;
    Sfh = spots100h;

    smile100 = @(k) (blackFormula.ivS(spots100,expiries,mean(spots100)*exp(k)));

    smile100h = @(k) (blackFormula.ivS(spots100h,expiries,mean(spots100h)*exp(k)));

    kk = kkRawData(idx,:);

    kk = log(kk);

    % Rsqe MC Vanilla Option's implied vol

    impVolRsqe = smile100(kk);
    impVolrsqeTest(idx,:) = impVolRsqe;

    impVolhqe = smile100h(kk);
    impVolhsqeTest(idx,:) = impVolhqe;

    aaa = 1.0;

    adams1 = RoughHestonAdams;
    nSteps = steps;
    volAdamsFun = adams1.impliedVolRoughHeston(params0, xi, nSteps);

    % Closed Form with Adams Scheme

    volAdams = volAdamsFun(kk,expiries);
    impVolAdamsTest(idx,:) = volAdams;

    aaa = 1.0;

    pade = RoughHestonPade;

    dhPade = @(params,a,x) (pade.dhPade33(params,a,x));

    n = steps;
    phiPade = @(u,t) (pade.phiRoughHestonDhApprox(params0,xi,dhPade,n,u,t));
    lewis = Lewis;
    impvRoughHestonPade = lewis.impvolPhi(phiPade);

    % closed form with Pade's approximation
    impvolOutPade = impvRoughHestonPade(kk,expiries);

    impVolPadeTest(idx,:) = impvolOutPade;
    
    
    aaa = 1.0;
    
end


aaa = 1.0;

% hold on
% plot(kk, volAdams)
% bbb = 1.0;
% phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
% impvHestonFun = lewis.impvolPhi(phiHestionFun);

In [ ]:

testAFVSimulationAndRoughHestonAdamsAndPade.m

clc;clear;
params0 = struct('al',0.55,'lam',0,'eta',0.8,'rho',-0.65,'H',0.05);
xi = @(s) (0.025 + 0.*s);
params0.nu = params0.eta.*sqrt(2.0.*params0.al-1).*gamma(params0.al);

afvSim = AFVSimulation;
% paths = 100000;
paths = 100000;
% paths = 1000;

steps = 500;
% steps = 100;

% expiries = 30/365;
expiries = 1;

% rsqeSim = afvSim.RSQESim(params0, xi,paths, steps, expiries);
hqeSim = afvSim.HQESim(params0, xi,paths, steps, expiries);
% HQESim(obj, params, xi,paths, steps, expiry)

% spots100 = exp(rsqeSim.x);
spots100 = exp(hqeSim.x);

figure(123)
hist(spots100,200)

blackFormula = BlackFormula;
Sf = spots100;
AK = mean(Sf)*0.9;
vol1 = blackFormula.ivS(Sf,expiries,AK);

smile100 = @(k) (blackFormula.ivS(spots100,expiries,mean(spots100)*exp(k)));

kk = 0.5:0.1:1.5;
kk = log(kk);
impVolRsqe = smile100(kk);

figure
plot(kk,impVolRsqe)

ccc = 1.0;
aaa = 1.0;

% kk = 0.5:0.1:1.5;
% % kk = 0.9;
% kk = log(kk);
% 
adams1 = RoughHestonAdams;
nSteps = 500;
volAdamsFun = adams1.impliedVolRoughHeston(params0, xi, nSteps);
volAdams = volAdamsFun(kk,expiries);
hold on
plot(kk,volAdams);


aaa = 1.0;

pade = RoughHestonPade;

dhPade = @(params,a,x) (pade.dhPade33(params,a,x));

n = 500;
phiPade = @(u,t) (pade.phiRoughHestonDhApprox(params0,xi,dhPade,n,u,t));
lewis = Lewis;
impvRoughHestonPade = lewis.impvolPhi(phiPade);

impvolOut1Y = impvRoughHestonPade(kk,expiries);

hold on
plot(kk,impvolOut1Y)

aaa = 1.0;

% hold on
% plot(kk, volAdams)
% bbb = 1.0;
% phiHestionFun = @(u,t) (phiHeston(subHeston,u,t));
% impvHestonFun = lewis.impvolPhi(phiHestionFun);

In [ ]:

test_YieldCurveGlobalCalib_SOFR_20220727_spot2D.m

clc
clear
tic

% valueDate = '2022-07-27';
valueDate = '2022-07-29';


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;
% useSimpleTeleScopingFlag = false;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);
USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

jointCalendar = JointCalendar({USGTCalendar,USFDCalendar});


%% import and construct an discount curve USD SOFR Curve

%loong tenor(~50Y) test data

dataType= {
'Deposit'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
};

dataTenor = {
'SN'
'1W'
'2W'
'3W'
'1M'
'2M'
'3M'
'6M'
'9M'
'12M'
'2Y'
'3Y'
'4Y'
'5Y'
'6Y'
'7Y'
'8Y'
'9Y'
'10Y'
'12Y'
'15Y'
'20Y'
'25Y'
'30Y'
'40Y'
'50Y'
};

dataRate = {
0.01530000
0.02285400
0.02287800
0.02287700
0.02289550
0.02359000
0.02542000
0.02895500
0.03030000
0.03074400
0.02949500
0.02761300
0.02640500
0.02579250
0.02556100
0.02548250
0.02549300
0.02559500
0.02578650
0.02628850
0.02684000
0.02687000
0.02612600
0.02534000
0.02309000
0.02071000
};


% construct iborIndex
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'1D','USD',0,USGTCalendar,'Following','Act360',false,dummyCurve});
                    
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',0,USGTCalendar,'ModifiedFollowing','Act360',false,dummyCurve});
                    
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',0,jointCalendar,'Following','Act360',false,dummyCurve});

% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',2,jointCalendar,'Following','Act360',false,dummyCurve});
                    
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','KRW',2,Calendar(),'ModifiedFollowing','Act360',false,dummyCurve});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;
IborIndexParams('useSimpleTeleScoping') = useSimpleTeleScopingFlag;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



% overnight index itself is nothing but 1D IborIndex
overnightIndex = IborIndex(IborIndexParams);

rateHelpers = {};

for i=1:length(dataRate)
    if strcmp(dataType{i},'Deposit')
        depositRateHelperParams = containers.Map({'IborIndexParams'}, ...
                        {IborIndexParams});

%         calcCalendar = USFDCalendar;
        calcCalendar = USGTCalendar;
        
        depositRateHelperParams('calcCalendar') = calcCalendar;
        
        % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
%         spotLag = 2;
        spotLag = 2;
        
        
        depositRateHelperParams('spotLag') = spotLag;
        
        depositRateHelperParams('rate') = dataRate{i};
        depositRateHelperParams('rateTenor') = dataTenor{i};
        depositRateHelper = DepositRateHelper(depositRateHelperParams);
    
        rateHelpers{end+1} = depositRateHelper;
        
    elseif strcmp(dataType{i},'OIS')
        
        if i== 17 || i == 16
            bbb = 1.0;
        end
        % cashflow interval(?)
        fixedLegTenor = '1Y';
        fixedLegConvention = 'ModifiedFollowing';
        fixedLegDayCounter = 'Act360';
        
        floatingLegTenor = '1Y';
        floatingLegConvention = 'ModifiedFollowing';
        floatingLegDayCounter = 'Act360';
        
        % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
%         spotLag = 2;
        spotLag = 2;
        
        
        % payLag : paymentDate Coupon of Legs of Swap:  paymentDate = endDate + 2D
        
        payLag  = 2;
%         payLag  = 0;

        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
        
        calcCalendar = USFDCalendar;

        oisRateHelperParams = containers.Map({'calcCalendar','spotLag','payLag','fixedLegTenor','fixedLegConvention','fixedLegDayCounter',...
                                'floatingLegTenor','floatingLegConvention','floatingLegDayCounter','overnightIndex','discountCurve'}, ...
                                {calcCalendar,spotLag,payLag, fixedLegTenor,fixedLegConvention,fixedLegDayCounter,...
                                 floatingLegTenor,floatingLegConvention,floatingLegDayCounter,overnightIndex,dummyCurve});

%         oisRateHelperParams('useRounding') = useRounding;
        
        oisRateHelperParams('rate') = dataRate{i};
        oisRateHelperParams('rateTenor') = dataTenor{i};
        oisRateHelper = OISRateHelper(oisRateHelperParams);
        rateHelpers{end+1} = oisRateHelper;
        
    elseif strcmp(dataType{i},'Swap')
        fixedLegTenor = '6M';
        fixedLegConvention = 'ModifiedFollowing';
        fixedLegDayCounter = '30360';

        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

        swapRateHelperParams = containers.Map({'fixedLegTenor','fixedLegConvention','fixedLegDayCounter','IborIndex','discountCurve'}, ...
                                {fixedLegTenor,fixedLegConvention,fixedLegDayCounter,KRW_Libor_3M,dummyCurve});


        swapRateHelperParams('rate') = dataRate{i};
        swapRateHelperParams('rateTenor') = dataTenor{i};
        swapRateHelper = SwapRateHelper(swapRateHelperParams);
        rateHelpers{end+1} = swapRateHelper;
    end
end

% forward & discount zeroCurve
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

yieldCurveParams = containers.Map({'rateHelpers','forwardCurve','discountCurve'},{rateHelpers,dummyCurve,dummyCurve1});

% tic
% yieldCurve = YieldCurve(yieldCurveParams);
yieldCurve = YieldCurveOIS(yieldCurveParams);

out = yieldCurve.BootStrap();
% out = yieldCurve.GlobalCalib();

toc
ddd = 1.0;

%% discount curve bootstrapping end USD SOFR Curve

% %% forward curve bootstrapping start USD SOFR Curve(Libor3M Curve)
% %short tenor(~2Y) test data
% dataType_fwd= {'Deposit'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% };
% 
% dataTenor_fwd = {'ON'
% '1W'
% '2W'
% '3W'
% '1M'
% '2M'
% '3M'
% '4M'
% '5M'
% '6M'
% '7M'
% '8M'
% '9M'
% '10M'
% '11M'
% '12M'
% '18M'
% '2Y'
% };
% 
% dataRate_fwd = {0.0005
% 0.0004356
% 0.000432
% 0.000431
% 0.000425
% 0.000437
% 0.00044
% 0.000437
% 0.000444
% 0.000448
% 0.000472
% 0.0004735
% 0.0004835
% 0.0005195
% 0.0005495
% 0.0005895
% 0.000997
% 0.001733
% };
% 
% %loong tenor(~50Y) test data
% 
% dataType_fwd= {'Deposit'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% };
% 
% dataTenor_fwd = {'ON'
% '1W'
% '2W'
% '3W'
% '1M'
% '2M'
% '3M'
% '4M'
% '5M'
% '6M'
% '7M'
% '8M'
% '9M'
% '10M'
% '11M'
% '12M'
% '18M'
% '2Y'
% '3Y'
% '4Y'
% '5Y'
% '6Y'
% '7Y'
% '8Y'
% '9Y'
% '10Y'
% '12Y'
% '15Y'
% '20Y'
% '25Y'
% '30Y'
% '40Y'
% '50Y'
% };
% 
% dataRate_fwd = {0.0005
% 0.0004356
% 0.000432
% 0.000431
% 0.000425
% 0.000437
% 0.00044
% 0.000437
% 0.000444
% 0.000448
% 0.000472
% 0.0004735
% 0.0004835
% 0.0005195
% 0.0005495
% 0.0005895
% 0.000997
% 0.001733
% 0.0035535
% 0.00546
% 0.00707
% 0.00845
% 0.009575
% 0.01047
% 0.011205
% 0.011815
% 0.01279
% 0.013734
% 0.01457
% 0.0147565
% 0.01479
% 0.014144
% 0.01331
% };
% 
% % construct iborIndex
% dummyCurve_fwd = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% 
% IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',0,USGTCalendar,'ModifiedFollowing','Act360',false,dummyCurve_fwd});
%                     
% % IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
% %                         {'1D','USD',0,jointCalendar,'Following','Act360',false,dummyCurve});
% 
% % IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
% %                         {'1D','USD',2,jointCalendar,'Following','Act360',false,dummyCurve});
%                     
% % IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
% %                         {'3M','KRW',2,Calendar(),'ModifiedFollowing','Act360',false,dummyCurve});
% 
% IborIndexParams_fwd('forecastTodaysFixing') = true;
% IborIndexParams_fwd('useRounding') = useRounding;
% 
% % overnight index itself is nothing but 1D IborIndex
% overnightIndex_fwd = IborIndex(IborIndexParams_fwd);
% 
% rateHelpers_fwd = {};
% 
% for i=1:length(dataRate_fwd)
%     if strcmp(dataType_fwd{i},'Deposit')
%         depositRateHelperParams_fwd = containers.Map({'IborIndexParams'}, ...
%                         {IborIndexParams_fwd});
% 
%         calcCalendar = USFDCalendar;
% %         calcCalendar = USGTCalendar;
%         
%         depositRateHelperParams_fwd('calcCalendar') = calcCalendar;
%         
%         depositRateHelperParams_fwd('rate') = dataRate_fwd{i};
%         depositRateHelperParams_fwd('rateTenor') = dataTenor_fwd{i};
%         depositRateHelper_fwd = DepositRateHelper(depositRateHelperParams_fwd);
%     
%         rateHelpers_fwd{end+1} = depositRateHelper_fwd;
%         
%     elseif strcmp(dataType_fwd{i},'OIS')
%         
%         % cashflow interval(?)
%         fixedLegTenor = '1Y';
%         fixedLegConvention = 'ModifiedFollowing';
%         fixedLegDayCounter = 'Act360';
%         
%         floatingLegTenor = '1Y';
%         floatingLegConvention = 'ModifiedFollowing';
%         floatingLegDayCounter = 'Act360';
%         
%         % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
%         spotLag = 2;
%         
%         % payLag : paymentDate Coupon of Legs of Swap:  paymentDate = endDate + 2D
%         
%         payLag  = 2;
% 
% %         dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
%         % from bootStrapping SOFR Curve we have bootStrapped DiscountCurve
%         % zeroRates and we will use it for discountingCurve while
%         % bootStrapping forwardCurve
%         
%         discontCurveRawData = yieldCurve.forwardCurve.params('rawData');
%         mktData_fwd = containers.Map({'rawData','asOfDate'}, ...
%                             {discontCurveRawData,valueDateH});
%         zeroCurveMktData_fwd = MktData(mktData_fwd);
%         discountCurve_1 =  ZeroCurve(zeroCurveMktData_fwd);
%         
%         
%         calcCalendar = USFDCalendar;
% 
%         oisRateHelperParams_fwd = containers.Map({'calcCalendar','spotLag','payLag','fixedLegTenor','fixedLegConvention','fixedLegDayCounter',...
%                                 'floatingLegTenor','floatingLegConvention','floatingLegDayCounter','overnightIndex','discountCurve'}, ...
%                                 {calcCalendar,spotLag,payLag, fixedLegTenor,fixedLegConvention,fixedLegDayCounter,...
%                                  floatingLegTenor,floatingLegConvention,floatingLegDayCounter,overnightIndex_fwd,discountCurve_1});
%         
%         oisRateHelperParams_fwd('rate') = dataRate_fwd{i};
%         oisRateHelperParams_fwd('rateTenor') = dataTenor_fwd{i};
%         oisRateHelper_fwd = OISRateHelper_DS(oisRateHelperParams_fwd);
%         rateHelpers_fwd{end+1} = oisRateHelper_fwd;
%         
%     elseif strcmp(dataType{i},'Swap')
% %         fixedLegTenor = '6M';
% %         fixedLegConvention = 'ModifiedFollowing';
% %         fixedLegDayCounter = '30360';
% % 
% %         dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% % 
% %         swapRateHelperParams = containers.Map({'fixedLegTenor','fixedLegConvention','fixedLegDayCounter','IborIndex','discountCurve'}, ...
% %                                 {fixedLegTenor,fixedLegConvention,fixedLegDayCounter,KRW_Libor_3M,dummyCurve});
% % 
% % 
% %         swapRateHelperParams('rate') = dataRate{i};
% %         swapRateHelperParams('rateTenor') = dataTenor{i};
% %         swapRateHelper = SwapRateHelper(swapRateHelperParams);
% %         rateHelpers{end+1} = swapRateHelper;
%     end
% end
% 
% 
% discontCurveRawData = yieldCurve.forwardCurve.params('rawData');
% mktData_fwd = containers.Map({'rawData','asOfDate'}, ...
%                     {discontCurveRawData,valueDateH});
% zeroCurveMktData_fwd = MktData(mktData_fwd);
% discountCurve_1 =  ZeroCurve(zeroCurveMktData_fwd);
% 
% % forward & discount zeroCurve
% dummyCurve_fwd = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% % dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% 
% yieldCurveParams = containers.Map({'rateHelpers','forwardCurve','discountCurve'},{rateHelpers_fwd,dummyCurve_fwd,discountCurve_1});
% 
% tic
% % yieldCurve = YieldCurve(yieldCurveParams);
% yieldCurve_DS = YieldCurveOIS_DS(yieldCurveParams);
% 
% out = yieldCurve_DS.BootStrap();
% toc
% 
% 
% %% forward curve bootstrapping end USD SOFR Curve(Libor3M Curve)
% toc
% 
% 
% outInfo = struct;
% 
% outInfo.impliedRate = zeros(length(yieldCurve_DS.rateHelpers),1);
% outInfo.diff = zeros(length(yieldCurve_DS.rateHelpers),1);
% outInfo.NPV = zeros(length(yieldCurve_DS.rateHelpers),1);
% 
% for i=1:length(yieldCurve_DS.rateHelpers)
%     if strcmp(yieldCurve_DS.rateHelpers{i}.rateType,'Deposit')
%         outInfo.impliedRate(i) = yieldCurve_DS.rateHelpers{i}.ImpliedQuote();
%         outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
%     elseif (strcmp(yieldCurve_DS.rateHelpers{i}.rateType,'Swap')) || (strcmp(yieldCurve_DS.rateHelpers{i}.rateType,'OIS'))
%         for j=1:length(yieldCurve_DS.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates)
%             yieldCurve_DS.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates{j}.fixedRate = dataRate{i};
%         end
%         
%         swapOut = yieldCurve_DS.rateHelpers{i}.vanillaSwap.Calculate(yieldCurve_DS.rateHelpers{i}.discountCurve);
%         outInfo.NPV(i) = swapOut.NPV;
%         outInfo.impliedRate(i) = swapOut.fairRate;
%         outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
%     end
% end
% 
% 
% 
% aaa =1;
% 
% 
% targetTimeBucket = [0.0027397
% 0.25
% 0.5
% 0.75
% 1
% 2
% 3
% 5
% 10
% 20
% 30
%     ];
% targetZero = zeros(length(targetTimeBucket),1);
% for i=1:length(targetZero)
%     targetZero(i) = yieldCurve.forwardCurve.Zero(targetTimeBucket(i));
% end
% 
% targetOut = zeros(length(targetZero),2);
% targetOut(:,1) = targetTimeBucket(:);
% targetOut(:,2) = targetZero(:);
% bbb= 1.0;


In [ ]:
test_YieldCurveGlobalCalib_SOFR_20220727.m

clc
clear
tic

% valueDate = '2022-07-27';
valueDate = '2022-07-29';


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;
% useSimpleTeleScopingFlag = false;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);
USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

jointCalendar = JointCalendar({USGTCalendar,USFDCalendar});


%% import and construct an discount curve USD SOFR Curve

%loong tenor(~50Y) test data

dataType= {
'Deposit'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
'OIS'
};

dataTenor = {
'SN'
'1W'
'2W'
'3W'
'1M'
'2M'
'3M'
'6M'
'9M'
'12M'
'2Y'
'3Y'
'4Y'
'5Y'
'6Y'
'7Y'
'8Y'
'9Y'
'10Y'
'12Y'
'15Y'
'20Y'
'25Y'
'30Y'
'40Y'
'50Y'
};

dataRate = {
0.01530000
0.02285400
0.02287800
0.02287700
0.02289550
0.02359000
0.02542000
0.02895500
0.03030000
0.03074400
0.02949500
0.02761300
0.02640500
0.02579250
0.02556100
0.02548250
0.02549300
0.02559500
0.02578650
0.02628850
0.02684000
0.02687000
0.02612600
0.02534000
0.02309000
0.02071000
};


% construct iborIndex
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'1D','USD',0,USGTCalendar,'Following','Act360',false,dummyCurve});
                    
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',0,USGTCalendar,'ModifiedFollowing','Act360',false,dummyCurve});
                    
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',0,jointCalendar,'Following','Act360',false,dummyCurve});

% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',2,jointCalendar,'Following','Act360',false,dummyCurve});
                    
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','KRW',2,Calendar(),'ModifiedFollowing','Act360',false,dummyCurve});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;
IborIndexParams('useSimpleTeleScoping') = useSimpleTeleScopingFlag;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



% overnight index itself is nothing but 1D IborIndex
overnightIndex = IborIndex(IborIndexParams);

rateHelpers = {};

for i=1:length(dataRate)
    if strcmp(dataType{i},'Deposit')
        depositRateHelperParams = containers.Map({'IborIndexParams'}, ...
                        {IborIndexParams});

%         calcCalendar = USFDCalendar;
        calcCalendar = USGTCalendar;
        
        depositRateHelperParams('calcCalendar') = calcCalendar;
        
        % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
%         spotLag = 2;
        spotLag = 0;
        
        
        depositRateHelperParams('spotLag') = spotLag;
        
        depositRateHelperParams('rate') = dataRate{i};
        depositRateHelperParams('rateTenor') = dataTenor{i};
        depositRateHelper = DepositRateHelper(depositRateHelperParams);
    
        rateHelpers{end+1} = depositRateHelper;
        
    elseif strcmp(dataType{i},'OIS')
        
        if i== 17 || i == 16
            bbb = 1.0;
        end
        % cashflow interval(?)
        fixedLegTenor = '1Y';
        fixedLegConvention = 'ModifiedFollowing';
        fixedLegDayCounter = 'Act360';
        
        floatingLegTenor = '1Y';
        floatingLegConvention = 'ModifiedFollowing';
        floatingLegDayCounter = 'Act360';
        
        % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
%         spotLag = 2;
        spotLag = 0;
        
        
        % payLag : paymentDate Coupon of Legs of Swap:  paymentDate = endDate + 2D
        
        payLag  = 2;
%         payLag  = 0;

        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
        
        calcCalendar = USFDCalendar;

        oisRateHelperParams = containers.Map({'calcCalendar','spotLag','payLag','fixedLegTenor','fixedLegConvention','fixedLegDayCounter',...
                                'floatingLegTenor','floatingLegConvention','floatingLegDayCounter','overnightIndex','discountCurve'}, ...
                                {calcCalendar,spotLag,payLag, fixedLegTenor,fixedLegConvention,fixedLegDayCounter,...
                                 floatingLegTenor,floatingLegConvention,floatingLegDayCounter,overnightIndex,dummyCurve});

%         oisRateHelperParams('useRounding') = useRounding;
        
        oisRateHelperParams('rate') = dataRate{i};
        oisRateHelperParams('rateTenor') = dataTenor{i};
        oisRateHelper = OISRateHelper(oisRateHelperParams);
        rateHelpers{end+1} = oisRateHelper;
        
    elseif strcmp(dataType{i},'Swap')
        fixedLegTenor = '6M';
        fixedLegConvention = 'ModifiedFollowing';
        fixedLegDayCounter = '30360';

        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

        swapRateHelperParams = containers.Map({'fixedLegTenor','fixedLegConvention','fixedLegDayCounter','IborIndex','discountCurve'}, ...
                                {fixedLegTenor,fixedLegConvention,fixedLegDayCounter,KRW_Libor_3M,dummyCurve});


        swapRateHelperParams('rate') = dataRate{i};
        swapRateHelperParams('rateTenor') = dataTenor{i};
        swapRateHelper = SwapRateHelper(swapRateHelperParams);
        rateHelpers{end+1} = swapRateHelper;
    end
end

% forward & discount zeroCurve
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

yieldCurveParams = containers.Map({'rateHelpers','forwardCurve','discountCurve'},{rateHelpers,dummyCurve,dummyCurve1});

% tic
% yieldCurve = YieldCurve(yieldCurveParams);
yieldCurve = YieldCurveOIS(yieldCurveParams);

out = yieldCurve.BootStrap();
% out = yieldCurve.GlobalCalib();

toc
ddd = 1.0;

%% discount curve bootstrapping end USD SOFR Curve

% %% forward curve bootstrapping start USD SOFR Curve(Libor3M Curve)
% %short tenor(~2Y) test data
% dataType_fwd= {'Deposit'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% };
% 
% dataTenor_fwd = {'ON'
% '1W'
% '2W'
% '3W'
% '1M'
% '2M'
% '3M'
% '4M'
% '5M'
% '6M'
% '7M'
% '8M'
% '9M'
% '10M'
% '11M'
% '12M'
% '18M'
% '2Y'
% };
% 
% dataRate_fwd = {0.0005
% 0.0004356
% 0.000432
% 0.000431
% 0.000425
% 0.000437
% 0.00044
% 0.000437
% 0.000444
% 0.000448
% 0.000472
% 0.0004735
% 0.0004835
% 0.0005195
% 0.0005495
% 0.0005895
% 0.000997
% 0.001733
% };
% 
% %loong tenor(~50Y) test data
% 
% dataType_fwd= {'Deposit'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% 'OIS'
% };
% 
% dataTenor_fwd = {'ON'
% '1W'
% '2W'
% '3W'
% '1M'
% '2M'
% '3M'
% '4M'
% '5M'
% '6M'
% '7M'
% '8M'
% '9M'
% '10M'
% '11M'
% '12M'
% '18M'
% '2Y'
% '3Y'
% '4Y'
% '5Y'
% '6Y'
% '7Y'
% '8Y'
% '9Y'
% '10Y'
% '12Y'
% '15Y'
% '20Y'
% '25Y'
% '30Y'
% '40Y'
% '50Y'
% };
% 
% dataRate_fwd = {0.0005
% 0.0004356
% 0.000432
% 0.000431
% 0.000425
% 0.000437
% 0.00044
% 0.000437
% 0.000444
% 0.000448
% 0.000472
% 0.0004735
% 0.0004835
% 0.0005195
% 0.0005495
% 0.0005895
% 0.000997
% 0.001733
% 0.0035535
% 0.00546
% 0.00707
% 0.00845
% 0.009575
% 0.01047
% 0.011205
% 0.011815
% 0.01279
% 0.013734
% 0.01457
% 0.0147565
% 0.01479
% 0.014144
% 0.01331
% };
% 
% % construct iborIndex
% dummyCurve_fwd = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% 
% IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'1D','USD',0,USGTCalendar,'ModifiedFollowing','Act360',false,dummyCurve_fwd});
%                     
% % IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
% %                         {'1D','USD',0,jointCalendar,'Following','Act360',false,dummyCurve});
% 
% % IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
% %                         {'1D','USD',2,jointCalendar,'Following','Act360',false,dummyCurve});
%                     
% % IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
% %                         {'3M','KRW',2,Calendar(),'ModifiedFollowing','Act360',false,dummyCurve});
% 
% IborIndexParams_fwd('forecastTodaysFixing') = true;
% IborIndexParams_fwd('useRounding') = useRounding;
% 
% % overnight index itself is nothing but 1D IborIndex
% overnightIndex_fwd = IborIndex(IborIndexParams_fwd);
% 
% rateHelpers_fwd = {};
% 
% for i=1:length(dataRate_fwd)
%     if strcmp(dataType_fwd{i},'Deposit')
%         depositRateHelperParams_fwd = containers.Map({'IborIndexParams'}, ...
%                         {IborIndexParams_fwd});
% 
%         calcCalendar = USFDCalendar;
% %         calcCalendar = USGTCalendar;
%         
%         depositRateHelperParams_fwd('calcCalendar') = calcCalendar;
%         
%         depositRateHelperParams_fwd('rate') = dataRate_fwd{i};
%         depositRateHelperParams_fwd('rateTenor') = dataTenor_fwd{i};
%         depositRateHelper_fwd = DepositRateHelper(depositRateHelperParams_fwd);
%     
%         rateHelpers_fwd{end+1} = depositRateHelper_fwd;
%         
%     elseif strcmp(dataType_fwd{i},'OIS')
%         
%         % cashflow interval(?)
%         fixedLegTenor = '1Y';
%         fixedLegConvention = 'ModifiedFollowing';
%         fixedLegDayCounter = 'Act360';
%         
%         floatingLegTenor = '1Y';
%         floatingLegConvention = 'ModifiedFollowing';
%         floatingLegDayCounter = 'Act360';
%         
%         % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
%         spotLag = 2;
%         
%         % payLag : paymentDate Coupon of Legs of Swap:  paymentDate = endDate + 2D
%         
%         payLag  = 2;
% 
% %         dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
%         % from bootStrapping SOFR Curve we have bootStrapped DiscountCurve
%         % zeroRates and we will use it for discountingCurve while
%         % bootStrapping forwardCurve
%         
%         discontCurveRawData = yieldCurve.forwardCurve.params('rawData');
%         mktData_fwd = containers.Map({'rawData','asOfDate'}, ...
%                             {discontCurveRawData,valueDateH});
%         zeroCurveMktData_fwd = MktData(mktData_fwd);
%         discountCurve_1 =  ZeroCurve(zeroCurveMktData_fwd);
%         
%         
%         calcCalendar = USFDCalendar;
% 
%         oisRateHelperParams_fwd = containers.Map({'calcCalendar','spotLag','payLag','fixedLegTenor','fixedLegConvention','fixedLegDayCounter',...
%                                 'floatingLegTenor','floatingLegConvention','floatingLegDayCounter','overnightIndex','discountCurve'}, ...
%                                 {calcCalendar,spotLag,payLag, fixedLegTenor,fixedLegConvention,fixedLegDayCounter,...
%                                  floatingLegTenor,floatingLegConvention,floatingLegDayCounter,overnightIndex_fwd,discountCurve_1});
%         
%         oisRateHelperParams_fwd('rate') = dataRate_fwd{i};
%         oisRateHelperParams_fwd('rateTenor') = dataTenor_fwd{i};
%         oisRateHelper_fwd = OISRateHelper_DS(oisRateHelperParams_fwd);
%         rateHelpers_fwd{end+1} = oisRateHelper_fwd;
%         
%     elseif strcmp(dataType{i},'Swap')
% %         fixedLegTenor = '6M';
% %         fixedLegConvention = 'ModifiedFollowing';
% %         fixedLegDayCounter = '30360';
% % 
% %         dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% % 
% %         swapRateHelperParams = containers.Map({'fixedLegTenor','fixedLegConvention','fixedLegDayCounter','IborIndex','discountCurve'}, ...
% %                                 {fixedLegTenor,fixedLegConvention,fixedLegDayCounter,KRW_Libor_3M,dummyCurve});
% % 
% % 
% %         swapRateHelperParams('rate') = dataRate{i};
% %         swapRateHelperParams('rateTenor') = dataTenor{i};
% %         swapRateHelper = SwapRateHelper(swapRateHelperParams);
% %         rateHelpers{end+1} = swapRateHelper;
%     end
% end
% 
% 
% discontCurveRawData = yieldCurve.forwardCurve.params('rawData');
% mktData_fwd = containers.Map({'rawData','asOfDate'}, ...
%                     {discontCurveRawData,valueDateH});
% zeroCurveMktData_fwd = MktData(mktData_fwd);
% discountCurve_1 =  ZeroCurve(zeroCurveMktData_fwd);
% 
% % forward & discount zeroCurve
% dummyCurve_fwd = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% % dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% 
% yieldCurveParams = containers.Map({'rateHelpers','forwardCurve','discountCurve'},{rateHelpers_fwd,dummyCurve_fwd,discountCurve_1});
% 
% tic
% % yieldCurve = YieldCurve(yieldCurveParams);
% yieldCurve_DS = YieldCurveOIS_DS(yieldCurveParams);
% 
% out = yieldCurve_DS.BootStrap();
% toc
% 
% 
% %% forward curve bootstrapping end USD SOFR Curve(Libor3M Curve)
% toc
% 
% 
% outInfo = struct;
% 
% outInfo.impliedRate = zeros(length(yieldCurve_DS.rateHelpers),1);
% outInfo.diff = zeros(length(yieldCurve_DS.rateHelpers),1);
% outInfo.NPV = zeros(length(yieldCurve_DS.rateHelpers),1);
% 
% for i=1:length(yieldCurve_DS.rateHelpers)
%     if strcmp(yieldCurve_DS.rateHelpers{i}.rateType,'Deposit')
%         outInfo.impliedRate(i) = yieldCurve_DS.rateHelpers{i}.ImpliedQuote();
%         outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
%     elseif (strcmp(yieldCurve_DS.rateHelpers{i}.rateType,'Swap')) || (strcmp(yieldCurve_DS.rateHelpers{i}.rateType,'OIS'))
%         for j=1:length(yieldCurve_DS.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates)
%             yieldCurve_DS.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates{j}.fixedRate = dataRate{i};
%         end
%         
%         swapOut = yieldCurve_DS.rateHelpers{i}.vanillaSwap.Calculate(yieldCurve_DS.rateHelpers{i}.discountCurve);
%         outInfo.NPV(i) = swapOut.NPV;
%         outInfo.impliedRate(i) = swapOut.fairRate;
%         outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
%     end
% end
% 
% 
% 
% aaa =1;
% 
% 
% targetTimeBucket = [0.0027397
% 0.25
% 0.5
% 0.75
% 1
% 2
% 3
% 5
% 10
% 20
% 30
%     ];
% targetZero = zeros(length(targetTimeBucket),1);
% for i=1:length(targetZero)
%     targetZero(i) = yieldCurve.forwardCurve.Zero(targetTimeBucket(i));
% end
% 
% targetOut = zeros(length(targetZero),2);
% targetOut(:,1) = targetTimeBucket(:);
% targetOut(:,2) = targetZero(:);
% bbb= 1.0;


In [ ]:
test_YieldCurveBootStrap_HKD_IRS_20220727.m


clc
clear

HKCD.name = 'HKCD';
HKCD.holidaysList = {'2021-01-01'
'2021-02-12'
'2021-02-15'
'2021-04-02'
'2021-04-05'
'2021-04-06'
'2021-05-19'
'2021-06-14'
'2021-07-01'
'2021-09-22'
'2021-10-01'
'2021-10-14'
'2021-12-27'
'2022-02-01'
'2022-02-02'
'2022-02-03'
'2022-04-05'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-05-09'
'2022-06-03'
'2022-07-01'
'2022-09-12'
'2022-10-04'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-01-23'
'2023-01-24'
'2023-01-25'
'2023-04-05'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-26'
'2023-06-22'
'2023-10-02'
'2023-10-23'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-02-12'
'2024-02-13'
'2024-03-29'
'2024-04-01'
'2024-04-04'
'2024-05-01'
'2024-05-15'
'2024-06-10'
'2024-07-01'
'2024-09-18'
'2024-10-01'
'2024-10-11'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-01-29'
'2025-01-30'
'2025-01-31'
'2025-04-04'
'2025-04-18'
'2025-04-21'
'2025-05-01'
'2025-05-05'
'2025-07-01'
'2025-10-01'
'2025-10-07'
'2025-10-29'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-12-25'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2028-04-14'
'2028-04-17'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2033-04-15'
'2033-04-18'
'2033-12-26'
'2034-04-07'
'2034-04-10'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-12-25'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2039-04-08'
'2039-04-11'
'2039-12-26'
'2040-03-30'
'2040-04-02'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-12-25'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-12-26'
'2045-04-07'
'2045-04-10'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-12-25'
'2049-01-01'
'2049-04-16'
'2049-04-19'

    };

HKCDCalendar = Calendar();
HKCDCalendar.name = HKCD.name;

for i=1:length(HKCD.holidaysList)
    HKCDCalendar.AddHoliday(HKCD.holidaysList{i})
end

% jointCalendar = JointCalendar({KRNTCalendar,HKCalendar});
jointCalendar = HKCDCalendar;

valueDate = '2022-07-27';
valueDateH = H_Date(valueDate);
%% import and construct an YieldCurve
dataType= {'Deposit'
'Deposit'
'Swap'
'Swap'
'Swap'
'Swap'
'Swap'
'Swap'
'Swap'
};

dataTenor = {'1D'
          '3M'
          '1Y'
          '2Y'
          '3Y'
          '4Y'
          '5Y'
          '7Y'
          '10Y'
};

dataRate = {0.009135100
0.021415500
0.031300000
0.031900000
0.030700000
0.029800000
0.029350000
0.029380000
0.029700000
};


% construct iborIndex
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','HKD',0,jointCalendar,'ModifiedFollowing','Act365',false,dummyCurve});
                    
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','KRW',2,Calendar(),'ModifiedFollowing','Act360',false,dummyCurve});

IborIndexParams('forecastTodaysFixing') = true;

KRW_Libor_3M = IborIndex(IborIndexParams);

rateHelpers = {};

for i=1:length(dataRate)
    if strcmp(dataType{i},'Deposit')
        depositRateHelperParams = containers.Map({'IborIndexParams','calcCalendar','spotLag'}, ...
                        {IborIndexParams,jointCalendar,0});

        depositRateHelperParams('rate') = dataRate{i};
        depositRateHelperParams('rateTenor') = dataTenor{i};
        depositRateHelper = DepositRateHelper(depositRateHelperParams);
    
        rateHelpers{end+1} = depositRateHelper;
        
    elseif strcmp(dataType{i},'Swap')
        fixedLegTenor = '3M';
        fixedLegConvention = 'ModifiedFollowing';
        fixedLegDayCounter = 'Act365';

        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

        swapRateHelperParams = containers.Map({'calcCalendar','spotLag','fixedLegTenor','fixedLegConvention','fixedLegDayCounter',...
            'floatingLegTenor','floatingLegConvention','floatingLegDayCounter','IborIndex','discountCurve'}, ...
                                {jointCalendar,0,fixedLegTenor,fixedLegConvention,fixedLegDayCounter,...,
                                '3M','ModifiedFollowing','Act365',KRW_Libor_3M,dummyCurve});


        swapRateHelperParams('rate') = dataRate{i};
        swapRateHelperParams('rateTenor') = dataTenor{i};
        swapRateHelper = SwapRateHelper(swapRateHelperParams);
        rateHelpers{end+1} = swapRateHelper;
    end
end

% forward & discount zeroCurve
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

yieldCurveParams = containers.Map({'rateHelpers','forwardCurve','discountCurve'},{rateHelpers,dummyCurve,dummyCurve1});

tic
yieldCurve = YieldCurve(yieldCurveParams);
out = yieldCurve.BootStrap();
toc

outInfo.impliedRate = zeros(length(yieldCurve.rateHelpers),1);outInfo = struct;

outInfo.diff = zeros(length(yieldCurve.rateHelpers),1);
outInfo.NPV = zeros(length(yieldCurve.rateHelpers),1);

for i=1:length(yieldCurve.rateHelpers)
    if strcmp(yieldCurve.rateHelpers{i}.rateType,'Deposit')
        outInfo.impliedRate(i) = yieldCurve.rateHelpers{i}.ImpliedQuote();
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    elseif strcmp(yieldCurve.rateHelpers{i}.rateType,'Swap')
        for j=1:length(yieldCurve.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates)
            yieldCurve.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates{j}.fixedRate = dataRate{i};
        end
        
        swapOut = yieldCurve.rateHelpers{i}.vanillaSwap.Calculate(yieldCurve.rateHelpers{i}.discountCurve);
        outInfo.NPV(i) = swapOut.NPV;
        outInfo.impliedRate(i) = swapOut.fairRate;
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    end
end
aaa =1;

targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
4
5
7
10
12
15
20
25
30
    ];
targetZero = zeros(length(targetTimeBucket),1);
targetDF = zeros(length(targetTimeBucket),1);
for i=1:length(targetZero)
    targetZero(i) = yieldCurve.forwardCurve.Zero(targetTimeBucket(i));
    targetDF(i) = exp(-1.0*targetTimeBucket(i)*targetZero(i));
end

targetOut = zeros(length(targetZero),2);
targetOut(:,1) = targetTimeBucket(:);
targetOut(:,2) = targetDF(:);
targetOut(:,3) = targetZero(:);
bbb= 1.0;


In [ ]:
test_VolTargetOption_kospi200_202161026.m

clc
clear

% valueDate = '2020-12-18';

    
alphaTrace = cell(100,1);
volIdxTrace = cell(100,1);

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start

rawBusinessDates = [
    20161026
    20161027
    20161028
    20161031
    20161101
    20161102
    20161103
    20161104
    20161107
    20161108
    20161109
    20161110
    20161111
    20161114
    20161115
    20161116
    20161117
    20161118
    20161121
    20161122
    20161123
    20161124
    20161125
    20161128
    20161129
    20161130
    20161201
    20161202
    20161205
    20161206
    20161207
    20161208
    20161209
    20161212
    20161213
    20161214
    20161215
    20161216
    20161219
    20161220
    20161221
    20161222
    20161223
    20161226
    20161227
    20161228
    20161229
    20170102
    20170103
    20170104
    20170105
    20170106
    20170109
    20170110
    20170111
    20170112
    20170113
    20170116
    20170117
    20170118
    20170119
    20170120
    20170123
    20170124
    20170125
    20170126
    20170131
    20170201
    20170202
    20170203
    20170206
    20170207
    20170208
    20170209
    20170210
    20170213
    20170214
    20170215
    20170216
    20170217
    20170220
    20170221
    20170222
    20170223
    20170224
    20170227
    20170228
    20170302
    20170303
    20170306
    20170307
    20170308
    20170309
    20170310
    20170313
    20170314
    20170315
    20170316
    20170317
    20170320
    20170321
    20170322
    20170323
    20170324
    20170327
    20170328
    20170329
    20170330
    20170331
    20170403
    20170404
    20170405
    20170406
    20170407
    20170410
    20170411
    20170412
    20170413
    20170414
    20170417
    20170418
    20170419
    20170420
    20170421
    20170424
    20170425
    20170426
    20170427
    20170428
    20170502
    20170504
    20170508
    20170510
    20170511
    20170512
    20170515
    20170516
    20170517
    20170518
    20170519
    20170522
    20170523
    20170524
    20170525
    20170526
    20170529
    20170530
    20170531
    20170601
    20170602
    20170605
    20170607
    20170608
    20170609
    20170612
    20170613
    20170614
    20170615
    20170616
    20170619
    20170620
    20170621
    20170622
    20170623
    20170626
    20170627
    20170628
    20170629
    20170630
    20170703
    20170704
    20170705
    20170706
    20170707
    20170710
    20170711
    20170712
    20170713
    20170714
    20170717
    20170718
    20170719
    20170720
    20170721
    20170724
    20170725
    20170726
    20170727
    20170728
    20170731
    20170801
    20170802
    20170803
    20170804
    20170807
    20170808
    20170809
    20170810
    20170811
    20170814
    20170816
    20170817
    20170818
    20170821
    20170822
    20170823
    20170824
    20170825
    20170828
    20170829
    20170830
    20170831
    20170901
    20170904
    20170905
    20170906
    20170907
    20170908
    20170911
    20170912
    20170913
    20170914
    20170915
    20170918
    20170919
    20170920
    20170921
    20170922
    20170925
    20170926
    20170927
    20170928
    20170929
    20171010
    20171011
    20171012
    20171013
    20171016
    20171017
    20171018
    20171019
    20171020
    20171023
    20171024
    20171025
    20171026
    20171027
    20171030
    20171031
    20171101
    20171102
    20171103
    20171106
    20171107
    20171108
    20171109
    20171110
    20171113
    20171114
    20171115
    20171116
    20171117
    20171120
    20171121
    20171122
    20171123
    20171124
    20171127
    20171128
    20171129
    20171130
    20171201
    20171204
    20171205
    20171206
    20171207
    20171208
    20171211
    20171212
    20171213
    20171214
    20171215
    20171218
    20171219
    20171220
    20171221
    20171222
    20171226
    20171227
    20171228
    20180102
    20180103
    20180104
    20180105
    20180108
    20180109
    20180110
    20180111
    20180112
    20180115
    20180116
    20180117
    20180118
    20180119
    20180122
    20180123
    20180124
    20180125
    20180126
    20180129
    20180130
    20180131
    20180201
    20180202
    20180205
    20180206
    20180207
    20180208
    20180209
    20180212
    20180213
    20180214
    20180219
    20180220
    20180221
    20180222
    20180223
    20180226
    20180227
    20180228
    20180302
    20180305
    20180306
    20180307
    20180308
    20180309
    20180312
    20180313
    20180314
    20180315
    20180316
    20180319
    20180320
    20180321
    20180322
    20180323
    20180326
    20180327
    20180328
    20180329
    20180330
    20180402
    20180403
    20180404
    20180405
    20180406
    20180409
    20180410
    20180411
    20180412
    20180413
    20180416
    20180417
    20180418
    20180419
    20180420
    20180423
    20180424
    20180425
    20180426
    20180427
    20180430
    20180502
    20180503
    20180504
    20180508
    20180509
    20180510
    20180511
    20180514
    20180515
    20180516
    20180517
    20180518
    20180521
    20180523
    20180524
    20180525
    20180528
    20180529
    20180530
    20180531
    20180601
    20180604
    20180605
    20180607
    20180608
    20180611
    20180612
    20180614
    20180615
    20180618
    20180619
    20180620
    20180621
    20180622
    20180625
    20180626
    20180627
    20180628
    20180629
    20180702
    20180703
    20180704
    20180705
    20180706
    20180709
    20180710
    20180711
    20180712
    20180713
    20180716
    20180717
    20180718
    20180719
    20180720
    20180723
    20180724
    20180725
    20180726
    20180727
    20180730
    20180731
    20180801
    20180802
    20180803
    20180806
    20180807
    20180808
    20180809
    20180810
    20180813
    20180814
    20180816
    20180817
    20180820
    20180821
    20180822
    20180823
    20180824
    20180827
    20180828
    20180829
    20180830
    20180831
    20180903
    20180904
    20180905
    20180906
    20180907
    20180910
    20180911
    20180912
    20180913
    20180914
    20180917
    20180918
    20180919
    20180920
    20180921
    20180927
    20180928
    20181001
    20181002
    20181004
    20181005
    20181008
    20181010
    20181011
    20181012
    20181015
    20181016
    20181017
    20181018
    20181019
    20181022
    20181023
    20181024
    20181025
    20181026
    20181029
    20181030
    20181031
    20181101
    20181102
    20181105
    20181106
    20181107
    20181108
    20181109
    20181112
    20181113
    20181114
    20181115
    20181116
    20181119
    20181120
    20181121
    20181122
    20181123
    20181126
    20181127
    20181128
    20181129
    20181130
    20181203
    20181204
    20181205
    20181206
    20181207
    20181210
    20181211
    20181212
    20181213
    20181214
    20181217
    20181218
    20181219
    20181220
    20181221
    20181224
    20181226
    20181227
    20181228
    20181231
    20190102
    20190103
    20190104
    20190107
    20190108
    20190109
    20190110
    20190111
    20190114
    20190115
    20190116
    20190117
    20190118
    20190121
    20190122
    20190123
    20190124
    20190125
    20190128
    20190129
    20190130
    20190131
    20190201
    20190207
    20190208
    20190211
    20190212
    20190213
    20190214
    20190215
    20190218
    20190219
    20190220
    20190221
    20190222
    20190225
    20190226
    20190227
    20190228
    20190304
    20190305
    20190306
    20190307
    20190308
    20190311
    20190312
    20190313
    20190314
    20190315
    20190318
    20190319
    20190320
    20190321
    20190322
    20190325
    20190326
    20190327
    20190328
    20190329
    20190401
    20190402
    20190403
    20190404
    20190405
    20190408
    20190409
    20190410
    20190411
    20190412
    20190415
    20190416
    20190417
    20190418
    20190419
    20190422
    20190423
    20190424
    20190425
    20190426
    20190429
    20190430
    20190502
    20190503
    20190507
    20190508
    20190509
    20190510
    20190513
    20190514
    20190515
    20190516
    20190517
    20190520
    20190521
    20190522
    20190523
    20190524
    20190527
    20190528
    20190529
    20190530
    20190531
    20190603
    20190604
    20190605
    20190607
    20190610
    20190611
    20190612
    20190613
    20190614
    20190617
    20190618
    20190619
    20190620
    20190621
    20190624
    20190625
    20190626
    20190627
    20190628
    20190701
    20190702
    20190703
    20190704
    20190705
    20190708
    20190709
    20190710
    20190711
    20190712
    20190715
    20190716
    20190717
    20190718
    20190719
    20190722
    20190723
    20190724
    20190725
    20190726
    20190729
    20190730
    20190731
    20190801
    20190802
    20190805
    20190806
    20190807
    20190808
    20190809
    20190812
    20190813
    20190814
    20190816
    20190819
    20190820
    20190821
    20190822
    20190823
    20190826
    20190827
    20190828
    20190829
    20190830
    20190902
    20190903
    20190904
    20190905
    20190906
    20190909
    20190910
    20190911
    20190916
    20190917
    20190918
    20190919
    20190920
    20190923
    20190924
    20190925
    20190926
    20190927
    20190930
    20191001
    20191002
    20191004
    20191007
    20191008
    20191010
    20191011
    20191014
    20191015
    20191016
    20191017
    20191018
    20191021
    20191022
    20191023
    20191024
    20191025
    20191028
    20191029
    20191030
    20191031
    20191101
    20191104
    20191105
    20191106
    20191107
    20191108
    20191111
    20191112
    20191113
    20191114
    20191115
    20191118
    20191119
    20191120
    20191121
    20191122
    20191125
    20191126
    20191127
    20191128
    20191129
    20191202
    20191203
    20191204
    20191205
    20191206
    20191209
    20191210
    20191211
    20191212
    20191213
    20191216
    20191217
    20191218
    20191219
    20191220
    20191223
    20191224
    20191226
    20191227
    20191230
    20191231
    20200102
    20200103
    20200106
    20200107
    20200108
    20200109
    20200110
    20200113
    20200114
    20200115
    20200116
    20200117
    20200120
    20200121
    20200122
    20200123
    20200128
    20200129
    20200130
    20200131
    20200203
    20200204
    20200205
    20200206
    20200207
    20200210
    20200211
    20200212
    20200213
    20200214
    20200217
    20200218
    20200219
    20200220
    20200221
    20200224
    20200225
    20200226
    20200227
    20200228
    20200302
    20200303
    20200304
    20200305
    20200306
    20200309
    20200310
    20200311
    20200312
    20200313
    20200316
    20200317
    20200318
    20200319
    20200320
    20200323
    20200324
    20200325
    20200326
    20200327
    20200330
    20200331
    20200401
    20200402
    20200403
    20200406
    20200407
    20200408
    20200409
    20200410
    20200413
    20200414
    20200416
    20200417
    20200420
    20200421
    20200422
    20200423
    20200424
    20200427
    20200428
    20200429
    20200504
    20200506
    20200507
    20200508
    20200511
    20200512
    20200513
    20200514
    20200515
    20200518
    20200519
    20200520
    20200521
    20200522
    20200525
    20200526
    20200527
    20200528
    20200529
    20200601
    20200602
    20200603
    20200604
    20200605
    20200608
    20200609
    20200610
    20200611
    20200612
    20200615
    20200616
    20200617
    20200618
    20200619
    20200622
    20200623
    20200624
    20200625
    20200626
    20200629
    20200630
    20200701
    20200702
    20200703
    20200706
    20200707
    20200708
    20200709
    20200710
    20200713
    20200714
    20200715
    20200716
    20200717
    20200720
    20200721
    20200722
    20200723
    20200724
    20200727
    20200728
    20200729
    20200730
    20200731
    20200803
    20200804
    20200805
    20200806
    20200807
    20200810
    20200811
    20200812
    20200813
    20200814
    20200818
    20200819
    20200820
    20200821
    20200824
    20200825
    20200826
    20200827
    20200828
    20200831
    20200901
    20200902
    20200903
    20200904
    20200907
    20200908
    20200909
    20200910
    20200911
    20200914
    20200915
    20200916
    20200917
    20200918
    20200921
    20200922
    20200923
    20200924
    20200925
    20200928
    20200929
    20201005
    20201006
    20201007
    20201008
    20201012
    20201013
    20201014
    20201015
    20201016
    20201019
    20201020
    20201021
    20201022
    20201023
    20201026
    20201027
    20201028
    20201029
    20201030
    20201031
    20201102
    20201103
    20201104
    20201105
    20201106
    20201109
    20201110
    20201111
    20201112
    20201113
    20201116
    20201117
    20201118
    20201119
    20201120
    20201123
    20201124
    20201125
    20201126
    20201127
    20201130
    20201201
    20201202
    20201203
    20201204
    20201207
    20201208
    20201209
    20201210
    20201211
    20201214
    20201215
    20201216
    20201217
    20201218
    20201221
    20201222
    20201223
    20201224
    20201228
    20201229
    20201230
    20201231
    20210104
    20210105
    20210106
    20210107
    20210108
    20210111
    20210112
    20210113
    20210114
    20210115
    20210118
    20210119
    20210120
    20210121
    20210122
    20210125
    20210126
    20210127
    20210128
    20210129
    20210201
    20210202
    20210203
    20210204
    20210205
    20210208
    20210209
    20210210
    20210215
    20210216
    20210217
    20210218
    20210219
    20210222
    20210223
    20210224
    20210225
    20210226
    20210302
    20210303
    20210304
    20210305
    20210308
    20210309
    20210310
    20210311
    20210312
    20210315
    20210316
    20210317
    20210318
    20210319
    20210322
    20210323
    20210324
    20210325
    20210326
    20210329
    20210330
    20210331
    20210401
    20210402
    20210405
    20210406
    20210407
    20210408
    20210409
    20210412
    20210413
    20210414
    20210415
    20210416
    20210419
    20210420
    20210421
    20210422
    20210423
    20210426
    20210427
    20210428
    20210429
    20210430
    20210503
    20210504
    20210506
    20210507
    20210510
    20210511
    20210512
    20210513
    20210514
    20210517
    20210518
    20210520
    20210521
    20210524
    20210525
    20210526
    20210527
    20210528
    20210531
    20210601
    20210602
    20210603
    20210604
    20210607
    20210608
    20210609
    20210610
    20210611
    20210614
    20210615
    20210616
    20210617
    20210618
    20210621
    20210622
    20210623
    20210624
    20210625
    20210628
    20210629
    20210630
    20210701
    20210702
    20210705
    20210706
    20210707
    20210708
    20210709
    20210712
    20210713
    20210714
    20210715
    20210716
    20210719
    20210720
    20210721
    20210722
    20210723
    20210726
    20210727
    20210728
    20210729
    20210730
    20210802
    20210803
    20210804
    20210805
    20210806
    20210809
    20210810
    20210811
    20210812
    20210813
    20210817
    20210818
    20210819
    20210820
    20210823
    20210824
    20210825
    20210826
    20210827
    20210830
    20210831
    20210901
    20210902
    20210903
    20210906
    20210907
    20210908
    20210909
    20210910
    20210913
    20210914
    20210915
    20210916
    20210917
    20210923
    20210924
    20210927
    20210928
    20210929
    20210930
    20211001
    20211005
    20211006
    20211007
    20211008
    20211012
    20211013
    20211014
    20211015
    20211018
    20211019
    20211020
    20211021
    20211022
    20211025
    20211026
    20211027
    20211028
    20211029
    20211101
    20211102
    20211103
    20211104
    20211105
    20211108
    20211109
    20211110
    20211111
    20211112
    20211115
    20211116
    20211117
    20211118
    20211119
    20211122
    20211123
    20211124
    20211125
    20211126
    20211129
    20211130
    20211201
    20211202
    20211203
    20211206
    20211207
    20211208
    20211209
    20211210
    20211213
    20211214
    20211215
    20211216
    20211217
    20211220
    20211221
    20211222
    20211223
    20211224
    20211227
    20211228
    20211229
    20211230
    20211231
    20220103
    20220104
    20220105
    20220106
    20220107
    20220110
    20220111
    20220112
    20220113
    20220114
    20220117
    20220118
    20220119
    20220120
    20220121
    20220124
    20220125
    20220126
    20220127
    20220128
    20220131
    20220203
    20220204
    20220207
    20220208
    20220209
    20220210
    20220211
    20220214
    20220215
    20220216
    20220217
    20220218
    20220221
    20220222
    20220223
    20220224
    20220225
    20220228
    20220302
    20220303
    20220304
    20220307
    20220308
    20220310
    20220311
    20220314
    20220315
    20220316
    20220317
    20220318
    20220321
    20220322
    20220323
    20220324
    20220325
    20220328
    20220329
    20220330
    20220331
    20220401
    20220404
    20220405
    20220406
    20220407
    20220408
    20220411
    20220412
    20220413
    20220414
    20220415
    20220418
    20220419
    20220420
    20220421
    20220422
    20220425
    20220426
    20220427
    20220428
    20220429
    20220502
    20220503
    20220504
    20220506
    20220509
    20220510
    20220511
    20220512
    20220513
    20220516
    20220517
    20220518
    20220519
    20220520
    20220523
    20220524
    20220525
    20220526
    20220527
    20220530
    20220531
    20220602
    20220603
    20220607
    20220608
    20220609
    20220610
    20220613
    20220614
    20220615
    20220616
    20220617
    20220620
    20220621
    20220622
    20220623
    20220624
    20220627
    20220628
    20220629
    20220630
    20220701
    20220704
    20220705
    20220706
    20220707
    20220708
    20220711
    20220712
    20220713
    20220714
    20220715
    20220718
    20220719
    20220720
    20220721
    20220722
    20220725
    20220726
    20220727
    20220728
    20220729
    20220801
    20220802
    20220803
    20220804
    20220805
    20220808
    20220809
    20220810
    20220811
    20220812
    20220816
    20220817
    20220818
    20220819
    20220822
    20220823
    20220824
    20220825
    20220826
    20220829
    20220830
    20220831
    20220901
    20220902
    20220905
    20220906
    20220907
    20220908
    20220913
    20220914
    20220915
    20220916
    20220919
    20220920
    20220921
    20220922
    20220923
    20220926
    20220927
    20220928
    20220929
    20220930
    20221004
    20221005
    20221006
    20221007
    20221011
    20221012
    20221013
    20221014
    20221017
    20221018
    20221019
    20221020
    20221021
    20221024
    20221025
    20221026
    ];

rawBusinessDatesSize = length(rawBusinessDates);

dummyfixing = [
255.52
256.66
256.53
255.93
255.99
252.83
252.9
252.57
254.6
255.26
249.67
255.31
251.99
250.01
249.2
250.32
250.46
250.21
249.51
252.37
253.66
252.03
252.26
252.96
252.91
254.26
254.48
252.88
252.14
255.63
256.34
262
260.91
260.72
261.63
262
261.45
262.56
261.93
262.59
262.1
261.97
261.61
261.9
262.49
259.7
260.01
260.36
262.97
263.12
261.98
263.2
263.74
263.73
268.15
269.95
267.91
266.15
267.37
267.08
267.7
266.52
267.12
267.3
268.15
270.48
268.09
269.56
268.49
268.85
269.49
268.97
267.72
267.69
268.66
268.81
267.9
268.81
268.93
268.84
269.73
272.11
272.85
272.89
270.38
268.97
270.06
272.65
269.77
270.41
271.87
272.29
271.62
272.29
275.41
277.89
277.89
280.11
281.86
280.75
283.84
282.63
282.81
282.31
280.36
281.25
281.86
281.66
280.64
281.6
280.92
280.74
279.47
279.01
277.21
275.49
276.21
279.12
277.31
278.1
278.23
276.49
277.76
280.05
281.96
285.41
286.78
287.34
287.21
289.65
292.47
299.86
296.19
299.75
298.25
298.87
299.67
299.34
298.47
298.08
300.63
301.36
301.71
305.22
306.96
306.52
304.59
304.67
304.03
307.83
307.33
305.68
306.25
309.38
306.24
308.02
307.95
306.69
306.79
308.61
309.31
307.52
309.47
310.26
311.89
312.12
310.84
312.56
311.76
312.39
310.46
311.77
311.88
310.66
311.46
313.59
313.58
316.35
317.35
318.52
318.65
318.91
320.61
321.79
322.01
320.15
319.46
320.51
314.13
314.6
317.04
317.69
312.29
313.46
313.96
313.4
309.52
308.37
302.72
304.93
307.16
309.02
308.39
308.14
309.34
309.71
310.73
311.21
309.53
308.64
309.65
308.28
307.78
304.19
304.06
303.18
306.97
307.43
309.83
310.42
310
312.52
313.69
318.93
318.51
318.48
318.25
316.71
316.4
314.29
313.82
313.85
316.27
322.49
326.12
328.11
327.35
327.82
328.51
328.4
326.47
328.89
329.45
328.96
329.31
326.99
329.23
330.38
333.57
338.76
337.53
338.83
337.4
336.76
337.8
337.14
335.96
334.33
333.62
332.17
334.36
334.18
332.92
333.51
335.49
334.51
335.44
329.53
330.99
330.68
325.25
324.78
328.77
329.61
324.52
323.59
324.68
325.09
324.07
326.18
324.74
326.23
326.6
326.76
325.79
319.72
321.21
318.89
320.25
324.74
326
327.13
324.58
328.97
331.28
330.25
327.61
325.67
326.72
327.39
330.02
329.19
329.13
329.52
326.27
330.93
330.91
334.08
335.38
338.05
333.4
333.38
333.36
327.01
323.29
318.01
310.48
311.62
304.83
308.13
310.87
314.4
316.61
312.83
314.47
312.17
316.81
317.34
316.83
312.86
308.35
304.58
310.46
311.14
314.78
317.67
320.7
322.98
321.99
322.74
322.88
320.35
321.79
321.6
323.32
312.42
314.89
316.34
311.81
313.82
314.61
314
313.38
308.54
313.04
311.61
313.45
313.94
312.94
313.22
314.86
315.17
314.64
318.64
320.12
317.89
317.89
315.86
313.99
318.08
320.8
323.56
322.53
320.55
316.75
315.87
314.42
317.31
318.51
317.72
314.94
315.97
314.29
315.37
316.25
318.31
317.48
318.05
318.99
316.2
309.26
310.92
313.22
314.84
315.76
317.67
314.97
317.22
316.84
311.2
308.73
305.35
301.35
304.02
301.02
303.69
303.37
302.76
302.25
298.26
299.66
292.93
293.35
292.26
291.63
293.11
295.2
296.19
294.43
294.41
298.65
297.37
296.88
296.47
295.9
296.95
294.56
295.49
294.85
296.95
297.78
297.29
297.45
298.69
293.55
295.61
295.24
297.18
297.19
297.41
293.64
289.85
291.08
288.24
288.57
288.61
291.93
293
294.29
295.54
296.83
297.22
298.05
298.05
300.07
297.49
298.8
294.96
294.28
292.86
293.7
292.46
292.35
292.42
297.29
295.23
296.1
296.64
299.04
300.81
302.59
300.51
300.17
296.53
291.78
291.28
289.91
287.85
275.15
279.37
277.19
276.9
280.05
277.48
278.63
279.4
272.54
271.36
266.72
262.69
260.19
262.47
264.01
262.95
272.56
270.07
271.57
270.45
272.11
271.33
271.57
269.7
268.88
271.54
271.65
272.31
269.58
268.27
267.76
266.33
269.79
272.08
273.45
274.16
271.35
276.18
273.29
271.49
267.18
268.28
265.35
264.69
268.47
269.41
265.55
266.33
265.18
267.31
264.81
265.06
264.45
260.98
260.57
261.98
261.98
258.23
255.77
257.89
261.57
259.65
265.6
265.55
267.13
264.99
269.93
271.23
271.88
273.88
274
272.71
273.87
276.61
281.67
281.64
283
286.62
285.89
285.79
285.65
281.49
282.1
283.69
284.97
288.68
283.98
285.9
284.91
288.4
288.41
288.74
289
287.89
288.76
283.8
282.51
280.45
279.85
278.87
274.84
275.06
277.73
276
276.64
279.87
279.93
279.82
279.92
282.38
283.13
277.45
277.6
277.28
275.08
276.48
280.32
281.72
285.65
286.15
286.68
286.38
286.71
287.92
287.42
288.37
289.38
290.62
290.33
285.82
285.89
285.87
286.47
283.7
282.83
281.34
286.79
284.92
286.12
283.67
281.31
280.15
271
272.06
268.72
268.57
269.64
265.99
264.35
264.96
266.12
266.65
266.38
264.42
264.4
264.7
261.37
263.93
263.89
268.16
267.66
267.99
268.27
272.1
273.21
272.74
271.13
270.48
270.09
271.36
275.31
276.01
275.48
275.58
275.28
275.53
277.75
277.5
277.27
275.97
272.37
274.54
274.35
268.55
267.72
268.75
271.78
272.68
272.64
274.05
271.48
270.67
274.55
274.55
275.92
273.22
273.04
272.25
267.75
268.95
266.34
265.73
262.99
257.19
253.41
252
252.4
254.85
255.35
252.83
254.86
253.9
255.11
258.05
258.38
256.28
256.11
252.19
253.5
255.53
254.31
259
259.08
258.25
261.69
264.44
265.53
267.4
268.76
270.84
272.02
271.68
273.06
274.92
276.44
276.65
277.97
274.42
274.89
271.33
273.55
274.42
268.55
267.39
267.67
271.36
268.8
271.3
274.23
274.34
276.32
275.57
272.81
273.79
277.41
276.23
276.7
277.07
277.78
277.55
275.51
275.82
278.43
282.54
284.38
284.52
284.37
283.13
280.85
283.65
281.28
283.93
287.29
287.18
286.23
282.21
278.28
279.43
282.19
281.83
282.85
281.23
276.78
277.44
276.19
274.15
273.3
276.56
278.03
279.31
280.28
285.11
290.11
289.76
294.4
294.22
294.31
295.31
294.87
292.9
294.06
295.2
293.77
293.77
290.35
290.74
288.43
291.23
289.42
294.41
297.06
300.13
301.53
299.74
302.78
303.3
305.58
302.11
306.08
302.33
292.77
293.98
288.37
284.53
285.05
290.68
292.02
300.65
298.21
296.24
299.28
301.54
300.93
303.01
302.76
297.74
298.33
296.65
292.42
281.02
284.24
279.94
277.09
268.02
270.37
271.56
278.13
281.38
275.1
263.11
264.67
257.01
247.62
240.65
232.97
226.89
215.83
199.28
213.67
201.87
220.34
232.89
229.34
233.79
232.45
236.82
226.35
231.84
231.7
240.81
244.87
241.89
245.61
248
243.4
247.45
247.1
255.02
252.14
249.4
251.88
253.74
250.28
254.84
256.39
258.15
250.6
255
254.46
256.62
254.95
253.37
255.85
253.65
253.79
255.44
261.79
262.72
263.74
259.62
262.76
267.31
267.64
268.29
268.32
273.19
276.08
285.91
286.45
290.62
290.77
291.32
291.9
288.62
281.78
267.95
282.59
283.02
281.91
283.37
281.42
281.94
286.7
279.73
283.38
278.04
280.09
280.26
283.86
285.89
290.62
286.77
285.97
287.25
285.06
289.84
289.63
292.27
289.25
291.57
290.81
295.16
294.04
292.37
290.66
293.51
300.13
301.25
301.85
299.32
299.46
303.04
306.64
311.32
312.57
316.77
321.02
322.68
323.33
319.24
312.84
313.54
301.59
306.16
309.33
313.59
314.19
311.38
312.24
307.14
309.81
311.5
316.43
312.03
313.67
317.38
313.77
316.53
316.45
320.98
323.36
322.31
318
318.39
315.89
308.82
309.64
302.48
303.57
307.03
309.44
313.51
314.45
317.35
317.47
319.15
319.58
316.57
313.92
311.83
312.59
314.71
315.76
313.77
314.79
313.53
311.8
313.17
309.94
301.6
301.6
305.95
312.07
313.83
321.51
322.1
325.83
326.13
331.12
329.72
333.03
340.63
339.85
339.96
340.16
340.62
348.46
350.28
348.3
352.12
352.73
346.05
352.4
358.66
361.66
367.63
369.86
363.45
371.47
369.37
372.24
371.56
370.88
372.53
372
371.55
372.18
366.43
371.32
379.18
379.93
381.27
389.29
389.29
399.88
406.03
402.08
410.81
430.22
431.13
427.87
430.38
429.85
420.43
410.5
421.35
424.01
430.27
427.13
437.3
426.73
423.67
416.4
404.56
415.45
421.03
425.21
418.31
423.3
418.95
417.91
419.65
427.01
429.24
424.2
417.44
420.94
416.88
415.82
407.16
422.52
409.91
414.8
420.27
414.17
412
408.34
405.79
403.48
411.02
416.13
414.18
417.31
414.01
417.06
412.91
412.64
408.71
407.1
408.53
412.89
412.05
416.61
415.04
419.42
423.74
424.85
425.88
426.7
426.86
424.3
424.22
429.08
430.71
432.66
432.43
431.96
434.6
426.75
427.37
428.34
432.36
432.14
427.48
426.28
422.36
420.36
423.57
426.64
428.56
435.63
428.77
421.86
416.07
420.17
418.27
423.09
421.68
422.08
420.57
424.35
423.54
423
425.95
427.91
430.32
430.15
433.87
432.22
433.26
432.88
428.16
429.35
432.78
433.22
434.13
436.96
434.73
435.02
431.46
434.42
436.73
438.38
440.4
439.98
437.54
438.84
436.34
436
437.66
439.5
436.97
432.63
427.84
431.28
434.71
433.57
436.76
434.82
429.89
428.55
426
431.38
431.55
427.47
428.22
429.15
429.49
423.91
426.79
429.98
436.05
434.74
433.29
431.65
428.56
424.74
422.38
416.06
411.31
413.36
404.96
400.78
404.58
412.48
413.95
410.38
410.24
411.5
419.79
419.92
415.89
419.22
420.37
417.8
414.54
407.28
408.7
409.7
412.36
412.85
409.22
410.99
410.46
409.84
410.97
406.2
400.09
401.3
394.79
388.01
381.93
388.14
387.42
380.69
384.1
389.59
393.19
392.02
395.41
393.74
393.18
393.34
395.01
399.07
395.48
393.89
388.47
389.81
394.82
389.48
391.54
389.86
389.59
390.36
386.24
385.97
390.38
394.36
394.07
389.44
387.44
390.53
399.07
396.44
395.4
391.93
385.07
381.63
373.24
383.13
389.64
391.96
392.85
396.31
397.27
401.13
397.97
397.27
395.41
395.66
397.68
398.96
391.37
393.95
396.07
398.15
400.53
398.61
401.21
396.72
394.19
394.19
395.51
395.4
390.39
386.22
390.85
387.52
388.97
394.66
393
387.45
383.81
381.01
378.21
380.39
376.08
371.3
362.39
360.83
348.58
357.98
357.98
361.6
366.34
362.92
363.48
368.65
371.57
368.61
362.98
360.43
367.3
369.55
368.55
368.29
363.51
364.44
355.04
358.44
361.54
360.59
367.06
361.69
353.03
349.38
358.01
355.31
354.61
351.03
356.09
361.46
362.33
358.77
361.66
364.84
363.38
362.06
362.06
363.77
364.5
365.61
363.19
364.94
364.74
361.01
355.73
356.01
355.78
352.32
359
357.38
353.84
353.63
357.36
357.52
359.02
355.43
348.98
350.67
346.21
350.16
355.08
354.02
353.32
353.85
349
345.41
343.67
343.44
337.94
345.5
343.94
347.53
348.34
343.24
349.29
350.34
344.08
345.6
344.26
348.04
352.49
354.54
350.51
352.01
345.67
346.22
346.62
341.89
329.88
328.23
322.21
322.25
320.09
314.44
316.38
308.08
305.45
312.07
316.2
319.11
312.86
307.2
304.02
304.01
309.03
302.28
308.31
310.15
308.29
305.03
306.59
305.35
307.91
314.87
313.9
316.02
319.68
317.33
318.34
319.61
319.74
321.62
323.31
322.97
321.64
323.8
325.47
327.5
327.23
328.11
324.35
329.92
330.67
331.78
329.91
328.8
326.79
322.77
318.69
319.86
323.63
323.91
316.35
319.49
322.96
315.02
312.92
312.47
313.01
308.16
309.12
318.56
312.74
310.63
308.69
306.49
307.54
304.67
301.65
297.74
289.77
290.2
283.19
282.65
281.36
288.27
289.05
291.26
290.41
284.96
287.09
282.57
289.36
289.57
293.59
291.29
288.61
288.57
291.47
291.58
293.85
];

idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;

idxMax = 366;
idxMax = 1;

for idx2= 1:idxMax
tempDate = H_Date(rawBusinessDates(end));    
tempDateHong = AddDate(tempDate,idx2-1,'day');
% tempDateHong = AddDate(H_Date('2022-11-25'),idx2-1,'day');

valueDateH = tempDateHong;
valueDate = StrDate(valueDateH,'str');

% valueDate = '2022-10-14';
% valueDate = '2022-10-17';

% valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';

% volTargetOptionParams.startDate = H_Date('2022-10-14');
% volTargetOptionParams.endDate = H_Date('2023-10-17');

volTargetOptionParams.startDate = H_Date(rawBusinessDates(1));
volTargetOptionParams.endDate = H_Date(rawBusinessDates(end));

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
% volTargetInfo.volTarget = 0.1;
% volTargetInfo.volTarget = 0.2;
volTargetInfo.volTarget = 30000;


volTargetInfo.volPeriod = 30;
% volTargetInfo.volPeriodParticipationRate = 0.5;
% volTargetInfo.volPeriodParticipationRate = 0.5;

volTargetInfo.volPeriodParticipationRate = 1.0;


volTargetInfo.leverageCap = 1.0;
% volTargetInfo.leverageCap = 0.5;



% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt= zeros(30,4);

volTargetInfo.firstScheduleInt(1:30,1) = rawBusinessDates(1:30);
volTargetInfo.firstScheduleInt(1:30,2) = rawBusinessDates(1:30);
volTargetInfo.firstScheduleInt(1:30,3) = rawBusinessDates(1+1:30+1);
volTargetInfo.firstScheduleInt(1:30,4) = rawBusinessDates(1+1:30+1);

volTargetInfo.lastScheduleInt = zeros(rawBusinessDatesSize-30-1,4);

volTargetInfo.lastScheduleInt(1:rawBusinessDatesSize-30-1,1) = rawBusinessDates(30+1:rawBusinessDatesSize-1);
volTargetInfo.lastScheduleInt(1:rawBusinessDatesSize-30-1,2) = rawBusinessDates(30+1:rawBusinessDatesSize-1);
volTargetInfo.lastScheduleInt(1:rawBusinessDatesSize-30-1,3) = rawBusinessDates(30+1+1:rawBusinessDatesSize-1+1);
volTargetInfo.lastScheduleInt(1:rawBusinessDatesSize-30-1,4) = rawBusinessDates(30+1+1:rawBusinessDatesSize-1+1);
bbb = 1.0;


% [20221014	20221014	20221017	20221017
% 20221017	20221017	20221018	20221018
% 20221018	20221018	20221019	20221019
% 20221019	20221019	20221020	20221020
% 20221020	20221020	20221021	20221021
% 20221021	20221021	20221024	20221024
% 20221024	20221024	20221025	20221025
% 20221025	20221025	20221026	20221026
% 20221026	20221026	20221027	20221027
% 20221027	20221027	20221028	20221028
% 20221028	20221028	20221031	20221031
% 20221031	20221031	20221101	20221101
% 20221101	20221101	20221102	20221102
% 20221102	20221102	20221103	20221103
% 20221103	20221103	20221104	20221104
% 20221104	20221104	20221107	20221107
% 20221107	20221107	20221108	20221108
% 20221108	20221108	20221109	20221109
% 20221109	20221109	20221110	20221110
% 20221110	20221110	20221111	20221111
% 20221111	20221111	20221114	20221114
% 20221114	20221114	20221115	20221115
% 20221115	20221115	20221116	20221116
% 20221116	20221116	20221117	20221117
% 20221117	20221117	20221118	20221118
% 20221118	20221118	20221121	20221121
% 20221121	20221121	20221122	20221122
% 20221122	20221122	20221123	20221123
% 20221123	20221123	20221124	20221124
% 20221124	20221124	20221125	20221125
% ];


% % next remaining fixing schedule with dynamic Participation  reset start end pay 
% volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
% 20221128	20221128	20221129	20221129
% 20221129	20221129	20221130	20221130
% 20221130	20221130	20221201	20221201
% 20221201	20221201	20221202	20221202
% 20221202	20221202	20221205	20221205
% 20221205	20221205	20221206	20221206
% 20221206	20221206	20221207	20221207
% 20221207	20221207	20221208	20221208
% 20221208	20221208	20221209	20221209
% 20221209	20221209	20221212	20221212
% 20221212	20221212	20221213	20221213
% 20221213	20221213	20221214	20221214
% 20221214	20221214	20221215	20221215
% 20221215	20221215	20221216	20221216
% 20221216	20221216	20221219	20221219
% 20221219	20221219	20221220	20221220
% 20221220	20221220	20221221	20221221
% 20221221	20221221	20221222	20221222
% 20221222	20221222	20221223	20221223
% 20221223	20221223	20221226	20221226
% 20221226	20221226	20221227	20221227
% 20221227	20221227	20221228	20221228
% 20221228	20221228	20221229	20221229
% 20221229	20221229	20221230	20221230
% 20221230	20221230	20230102	20230102
% 20230102	20230102	20230103	20230103
% 20230103	20230103	20230104	20230104
% 20230104	20230104	20230105	20230105
% 20230105	20230105	20230106	20230106
% 20230106	20230106	20230109	20230109
% 20230109	20230109	20230110	20230110
% 20230110	20230110	20230111	20230111
% 20230111	20230111	20230112	20230112
% 20230112	20230112	20230113	20230113
% 20230113	20230113	20230116	20230116
% 20230116	20230116	20230117	20230117
% 20230117	20230117	20230118	20230118
% 20230118	20230118	20230119	20230119
% 20230119	20230119	20230120	20230120
% 20230120	20230120	20230125	20230125
% 20230125	20230125	20230126	20230126
% 20230126	20230126	20230127	20230127
% 20230127	20230127	20230130	20230130
% 20230130	20230130	20230131	20230131
% 20230131	20230131	20230201	20230201
% 20230201	20230201	20230202	20230202
% 20230202	20230202	20230203	20230203
% 20230203	20230203	20230206	20230206
% 20230206	20230206	20230207	20230207
% 20230207	20230207	20230208	20230208
% 20230208	20230208	20230209	20230209
% 20230209	20230209	20230210	20230210
% 20230210	20230210	20230213	20230213
% 20230213	20230213	20230214	20230214
% 20230214	20230214	20230215	20230215
% 20230215	20230215	20230216	20230216
% 20230216	20230216	20230217	20230217
% 20230217	20230217	20230220	20230220
% 20230220	20230220	20230221	20230221
% 20230221	20230221	20230222	20230222
% 20230222	20230222	20230223	20230223
% 20230223	20230223	20230224	20230224
% 20230224	20230224	20230227	20230227
% 20230227	20230227	20230228	20230228
% 20230228	20230228	20230302	20230302
% 20230302	20230302	20230303	20230303
% 20230303	20230303	20230306	20230306
% 20230306	20230306	20230307	20230307
% 20230307	20230307	20230308	20230308
% 20230308	20230308	20230309	20230309
% 20230309	20230309	20230310	20230310
% 20230310	20230310	20230313	20230313
% 20230313	20230313	20230314	20230314
% 20230314	20230314	20230315	20230315
% 20230315	20230315	20230316	20230316
% 20230316	20230316	20230317	20230317
% 20230317	20230317	20230320	20230320
% 20230320	20230320	20230321	20230321
% 20230321	20230321	20230322	20230322
% 20230322	20230322	20230323	20230323
% 20230323	20230323	20230324	20230324
% 20230324	20230324	20230327	20230327
% 20230327	20230327	20230328	20230328
% 20230328	20230328	20230329	20230329
% 20230329	20230329	20230330	20230330
% 20230330	20230330	20230331	20230331
% 20230331	20230331	20230403	20230403
% 20230403	20230403	20230404	20230404
% 20230404	20230404	20230405	20230405
% 20230405	20230405	20230406	20230406
% 20230406	20230406	20230407	20230407
% 20230407	20230407	20230410	20230410
% 20230410	20230410	20230411	20230411
% 20230411	20230411	20230412	20230412
% 20230412	20230412	20230413	20230413
% 20230413	20230413	20230414	20230414
% 20230414	20230414	20230417	20230417
% 20230417	20230417	20230418	20230418
% 20230418	20230418	20230419	20230419
% 20230419	20230419	20230420	20230420
% 20230420	20230420	20230421	20230421
% 20230421	20230421	20230424	20230424
% 20230424	20230424	20230425	20230425
% 20230425	20230425	20230426	20230426
% 20230426	20230426	20230427	20230427
% 20230427	20230427	20230428	20230428
% 20230428	20230428	20230502	20230502
% 20230502	20230502	20230503	20230503
% 20230503	20230503	20230504	20230504
% 20230504	20230504	20230508	20230508
% 20230508	20230508	20230509	20230509
% 20230509	20230509	20230510	20230510
% 20230510	20230510	20230511	20230511
% 20230511	20230511	20230512	20230512
% 20230512	20230512	20230515	20230515
% 20230515	20230515	20230516	20230516
% 20230516	20230516	20230517	20230517
% 20230517	20230517	20230518	20230518
% 20230518	20230518	20230519	20230519
% 20230519	20230519	20230522	20230522
% 20230522	20230522	20230523	20230523
% 20230523	20230523	20230524	20230524
% 20230524	20230524	20230525	20230525
% 20230525	20230525	20230526	20230526
% 20230526	20230526	20230529	20230529
% 20230529	20230529	20230530	20230530
% 20230530	20230530	20230531	20230531
% 20230531	20230531	20230601	20230601
% 20230601	20230601	20230602	20230602
% 20230602	20230602	20230605	20230605
% 20230605	20230605	20230607	20230607
% 20230607	20230607	20230608	20230608
% 20230608	20230608	20230609	20230609
% 20230609	20230609	20230612	20230612
% 20230612	20230612	20230613	20230613
% 20230613	20230613	20230614	20230614
% 20230614	20230614	20230615	20230615
% 20230615	20230615	20230616	20230616
% 20230616	20230616	20230619	20230619
% 20230619	20230619	20230620	20230620
% 20230620	20230620	20230621	20230621
% 20230621	20230621	20230622	20230622
% 20230622	20230622	20230623	20230623
% 20230623	20230623	20230626	20230626
% 20230626	20230626	20230627	20230627
% 20230627	20230627	20230628	20230628
% 20230628	20230628	20230629	20230629
% 20230629	20230629	20230630	20230630
% 20230630	20230630	20230703	20230703
% 20230703	20230703	20230704	20230704
% 20230704	20230704	20230705	20230705
% 20230705	20230705	20230706	20230706
% 20230706	20230706	20230707	20230707
% 20230707	20230707	20230710	20230710
% 20230710	20230710	20230711	20230711
% 20230711	20230711	20230712	20230712
% 20230712	20230712	20230713	20230713
% 20230713	20230713	20230714	20230714
% 20230714	20230714	20230717	20230717
% 20230717	20230717	20230718	20230718
% 20230718	20230718	20230719	20230719
% 20230719	20230719	20230720	20230720
% 20230720	20230720	20230721	20230721
% 20230721	20230721	20230724	20230724
% 20230724	20230724	20230725	20230725
% 20230725	20230725	20230726	20230726
% 20230726	20230726	20230727	20230727
% 20230727	20230727	20230728	20230728
% 20230728	20230728	20230731	20230731
% 20230731	20230731	20230801	20230801
% 20230801	20230801	20230802	20230802
% 20230802	20230802	20230803	20230803
% 20230803	20230803	20230804	20230804
% 20230804	20230804	20230807	20230807
% 20230807	20230807	20230808	20230808
% 20230808	20230808	20230809	20230809
% 20230809	20230809	20230810	20230810
% 20230810	20230810	20230811	20230811
% 20230811	20230811	20230814	20230814
% 20230814	20230814	20230816	20230816
% 20230816	20230816	20230817	20230817
% 20230817	20230817	20230818	20230818
% 20230818	20230818	20230821	20230821
% 20230821	20230821	20230822	20230822
% 20230822	20230822	20230823	20230823
% 20230823	20230823	20230824	20230824
% 20230824	20230824	20230825	20230825
% 20230825	20230825	20230828	20230828
% 20230828	20230828	20230829	20230829
% 20230829	20230829	20230830	20230830
% 20230830	20230830	20230831	20230831
% 20230831	20230831	20230901	20230901
% 20230901	20230901	20230904	20230904
% 20230904	20230904	20230905	20230905
% 20230905	20230905	20230906	20230906
% 20230906	20230906	20230907	20230907
% 20230907	20230907	20230908	20230908
% 20230908	20230908	20230911	20230911
% 20230911	20230911	20230912	20230912
% 20230912	20230912	20230913	20230913
% 20230913	20230913	20230914	20230914
% 20230914	20230914	20230915	20230915
% 20230915	20230915	20230918	20230918
% 20230918	20230918	20230919	20230919
% 20230919	20230919	20230920	20230920
% 20230920	20230920	20230921	20230921
% 20230921	20230921	20230922	20230922
% 20230922	20230922	20230925	20230925
% 20230925	20230925	20230926	20230926
% 20230926	20230926	20230927	20230927
% 20230927	20230927	20231002	20231002
% 20231002	20231002	20231004	20231004
% 20231004	20231004	20231005	20231005
% 20231005	20231005	20231006	20231006
% 20231006	20231006	20231010	20231010
% 20231010	20231010	20231011	20231011
% 20231011	20231011	20231012	20231012
% 20231012	20231012	20231013	20231013
% 20231013	20231013	20231016	20231016
% 20231016	20231016	20231017	20231017
% ];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
% rawData = 289.36;
rawData = dummyfixing(end);

% rawDataSim = rawData;
% rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
% rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
% rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
% 20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
% 20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
% 20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
% 20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
% 20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
% 20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
% 20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
% 20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
% 20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
% ];
% 
% for idxhong=2:size(rawData,1)
%     rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
% end


rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
52	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
87	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
143	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
234	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
332	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
423	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
605	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
787	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
969	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
1151	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];


rawDataSim = rawData;

% for idxhh=2:size(rawData,1)
%     for idxjj=2:size(rawData,2)
%         rawDataSim(idxhh,idxjj) = rawDataSim(idxhh,idxjj) + 0.01; 
%     end
% end

rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

%     dummyfixing  = [389.59
% 393.19
% 392.02
% 395.41
% 393.74
% 393.18
% 393.34
% 395.01
% 399.07
% 395.48
% 393.89
% 388.47
% 389.81
% 394.82
% 389.48
% 391.54
% 389.86
% 389.59
% 390.36
% 386.24
% 385.97
% 390.38
% 394.36
% 394.07
% 389.44
% 387.44
% 390.53
% 399.07
% 396.44
% 395.4
% 391.93
% 385.07
% 381.63
% 373.24
% 383.13
% 389.64
% 391.96
% 392.85
% 396.31
% 397.27
% 401.13
% 397.97
% 397.27
% 395.41
% 395.66
% 397.68
% 398.96
% 391.37
% 393.95
% 396.07
% 398.15
% 400.53
% 398.61
% 401.21
% 396.72
% 394.19
% 395.51
% 395.4
% 390.39
% 386.22
% 390.85
% 387.52
% 388.97
% 394.66
% 393
% 387.45
% 383.81
% 381.01
% 378.21
% 380.39
% 376.08
% 371.3
% 362.39
% 360.83
% 348.58
% 357.98
% 361.6
% 366.34
% 362.92
% 363.48
% 368.65
% 371.57
% 368.61
% 362.98
% 360.43
% 367.3
% 369.55
% 368.55
% 368.29
% 363.51
% 364.44
% 355.04
% 358.44
% 361.54
% 360.59
% 367.06
% 361.69
% 353.03
% 349.38
% 358.01
% 355.31
% 354.61
% 351.03
% 356.09
% 361.46
% 362.33
% 358.77
% 361.66
% 364.84
% 363.38
% 362.06
% 362.06
% 363.77
% 364.5
% 365.61
% 363.19
% 364.94
% 364.74
% 361.01
% 355.73
% 356.01
% 355.78
% 352.32
% 359
% 357.38
% 353.84
% 353.63
% 357.36
% 357.52
% 359.02
% 355.43
% 348.98
% 350.67
% 346.21
% 350.16
% 355.08
% 354.02
% 353.32
% 353.85
% 349
% 345.41
% 343.67
% 343.44
% 337.94
% 345.5
% 343.94
% 347.53
% 348.34
% 343.24
% 349.29
% 350.34
% 344.08
% 345.6
% 344.26
% 348.04
% 352.49
% 354.54
% 350.51
% 352.01
% 345.67
% 346.22
% 346.62
% 341.89
% 329.88
% 328.23
% 322.21
% 322.25
% 320.09
% 314.44
% 316.38
% 308.08
% 305.45
% 312.07
% 316.2
% 319.11
% 312.86
% 307.2
% 304.02
% 304.01
% 309.03
% 302.28
% 308.31
% 310.15
% 308.29
% 305.03
% 306.59
% 305.35
% 307.91
% 314.87
% 313.9
% 316.02
% 319.68
% 317.33
% 318.34
% 319.61
% 319.74
% 321.62
% 323.31
% 322.97
% 321.64
% 323.8
% 325.47
% 327.5
% 327.23
% 328.11
% 324.35
% 329.92
% 330.67
% 331.78
% 329.91
% 328.8
% 326.79
% 322.77
% 318.69
% 319.86
% 323.63
% 323.91
% 316.35
% 319.49
% 322.96
% 315.02
% 312.92
% 312.47
% 313.01
% 308.16
% 309.12
% 318.56
% 312.74
% 310.63
% 308.69
% 306.49
% 307.54
% 304.67
% 301.65
% 297.74
% 289.77
% 290.2
% 283.19
% 282.65
% 281.36
% 288.27
% 289.05
% 291.26
% 290.41
% 284.96
% 287.09
% 282.57
% 289.36
% 289.57
% 293.59
% 291.29
% 288.61
% 288.57
% 291.47
% ];
% dummysize = length(dummyfixing);
% 
%     for idxhong=1:10000
%         
%         tempDate = AddDate(tempDate,1,'day');
%         if idxhong<= dummysize
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(idxhong);
%         else
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(end);
%         end
%     end
    
    
    for idxhong=1:rawBusinessDatesSize
        
        tempDate = H_Date(rawBusinessDates(idxhong));
        equityFixing1(StrDate(tempDate,'str')) = dummyfixing(idxhong);
        
    end
    
%     for idxhong=1:10000
%         
%         tempDate = AddDate(tempDate,1,'day');
%         equityFixing1(StrDate(tempDate,'str')) = 289.36;
%         
%     end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;
% numOfRandom = 100;


numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);

if idx2 == 51
   abcd = 1.0; 
end

% pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);
pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC2(valueDateH,volTargetOptionParams,numMethodMC);


callablePrice(idx2) = pricingOutMC.priceOption.npv;

% alphaTrace{idx2,1} = pricingOutMC.alphaTrace;
% volIdxTrace{idx2,1} = pricingOutMC.volIdxTrace;

aaa = 1.0;
 
end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_vega.m

clc
clear

% valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2020-12-18'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;
volTargetInfo.volPeriodParticipationRate = 0.5;
volTargetInfo.leverageCap = 1.0;


% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

% volTargetInfo.lastSchedule_eventType = 'END';
volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end

rawDataSim = rawData;

for idxhh=2:size(rawData,1)
    for idxjj=2:size(rawData,2)
        rawDataSim(idxhh,idxjj) = rawDataSim(idxhh,idxjj) + 0.01; 
    end
end

rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

    
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);


pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);

callablePrice(idx2) = pricingOutMC.priceOption.npv;
aaa = 1.0;
 
%     %% import and construct a discountCurve
% %         rawData = [0.00274	0.004799968
% % 0.25	0.004019196
% % 0.5	0.004772386
% % 0.75	0.005368417
% % 1	0.005819943
% % 2	0.00885494
% % 3	0.009626873
% % 5	0.013470451
% % 10	0.017206717
% % 20	0.018218138
% % 30	0.018261571
% %     ];
% 
% rawData = [0.00274	0.004799968
% 0.25	0.004019196
% 0.5	0.004772386
% 0.75	0.005368417
% 1	0.00581667
% 2	0.008853178
% 3	0.009625741
% 5	0.013468644
% 10	0.017205609
% 20	0.018217501
% 30	0.018261113
%     ];
% 
%     mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                             {rawData,valueDate,'KRW','discountZero'});
%     discountCurveMktData = MktData(mktData);
%     discountCurveMktData.params('convention') = {};
%     discountCurve =  ZeroCurve(discountCurveMktData);
% 
%     KRWAllMktData('discountCurve') = discountCurve;
% 
%     %% import and construct a swaptionVolSurfaceNormal
% 
% 
% 
% % % GIROPremiumToNormalVol From LIB
% % rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% % 1	0.0035	0.0038	0.0041	0.0044	0.0047	0.0049	0.0052	0.0053	0.0050	0.0044
% % 2	0.0042	0.0043	0.0044	0.0045	0.0046	0.0047	0.0049	0.0049	0.0046	0.0040
% % 3	0.0044	0.0044	0.0044	0.0044	0.0044	0.0046	0.0048	0.0049	0.0044	0.0039
% % 4	0.0045	0.0045	0.0045	0.0045	0.0044	0.0046	0.0048	0.0048	0.0042	0.0039
% % 5	0.0046	0.0046	0.0045	0.0045	0.0045	0.0046	0.0047	0.0047	0.0041	0.0039
% % 7	0.0046	0.0046	0.0046	0.0046	0.0045	0.0046	0.0048	0.0048	0.0044	0.0043
% % 10	0.0047	0.0047	0.0047	0.0047	0.0046	0.0047	0.0042	0.0043	0.0042	0.0043
% % 15	0.0032	0.0031	0.0029	0.0028	0.0026	0.0031	0.0033	0.0035	0.0037	0.0040
% % 20	0.0037	0.0037	0.0037	0.0037	0.0037	0.0038	0.0038	0.0039	0.0040	0.0041
% %     ];
% 
% % SwaptionVol From FrontSystem 
% % 20 x 20
% rawDataSwaption = [0	1	2	3	4	5	7		10	12	15	20
% 1	0.00313868	0.00345381	0.00376428	0.00408151	0.00440039	0.00454026		0.00475148	0.00489589	0.00489589	0.00489589
% 2	0.00372804	0.00387327	0.00400155	0.00414319	0.00428737	0.00444983		0.00469545	0.00476574	0.00476574	0.00476574
% 3	0.00390354	0.00401313	0.00409429	0.00418996	0.0042849	0.0044097		0.00459433	0.00464883	0.00464883	0.00464883
% 4	0.00408523	0.00411002	0.00416392	0.00421762	0.00426983	0.00436359		0.00449447	0.00453408	0.00453408	0.00453408
% 5	0.00418943	0.00421751	0.00423044	0.00424642	0.0042546	0.00430589		0.0043945	0.0044123	0.0044123	0.0044123
% 7	0.00421476	0.00425097	0.00425793	0.00428002	0.00430127	0.00436624		0.00446172	0.00448601	0.00448601	0.00448601
% 10	0.0042714	0.0042711	0.00431105	0.00434579	0.00438057	0.00447166		0.00461303	0.00463415	0.00463415	0.00463415
% 15	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022		0.00463187	0.00464424	0.00464424	0.00464424
% 20	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022		0.00463187	0.00464424	0.00464424	0.00464424
%     ];
% 
% % % 15 x 15
% % rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
% % 1	0.00313868	0.00345381	0.00376428	0.00408151	0.00440039	0.00454026	0.00475148	0.00489589	0.00489589
% % 2	0.00372804	0.00387327	0.00400155	0.00414319	0.00428737	0.00444983	0.00469545	0.00476574	0.00476574
% % 3	0.00390354	0.00401313	0.00409429	0.00418996	0.0042849	0.0044097	0.00459433	0.00464883	0.00464883
% % 4	0.00408523	0.00411002	0.00416392	0.00421762	0.00426983	0.00436359	0.00449447	0.00453408	0.00453408
% % 5	0.00418943	0.00421751	0.00423044	0.00424642	0.0042546	0.00430589	0.0043945	0.0044123	0.0044123
% % 7	0.00421476	0.00425097	0.00425793	0.00428002	0.00430127	0.00436624	0.00446172	0.00448601	0.00448601
% % 10	0.0042714	0.0042711	0.00431105	0.00434579	0.00438057	0.00447166	0.00461303	0.00463415	0.00463415
% % 15	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022	0.00463187	0.00464424	0.00464424
% %     ];
% 
%     mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                             {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
%     swaptionVolMktData = MktData(mktData);
%     KOSPI200AllMktData('swaptionVol') = swaptionVolMktData;
% 
%     %% create  a KRWBlackModel to calculate market Swaption
%     modelParams= containers.Map({'modelName'},{'Black'});
%     KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 
% 
%     expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
%     tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
%     swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
%     swaptionPriceSurf.expiry= expiry;
%     swaptionPriceSurf.tenor = tenor;
%     
%     fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
% 
%     tempdHTenor = zeros(size(expiry,1),size(tenor,1));
%     for i=1:size(expiry,1)
%         for j=1:size(tenor,1)
%             fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
%             swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
%             swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
%             tempdHTenor(i,j) = expiry(i) + tenor(j);
%         end
%     end
%    
% %% create a CK1FRisky Model with initial modelParams
% %     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
%     
%     vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
%     meanR_r = 0.03;
% %     meanR_r = 0.0;
%     
%     
%     joint_swap_tenor = 1;
% %     targetSize = 10;
% 
%     targetSize = 20;
%     
%     freq = 4;
%     
%     
%     matu = DateDiff(volTargetOptionParams.endDate,valueDateH)/365.0;
%     
%     modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
%                             {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
% 
%     %% allFixings
%     %% CMS10Y & CMS2Y
%     tempDate = volTargetOptionParams.startDate;
% 
%     %CMS10Y
%     irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});
% 
%     %CMS2Y
%     irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});
% 
%     %LIBOR3M
%     irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.006466038});
% 
%     for idxhong=1:10000
%         tempDate = AddDate(tempDate,1,'day');
%         irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
%         irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
%         irIndexFixing3(StrDate(tempDate,'str')) = 0.006466038;
%     end
% 
%     %% fixing info -> let it be the modelParams !
% 
%     allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;
% 
%     modelParams('allFixings') = allFixings;
%     
%     KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
%     
% %     tic
% 
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
%     
%     for idx3 = 1:length(out.vol_r.quote)
%         KRWVol_r(1,idx3) = out.vol_r.quote(idx3);
%         KRWRe(1,idx3) = out.re(idx3);
%     end
%     KRWRMSE(1) = out.rmseTotal;
%     
% %     toc
%     %% CK1FRisky model building end
%     %% KRW model building end
%     
% %   %% create a LGM2FModel with initial modelParams
% % % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
% % tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
% % dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
% % dH2.tenor = dH1.tenor;
% % 
% % %swaption only initial parameters
% % % dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
% % 
% % %Samsung
% % % dHTest = [0.058507682	0.500210296
% % % 0.06145709	0.515096302
% % % 0.057841075	0.632558982
% % % 0.069442883	0.632845436
% % % 0.081752868	0.607322125
% % % 0.087574376	0.586095839
% % % 0.103284643	0.52688632
% % % 0.113936628	0.552598585
% % % 0.131046999	0.498409755
% % % 0.153553385	0.437472215
% % % 0.173381122	0.423805445
% % % 0.180703372	0.380344606
% % % 0.207484315	0.305927652
% % % 0.218329282	0.268192317
% % % 0.224885432	0.227191192
% % % 0.221244544	0.231204789
% % % 0.262502203	0.212209604
% % % 0.264579505	0.135257044
% % % 0.318821555	0.118179799
% % % 0.271515922	0.107957745
% % % 0.358024755	0.088834433
% % % 0.33143161	0.069384916
% % % 0.457059078	0.033745097
% % % 0.374067262	0.006573072
% % % ];
% % 
% % %Daewoo
% % % dHTest = [0.037286763	0.525314433
% % % 0.037286763	0.525314433
% % % 0.054571525	0.586928199
% % % 0.069289617	0.618362181
% % % 0.099111958	0.605543022
% % % 0.1123501	0.589459841
% % % 0.139006516	0.523225356
% % % 0.175987354	0.505500158
% % % 0.184616726	0.463242812
% % % 0.204164537	0.412151488
% % % 0.263444475	0.364947643
% % % 0.251615512	0.358337407
% % % 0.259708458	0.315375986
% % % 0.269854863	0.270026782
% % % 0.337623406	0.253701114
% % % 0.258270519	0.179914465
% % % 0.321985902	0.191343978
% % % 0.347503907	0.154371663
% % % 0.36490148	0.127052474
% % % 0.36706888	0.095165077
% % % 0.480469769	0.05084228
% % % 0.423787026	0.033262976
% % % 0.531944266	0.006463726
% % % 0.333342463	0.000100001
% % % ];
% % 
% % % %Sungwoo
% % % dHTest = [0.060654929	0.585843811
% % % 0.065181321	0.621357434
% % % 0.059213061	0.705433024
% % % 0.070356009	0.697274699
% % % 0.084854535	0.641491331
% % % 0.078986949	0.60204605
% % % 0.103979263	0.504724505
% % % 0.128905871	0.522635178
% % % 0.142868097	0.436948669
% % % 0.167927649	0.341253642
% % % 0.171414266	0.384418469
% % % 0.173137707	0.332946905
% % % 0.205465936	0.252922264
% % % 0.190811162	0.233849244
% % % 0.212134317	0.235149902
% % % 0.208305697	0.261176412
% % % 0.267225845	0.213982892
% % % 0.244169602	0.131637885
% % % 0.293167102	0.110338057
% % % 0.302235653	0.107810085
% % % 0.341199539	0.095697121
% % % 0.353976	0.072468098
% % % 0.421517994	0.033547027
% % % 0.357938773	0.006568033
% % % ];
% % 
% % %JaeYeol
% % dHTest = [0.058982775	0.543001377
% % 0.061234997	0.56742444
% % 0.062961086	0.622386628
% % 0.067490351	0.649674014
% % 0.075410597	0.63934757
% % 0.087663903	0.593591285
% % 0.112669112	0.501377655
% % 0.117220704	0.537782529
% % 0.140692951	0.444873509
% % 0.168027009	0.390384507
% % 0.182609655	0.410559905
% % 0.184911175	0.327958872
% % 0.214621342	0.297359459
% % 0.213082379	0.268233279
% % 0.228595305	0.243594185
% % 0.220204578	0.241085585
% % 0.27611566	0.202850389
% % 0.244581841	0.134958255
% % 0.314853582	0.120816977
% % 0.317684576	0.109773131
% % 0.325368543	0.100229783
% % 0.348723657	0.073457815
% % 0.31747133	0.033017077
% % 0.359049924	0.006573421
% % ];
% % 
% % dH1.quote = dHTest(:,1);
% % dH2.quote = dHTest(:,2);
% % 
% % 
% % dH1Orig = dH1;
% % dH2Orig = dH2;
% % 
% % % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
% % Alpha1.tenor =  expiry;
% % Alpha2.tenor =  Alpha1.tenor;
% % correl.tenor = Alpha1.tenor;
% % correl.quote = zeros(length(expiry),1);
% % 
% % %swaption only initial parameters
% % 
% % %Samsung
% % % AlphaTest = [0.0289581	0.006633509
% % % 0.021585088	0.007003076
% % % 0.019812596	0.007056061
% % % 0.015014178	0.007801684
% % % 0.008683651	0.008360616
% % % 0.011075137	0.008248663
% % % 0.008794173	0.011226664
% % % 0.000329846	0.023982837
% % % ];
% % 
% % %Daewoo
% % % AlphaTest = [0.029142908	0.007983918
% % % 0.016018045	0.007219568
% % % 0.00912537	0.007233927
% % % 0.008101129	0.007774553
% % % 0.012850209	0.008916743
% % % 0.011065789	0.009477842
% % % 0.008930619	0.011214209
% % % 0.0001	0.032096513
% % % ];
% % 
% % % %sungwoo
% % % AlphaTest = [0.032526561	0.005901286
% % % 0.021888486	0.006159185
% % % 0.015500653	0.006183779
% % % 0.011483817	0.007702683
% % % 0.008112119	0.008910715
% % % 0.010323471	0.008310996
% % % 0.006966815	0.012340985
% % % 0.000302754	0.020086129
% % % ];
% % 
% % %jaeweol
% % AlphaTest = [0.027941111	0.006258382
% % 0.021327126	0.006691243
% % 0.015009253	0.006502172
% % 0.012188678	0.007718057
% % 0.010231971	0.009065476
% % 0.006437447	0.009477921
% % 0.004646654	0.012890028
% % 0.004203719	0.024248438
% % ];
% % 
% % 
% % Alpha1.quote = AlphaTest(:,1);
% % Alpha2.quote = AlphaTest(:,2);
% % 
% % smoothdH = 1.0;
% % smoothAlpha = 1.0;
% % smoothTermCorr =0.1;
% % targetTermCorr = 0.92;
% % 
% % freq = 4;
% % 
% % modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
% %                         {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% % % import irModels.*;
% % % import targetFunctions.*;
% %                     
% % KRWLGM = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));
% % 
% % 
% % tic
% % % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
% % out = KRWLGM.CalibrateToATMSwaptionSurfaceNormal(KRWBlack,'global',200);
% % % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
% % 
% % dH1Calib = KRWLGM.dH1;
% % dH2Calib = KRWLGM.dH2;
% % Alpha1Calib = KRWLGM.Alpha1;
% % Alpha2Calib = KRWLGM.Alpha2;
% % 
% % modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
% %                         {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});
% %                     
% %                     
% % %% allFixings
% % %% CMS10Y & CMS2Y
% % tempDate = volTargetOptionParams.startDate;
% % 
% % %CMS10Y
% % irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});
% % 
% % %CMS2Y
% % irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});
% % 
% % %LIBOR3M
% % irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.0063});
% % 
% % for idxhong=1:10000
% %     tempDate = AddDate(tempDate,1,'day');
% %     irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
% %     irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
% %     irIndexFixing3(StrDate(tempDate,'str')) = 0.0063;
% % end
% % 
% % %% fixing info -> let it be the modelParams !
% % 
% % allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
% % allFixings('CMS2Y') = irIndexFixing2;
% % allFixings('LIBOR3M') = irIndexFixing3;
% % 
% % modelParamsCalib('allFixings') = allFixings;
% % 
% % KRWLGM2FRisky = LGM2FRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
% %     
% % 
% % toc
% %     %% LGM2FRisky model building end
% %     %% KRW model building end
% 
%     %% NumMethod Init Start
%     numOfRandom = 40000;
%     
%     numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
%                             {'AMC', 7, numOfRandom});
% 
%     numMethodAMC = NumMethod(numMethodInfo);
%     %% NumMethod Init End
%     tic
%     pricingOutMC1 = KRWCK1FRisky.computeInverseFloaterSwapGeneric2(valueDateH,volTargetOptionParams,numMethodAMC);
% %     pricingOutMC1 = KRWCK1FRisky.computeInverseFloaterSwapGeneric2_LSMC(valueDateH,volTargetOptionParams,numMethodAMC);
% %     pricingOutMC1 = KRWCK1FRisky.computeInverseFloaterSwapGeneric3_LSMC(valueDateH,volTargetOptionParams,numMethodAMC);
%     toc
%     aaa = 1.0;
% 
%     
%     callablePrice(idx2) = pricingOutMC1.callable.npv;
%     nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     nonCallBPrice(idx2) = pricingOutMC1.nonCallB.npv;
%     eventTimeStepT= pricingOutMC1.eventTimeStep;
%     eventTimeStepTSize = length(eventTimeStepT);
%     eventTimeStep(1:eventTimeStepTSize,idx2) = pricingOutMC1.eventTimeStep(:);
end

aaa =1.0;



In [ ]:

test_VolTargetOption_kospi200_20221021_simul_theta_part05.m

clc
clear

% valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 366;
maxIdx = 366;
for idx2= 1:idxMax
%     
tempDateHong = AddDate(H_Date('2022-10-17'),idx2-1,'day');
valueDateH = tempDateHong;
% valueDate = '2022-10-14';
% valueDate = '2022-10-17';
valueDate = StrDate(valueDateH,'str');

% valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
% volTargetInfo.multiplier = 1.0;

volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;

% volTargetInfo.volPeriodParticipationRate = 0.5;
% volTargetInfo.volPeriodParticipationRate = 1.0;
volTargetInfo.volPeriodParticipationRate = 0.5;


volTargetInfo.leverageCap = 1.0;


% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

% rawDataSim = rawData;
% rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
% rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end
    

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

    
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));

out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);

if idx2 == 50
   abcd = 1.0; 
end
    
pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);

callablePrice(idx2) = pricingOutMC.priceOption.npv;
aaa = 1.0;
 

end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_theta.m

clc
clear

% valueDate = '2020-12-18';

    
alphaTrace = cell(100,1);
volIdxTrace = cell(100,1);

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start

rawBusinessDates = [
    20161026
    20161027
    20161028
    20161031
    20161101
    20161102
    20161103
    20161104
    20161107
    20161108
    20161109
    20161110
    20161111
    20161114
    20161115
    20161116
    20161117
    20161118
    20161121
    20161122
    20161123
    20161124
    20161125
    20161128
    20161129
    20161130
    20161201
    20161202
    20161205
    20161206
    20161207
    20161208
    20161209
    20161212
    20161213
    20161214
    20161215
    20161216
    20161219
    20161220
    20161221
    20161222
    20161223
    20161226
    20161227
    20161228
    20161229
    20170102
    20170103
    20170104
    20170105
    20170106
    20170109
    20170110
    20170111
    20170112
    20170113
    20170116
    20170117
    20170118
    20170119
    20170120
    20170123
    20170124
    20170125
    20170126
    20170131
    20170201
    20170202
    20170203
    20170206
    20170207
    20170208
    20170209
    20170210
    20170213
    20170214
    20170215
    20170216
    20170217
    20170220
    20170221
    20170222
    20170223
    20170224
    20170227
    20170228
    20170302
    20170303
    20170306
    20170307
    20170308
    20170309
    20170310
    20170313
    20170314
    20170315
    20170316
    20170317
    20170320
    20170321
    20170322
    20170323
    20170324
    20170327
    20170328
    20170329
    20170330
    20170331
    20170403
    20170404
    20170405
    20170406
    20170407
    20170410
    20170411
    20170412
    20170413
    20170414
    20170417
    20170418
    20170419
    20170420
    20170421
    20170424
    20170425
    20170426
    20170427
    20170428
    20170502
    20170504
    20170508
    20170510
    20170511
    20170512
    20170515
    20170516
    20170517
    20170518
    20170519
    20170522
    20170523
    20170524
    20170525
    20170526
    20170529
    20170530
    20170531
    20170601
    20170602
    20170605
    20170607
    20170608
    20170609
    20170612
    20170613
    20170614
    20170615
    20170616
    20170619
    20170620
    20170621
    20170622
    20170623
    20170626
    20170627
    20170628
    20170629
    20170630
    20170703
    20170704
    20170705
    20170706
    20170707
    20170710
    20170711
    20170712
    20170713
    20170714
    20170717
    20170718
    20170719
    20170720
    20170721
    20170724
    20170725
    20170726
    20170727
    20170728
    20170731
    20170801
    20170802
    20170803
    20170804
    20170807
    20170808
    20170809
    20170810
    20170811
    20170814
    20170816
    20170817
    20170818
    20170821
    20170822
    20170823
    20170824
    20170825
    20170828
    20170829
    20170830
    20170831
    20170901
    20170904
    20170905
    20170906
    20170907
    20170908
    20170911
    20170912
    20170913
    20170914
    20170915
    20170918
    20170919
    20170920
    20170921
    20170922
    20170925
    20170926
    20170927
    20170928
    20170929
    20171010
    20171011
    20171012
    20171013
    20171016
    20171017
    20171018
    20171019
    20171020
    20171023
    20171024
    20171025
    20171026
    20171027
    20171030
    20171031
    20171101
    20171102
    20171103
    20171106
    20171107
    20171108
    20171109
    20171110
    20171113
    20171114
    20171115
    20171116
    20171117
    20171120
    20171121
    20171122
    20171123
    20171124
    20171127
    20171128
    20171129
    20171130
    20171201
    20171204
    20171205
    20171206
    20171207
    20171208
    20171211
    20171212
    20171213
    20171214
    20171215
    20171218
    20171219
    20171220
    20171221
    20171222
    20171226
    20171227
    20171228
    20180102
    20180103
    20180104
    20180105
    20180108
    20180109
    20180110
    20180111
    20180112
    20180115
    20180116
    20180117
    20180118
    20180119
    20180122
    20180123
    20180124
    20180125
    20180126
    20180129
    20180130
    20180131
    20180201
    20180202
    20180205
    20180206
    20180207
    20180208
    20180209
    20180212
    20180213
    20180214
    20180219
    20180220
    20180221
    20180222
    20180223
    20180226
    20180227
    20180228
    20180302
    20180305
    20180306
    20180307
    20180308
    20180309
    20180312
    20180313
    20180314
    20180315
    20180316
    20180319
    20180320
    20180321
    20180322
    20180323
    20180326
    20180327
    20180328
    20180329
    20180330
    20180402
    20180403
    20180404
    20180405
    20180406
    20180409
    20180410
    20180411
    20180412
    20180413
    20180416
    20180417
    20180418
    20180419
    20180420
    20180423
    20180424
    20180425
    20180426
    20180427
    20180430
    20180502
    20180503
    20180504
    20180508
    20180509
    20180510
    20180511
    20180514
    20180515
    20180516
    20180517
    20180518
    20180521
    20180523
    20180524
    20180525
    20180528
    20180529
    20180530
    20180531
    20180601
    20180604
    20180605
    20180607
    20180608
    20180611
    20180612
    20180614
    20180615
    20180618
    20180619
    20180620
    20180621
    20180622
    20180625
    20180626
    20180627
    20180628
    20180629
    20180702
    20180703
    20180704
    20180705
    20180706
    20180709
    20180710
    20180711
    20180712
    20180713
    20180716
    20180717
    20180718
    20180719
    20180720
    20180723
    20180724
    20180725
    20180726
    20180727
    20180730
    20180731
    20180801
    20180802
    20180803
    20180806
    20180807
    20180808
    20180809
    20180810
    20180813
    20180814
    20180816
    20180817
    20180820
    20180821
    20180822
    20180823
    20180824
    20180827
    20180828
    20180829
    20180830
    20180831
    20180903
    20180904
    20180905
    20180906
    20180907
    20180910
    20180911
    20180912
    20180913
    20180914
    20180917
    20180918
    20180919
    20180920
    20180921
    20180927
    20180928
    20181001
    20181002
    20181004
    20181005
    20181008
    20181010
    20181011
    20181012
    20181015
    20181016
    20181017
    20181018
    20181019
    20181022
    20181023
    20181024
    20181025
    20181026
    20181029
    20181030
    20181031
    20181101
    20181102
    20181105
    20181106
    20181107
    20181108
    20181109
    20181112
    20181113
    20181114
    20181115
    20181116
    20181119
    20181120
    20181121
    20181122
    20181123
    20181126
    20181127
    20181128
    20181129
    20181130
    20181203
    20181204
    20181205
    20181206
    20181207
    20181210
    20181211
    20181212
    20181213
    20181214
    20181217
    20181218
    20181219
    20181220
    20181221
    20181224
    20181226
    20181227
    20181228
    20181231
    20190102
    20190103
    20190104
    20190107
    20190108
    20190109
    20190110
    20190111
    20190114
    20190115
    20190116
    20190117
    20190118
    20190121
    20190122
    20190123
    20190124
    20190125
    20190128
    20190129
    20190130
    20190131
    20190201
    20190207
    20190208
    20190211
    20190212
    20190213
    20190214
    20190215
    20190218
    20190219
    20190220
    20190221
    20190222
    20190225
    20190226
    20190227
    20190228
    20190304
    20190305
    20190306
    20190307
    20190308
    20190311
    20190312
    20190313
    20190314
    20190315
    20190318
    20190319
    20190320
    20190321
    20190322
    20190325
    20190326
    20190327
    20190328
    20190329
    20190401
    20190402
    20190403
    20190404
    20190405
    20190408
    20190409
    20190410
    20190411
    20190412
    20190415
    20190416
    20190417
    20190418
    20190419
    20190422
    20190423
    20190424
    20190425
    20190426
    20190429
    20190430
    20190502
    20190503
    20190507
    20190508
    20190509
    20190510
    20190513
    20190514
    20190515
    20190516
    20190517
    20190520
    20190521
    20190522
    20190523
    20190524
    20190527
    20190528
    20190529
    20190530
    20190531
    20190603
    20190604
    20190605
    20190607
    20190610
    20190611
    20190612
    20190613
    20190614
    20190617
    20190618
    20190619
    20190620
    20190621
    20190624
    20190625
    20190626
    20190627
    20190628
    20190701
    20190702
    20190703
    20190704
    20190705
    20190708
    20190709
    20190710
    20190711
    20190712
    20190715
    20190716
    20190717
    20190718
    20190719
    20190722
    20190723
    20190724
    20190725
    20190726
    20190729
    20190730
    20190731
    20190801
    20190802
    20190805
    20190806
    20190807
    20190808
    20190809
    20190812
    20190813
    20190814
    20190816
    20190819
    20190820
    20190821
    20190822
    20190823
    20190826
    20190827
    20190828
    20190829
    20190830
    20190902
    20190903
    20190904
    20190905
    20190906
    20190909
    20190910
    20190911
    20190916
    20190917
    20190918
    20190919
    20190920
    20190923
    20190924
    20190925
    20190926
    20190927
    20190930
    20191001
    20191002
    20191004
    20191007
    20191008
    20191010
    20191011
    20191014
    20191015
    20191016
    20191017
    20191018
    20191021
    20191022
    20191023
    20191024
    20191025
    20191028
    20191029
    20191030
    20191031
    20191101
    20191104
    20191105
    20191106
    20191107
    20191108
    20191111
    20191112
    20191113
    20191114
    20191115
    20191118
    20191119
    20191120
    20191121
    20191122
    20191125
    20191126
    20191127
    20191128
    20191129
    20191202
    20191203
    20191204
    20191205
    20191206
    20191209
    20191210
    20191211
    20191212
    20191213
    20191216
    20191217
    20191218
    20191219
    20191220
    20191223
    20191224
    20191226
    20191227
    20191230
    20191231
    20200102
    20200103
    20200106
    20200107
    20200108
    20200109
    20200110
    20200113
    20200114
    20200115
    20200116
    20200117
    20200120
    20200121
    20200122
    20200123
    20200128
    20200129
    20200130
    20200131
    20200203
    20200204
    20200205
    20200206
    20200207
    20200210
    20200211
    20200212
    20200213
    20200214
    20200217
    20200218
    20200219
    20200220
    20200221
    20200224
    20200225
    20200226
    20200227
    20200228
    20200302
    20200303
    20200304
    20200305
    20200306
    20200309
    20200310
    20200311
    20200312
    20200313
    20200316
    20200317
    20200318
    20200319
    20200320
    20200323
    20200324
    20200325
    20200326
    20200327
    20200330
    20200331
    20200401
    20200402
    20200403
    20200406
    20200407
    20200408
    20200409
    20200410
    20200413
    20200414
    20200416
    20200417
    20200420
    20200421
    20200422
    20200423
    20200424
    20200427
    20200428
    20200429
    20200504
    20200506
    20200507
    20200508
    20200511
    20200512
    20200513
    20200514
    20200515
    20200518
    20200519
    20200520
    20200521
    20200522
    20200525
    20200526
    20200527
    20200528
    20200529
    20200601
    20200602
    20200603
    20200604
    20200605
    20200608
    20200609
    20200610
    20200611
    20200612
    20200615
    20200616
    20200617
    20200618
    20200619
    20200622
    20200623
    20200624
    20200625
    20200626
    20200629
    20200630
    20200701
    20200702
    20200703
    20200706
    20200707
    20200708
    20200709
    20200710
    20200713
    20200714
    20200715
    20200716
    20200717
    20200720
    20200721
    20200722
    20200723
    20200724
    20200727
    20200728
    20200729
    20200730
    20200731
    20200803
    20200804
    20200805
    20200806
    20200807
    20200810
    20200811
    20200812
    20200813
    20200814
    20200818
    20200819
    20200820
    20200821
    20200824
    20200825
    20200826
    20200827
    20200828
    20200831
    20200901
    20200902
    20200903
    20200904
    20200907
    20200908
    20200909
    20200910
    20200911
    20200914
    20200915
    20200916
    20200917
    20200918
    20200921
    20200922
    20200923
    20200924
    20200925
    20200928
    20200929
    20201005
    20201006
    20201007
    20201008
    20201012
    20201013
    20201014
    20201015
    20201016
    20201019
    20201020
    20201021
    20201022
    20201023
    20201026
    20201027
    20201028
    20201029
    20201030
    20201031
    20201102
    20201103
    20201104
    20201105
    20201106
    20201109
    20201110
    20201111
    20201112
    20201113
    20201116
    20201117
    20201118
    20201119
    20201120
    20201123
    20201124
    20201125
    20201126
    20201127
    20201130
    20201201
    20201202
    20201203
    20201204
    20201207
    20201208
    20201209
    20201210
    20201211
    20201214
    20201215
    20201216
    20201217
    20201218
    20201221
    20201222
    20201223
    20201224
    20201228
    20201229
    20201230
    20201231
    20210104
    20210105
    20210106
    20210107
    20210108
    20210111
    20210112
    20210113
    20210114
    20210115
    20210118
    20210119
    20210120
    20210121
    20210122
    20210125
    20210126
    20210127
    20210128
    20210129
    20210201
    20210202
    20210203
    20210204
    20210205
    20210208
    20210209
    20210210
    20210215
    20210216
    20210217
    20210218
    20210219
    20210222
    20210223
    20210224
    20210225
    20210226
    20210302
    20210303
    20210304
    20210305
    20210308
    20210309
    20210310
    20210311
    20210312
    20210315
    20210316
    20210317
    20210318
    20210319
    20210322
    20210323
    20210324
    20210325
    20210326
    20210329
    20210330
    20210331
    20210401
    20210402
    20210405
    20210406
    20210407
    20210408
    20210409
    20210412
    20210413
    20210414
    20210415
    20210416
    20210419
    20210420
    20210421
    20210422
    20210423
    20210426
    20210427
    20210428
    20210429
    20210430
    20210503
    20210504
    20210506
    20210507
    20210510
    20210511
    20210512
    20210513
    20210514
    20210517
    20210518
    20210520
    20210521
    20210524
    20210525
    20210526
    20210527
    20210528
    20210531
    20210601
    20210602
    20210603
    20210604
    20210607
    20210608
    20210609
    20210610
    20210611
    20210614
    20210615
    20210616
    20210617
    20210618
    20210621
    20210622
    20210623
    20210624
    20210625
    20210628
    20210629
    20210630
    20210701
    20210702
    20210705
    20210706
    20210707
    20210708
    20210709
    20210712
    20210713
    20210714
    20210715
    20210716
    20210719
    20210720
    20210721
    20210722
    20210723
    20210726
    20210727
    20210728
    20210729
    20210730
    20210802
    20210803
    20210804
    20210805
    20210806
    20210809
    20210810
    20210811
    20210812
    20210813
    20210817
    20210818
    20210819
    20210820
    20210823
    20210824
    20210825
    20210826
    20210827
    20210830
    20210831
    20210901
    20210902
    20210903
    20210906
    20210907
    20210908
    20210909
    20210910
    20210913
    20210914
    20210915
    20210916
    20210917
    20210923
    20210924
    20210927
    20210928
    20210929
    20210930
    20211001
    20211005
    20211006
    20211007
    20211008
    20211012
    20211013
    20211014
    20211015
    20211018
    20211019
    20211020
    20211021
    20211022
    20211025
    20211026
    20211027
    20211028
    20211029
    20211101
    20211102
    20211103
    20211104
    20211105
    20211108
    20211109
    20211110
    20211111
    20211112
    20211115
    20211116
    20211117
    20211118
    20211119
    20211122
    20211123
    20211124
    20211125
    20211126
    20211129
    20211130
    20211201
    20211202
    20211203
    20211206
    20211207
    20211208
    20211209
    20211210
    20211213
    20211214
    20211215
    20211216
    20211217
    20211220
    20211221
    20211222
    20211223
    20211224
    20211227
    20211228
    20211229
    20211230
    20211231
    20220103
    20220104
    20220105
    20220106
    20220107
    20220110
    20220111
    20220112
    20220113
    20220114
    20220117
    20220118
    20220119
    20220120
    20220121
    20220124
    20220125
    20220126
    20220127
    20220128
    20220131
    20220203
    20220204
    20220207
    20220208
    20220209
    20220210
    20220211
    20220214
    20220215
    20220216
    20220217
    20220218
    20220221
    20220222
    20220223
    20220224
    20220225
    20220228
    20220302
    20220303
    20220304
    20220307
    20220308
    20220310
    20220311
    20220314
    20220315
    20220316
    20220317
    20220318
    20220321
    20220322
    20220323
    20220324
    20220325
    20220328
    20220329
    20220330
    20220331
    20220401
    20220404
    20220405
    20220406
    20220407
    20220408
    20220411
    20220412
    20220413
    20220414
    20220415
    20220418
    20220419
    20220420
    20220421
    20220422
    20220425
    20220426
    20220427
    20220428
    20220429
    20220502
    20220503
    20220504
    20220506
    20220509
    20220510
    20220511
    20220512
    20220513
    20220516
    20220517
    20220518
    20220519
    20220520
    20220523
    20220524
    20220525
    20220526
    20220527
    20220530
    20220531
    20220602
    20220603
    20220607
    20220608
    20220609
    20220610
    20220613
    20220614
    20220615
    20220616
    20220617
    20220620
    20220621
    20220622
    20220623
    20220624
    20220627
    20220628
    20220629
    20220630
    20220701
    20220704
    20220705
    20220706
    20220707
    20220708
    20220711
    20220712
    20220713
    20220714
    20220715
    20220718
    20220719
    20220720
    20220721
    20220722
    20220725
    20220726
    20220727
    20220728
    20220729
    20220801
    20220802
    20220803
    20220804
    20220805
    20220808
    20220809
    20220810
    20220811
    20220812
    20220816
    20220817
    20220818
    20220819
    20220822
    20220823
    20220824
    20220825
    20220826
    20220829
    20220830
    20220831
    20220901
    20220902
    20220905
    20220906
    20220907
    20220908
    20220913
    20220914
    20220915
    20220916
    20220919
    20220920
    20220921
    20220922
    20220923
    20220926
    20220927
    20220928
    20220929
    20220930
    20221004
    20221005
    20221006
    20221007
    20221011
    20221012
    20221013
    20221014
    20221017
    20221018
    20221019
    20221020
    20221021
    20221024
    20221025
    20221026
    ];

idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;

idxMax = 366;
idxMax = 1;

for idx2= 1:idxMax
%     
tempDateHong = AddDate(H_Date('2022-10-17'),idx2-1,'day');
% tempDateHong = AddDate(H_Date('2022-11-25'),idx2-1,'day');

valueDateH = tempDateHong;
valueDate = StrDate(valueDateH,'str');

% valueDate = '2022-10-14';
% valueDate = '2022-10-17';

% valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;
% volTargetInfo.volPeriodParticipationRate = 0.5;
volTargetInfo.volPeriodParticipationRate = 0.5;
% volTargetInfo.volPeriodParticipationRate = 1.0;


volTargetInfo.leverageCap = 1.0;
% volTargetInfo.leverageCap = 0.5;



% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt= zeros(30,4);

volTargetInfo.firstScheduleInt(1:30,1) = ;

% [20221014	20221014	20221017	20221017
% 20221017	20221017	20221018	20221018
% 20221018	20221018	20221019	20221019
% 20221019	20221019	20221020	20221020
% 20221020	20221020	20221021	20221021
% 20221021	20221021	20221024	20221024
% 20221024	20221024	20221025	20221025
% 20221025	20221025	20221026	20221026
% 20221026	20221026	20221027	20221027
% 20221027	20221027	20221028	20221028
% 20221028	20221028	20221031	20221031
% 20221031	20221031	20221101	20221101
% 20221101	20221101	20221102	20221102
% 20221102	20221102	20221103	20221103
% 20221103	20221103	20221104	20221104
% 20221104	20221104	20221107	20221107
% 20221107	20221107	20221108	20221108
% 20221108	20221108	20221109	20221109
% 20221109	20221109	20221110	20221110
% 20221110	20221110	20221111	20221111
% 20221111	20221111	20221114	20221114
% 20221114	20221114	20221115	20221115
% 20221115	20221115	20221116	20221116
% 20221116	20221116	20221117	20221117
% 20221117	20221117	20221118	20221118
% 20221118	20221118	20221121	20221121
% 20221121	20221121	20221122	20221122
% 20221122	20221122	20221123	20221123
% 20221123	20221123	20221124	20221124
% 20221124	20221124	20221125	20221125
% ];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

% rawDataSim = rawData;
% rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
% rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
% rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
% 20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
% 20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
% 20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
% 20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
% 20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
% 20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
% 20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
% 20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
% 20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
% ];
% 
% for idxhong=2:size(rawData,1)
%     rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
% end


rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
52	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
87	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
143	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
234	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
332	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
423	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
605	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
787	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
969	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
1151	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];


rawDataSim = rawData;

% for idxhh=2:size(rawData,1)
%     for idxjj=2:size(rawData,2)
%         rawDataSim(idxhh,idxjj) = rawDataSim(idxhh,idxjj) + 0.01; 
%     end
% end

rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

%     dummyfixing  = [389.59
% 393.19
% 392.02
% 395.41
% 393.74
% 393.18
% 393.34
% 395.01
% 399.07
% 395.48
% 393.89
% 388.47
% 389.81
% 394.82
% 389.48
% 391.54
% 389.86
% 389.59
% 390.36
% 386.24
% 385.97
% 390.38
% 394.36
% 394.07
% 389.44
% 387.44
% 390.53
% 399.07
% 396.44
% 395.4
% 391.93
% 385.07
% 381.63
% 373.24
% 383.13
% 389.64
% 391.96
% 392.85
% 396.31
% 397.27
% 401.13
% 397.97
% 397.27
% 395.41
% 395.66
% 397.68
% 398.96
% 391.37
% 393.95
% 396.07
% 398.15
% 400.53
% 398.61
% 401.21
% 396.72
% 394.19
% 395.51
% 395.4
% 390.39
% 386.22
% 390.85
% 387.52
% 388.97
% 394.66
% 393
% 387.45
% 383.81
% 381.01
% 378.21
% 380.39
% 376.08
% 371.3
% 362.39
% 360.83
% 348.58
% 357.98
% 361.6
% 366.34
% 362.92
% 363.48
% 368.65
% 371.57
% 368.61
% 362.98
% 360.43
% 367.3
% 369.55
% 368.55
% 368.29
% 363.51
% 364.44
% 355.04
% 358.44
% 361.54
% 360.59
% 367.06
% 361.69
% 353.03
% 349.38
% 358.01
% 355.31
% 354.61
% 351.03
% 356.09
% 361.46
% 362.33
% 358.77
% 361.66
% 364.84
% 363.38
% 362.06
% 362.06
% 363.77
% 364.5
% 365.61
% 363.19
% 364.94
% 364.74
% 361.01
% 355.73
% 356.01
% 355.78
% 352.32
% 359
% 357.38
% 353.84
% 353.63
% 357.36
% 357.52
% 359.02
% 355.43
% 348.98
% 350.67
% 346.21
% 350.16
% 355.08
% 354.02
% 353.32
% 353.85
% 349
% 345.41
% 343.67
% 343.44
% 337.94
% 345.5
% 343.94
% 347.53
% 348.34
% 343.24
% 349.29
% 350.34
% 344.08
% 345.6
% 344.26
% 348.04
% 352.49
% 354.54
% 350.51
% 352.01
% 345.67
% 346.22
% 346.62
% 341.89
% 329.88
% 328.23
% 322.21
% 322.25
% 320.09
% 314.44
% 316.38
% 308.08
% 305.45
% 312.07
% 316.2
% 319.11
% 312.86
% 307.2
% 304.02
% 304.01
% 309.03
% 302.28
% 308.31
% 310.15
% 308.29
% 305.03
% 306.59
% 305.35
% 307.91
% 314.87
% 313.9
% 316.02
% 319.68
% 317.33
% 318.34
% 319.61
% 319.74
% 321.62
% 323.31
% 322.97
% 321.64
% 323.8
% 325.47
% 327.5
% 327.23
% 328.11
% 324.35
% 329.92
% 330.67
% 331.78
% 329.91
% 328.8
% 326.79
% 322.77
% 318.69
% 319.86
% 323.63
% 323.91
% 316.35
% 319.49
% 322.96
% 315.02
% 312.92
% 312.47
% 313.01
% 308.16
% 309.12
% 318.56
% 312.74
% 310.63
% 308.69
% 306.49
% 307.54
% 304.67
% 301.65
% 297.74
% 289.77
% 290.2
% 283.19
% 282.65
% 281.36
% 288.27
% 289.05
% 291.26
% 290.41
% 284.96
% 287.09
% 282.57
% 289.36
% 289.57
% 293.59
% 291.29
% 288.61
% 288.57
% 291.47
% ];
% dummysize = length(dummyfixing);
% 
%     for idxhong=1:10000
%         
%         tempDate = AddDate(tempDate,1,'day');
%         if idxhong<= dummysize
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(idxhong);
%         else
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(end);
%         end
%     end
    
    
     for idxhong=1:10000
        
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
        
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;
% numOfRandom = 100;


numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);

if idx2 == 51
   abcd = 1.0; 
end

% pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);
pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC2(valueDateH,volTargetOptionParams,numMethodMC);


callablePrice(idx2) = pricingOutMC.priceOption.npv;

% alphaTrace{idx2,1} = pricingOutMC.alphaTrace;
% volIdxTrace{idx2,1} = pricingOutMC.volIdxTrace;

aaa = 1.0;
 
end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_part05.m

clc
clear

% valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2020-12-18'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
% volTargetInfo.multiplier = 1.0;

volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;

% volTargetInfo.volPeriodParticipationRate = 0.5;
% volTargetInfo.volPeriodParticipationRate = 1.0;
volTargetInfo.volPeriodParticipationRate = 0.5;


volTargetInfo.leverageCap = 1.0;


% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end
    

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

    
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));

out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);


pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);

callablePrice(idx2) = pricingOutMC.priceOption.npv;
aaa = 1.0;
 

end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_part1.m

clc
clear

% valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2020-12-18'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
% volTargetInfo.multiplier = 0.7;
volTargetInfo.multiplier = 1.0;

volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;

% volTargetInfo.volPeriodParticipationRate = 0.5;
% volTargetInfo.volPeriodParticipationRate = 1.0;
volTargetInfo.volPeriodParticipationRate = 1.0;


volTargetInfo.leverageCap = 1.0;


% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end
    

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

    
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));

out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);


pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);

callablePrice(idx2) = pricingOutMC.priceOption.npv;
aaa = 1.0;
 

end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_fixedAlpha.m

clc
clear

% valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2020-12-18'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
% volTargetInfo.multiplier = 0.7;
volTargetInfo.multiplier = 1.0;

volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;
% volTargetInfo.volPeriodParticipationRate = 0.5;
volTargetInfo.volPeriodParticipationRate = 1.0;

volTargetInfo.leverageCap = 1.0;


% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';
volTargetInfo.firstSchedule_eventType = 'END';

% volTargetInfo.lastSchedule_eventType = 'END';
volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
% volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
% 20221017	20221017	20221018	20221018
% 20221018	20221018	20221019	20221019
% 20221019	20221019	20221020	20221020
% 20221020	20221020	20221021	20221021
% 20221021	20221021	20221024	20221024
% 20221024	20221024	20221025	20221025
% 20221025	20221025	20221026	20221026
% 20221026	20221026	20221027	20221027
% 20221027	20221027	20221028	20221028
% 20221028	20221028	20221031	20221031
% 20221031	20221031	20221101	20221101
% 20221101	20221101	20221102	20221102
% 20221102	20221102	20221103	20221103
% 20221103	20221103	20221104	20221104
% 20221104	20221104	20221107	20221107
% 20221107	20221107	20221108	20221108
% 20221108	20221108	20221109	20221109
% 20221109	20221109	20221110	20221110
% 20221110	20221110	20221111	20221111
% 20221111	20221111	20221114	20221114
% 20221114	20221114	20221115	20221115
% 20221115	20221115	20221116	20221116
% 20221116	20221116	20221117	20221117
% 20221117	20221117	20221118	20221118
% 20221118	20221118	20221121	20221121
% 20221121	20221121	20221122	20221122
% 20221122	20221122	20221123	20221123
% 20221123	20221123	20221124	20221124
% 20221124	20221124	20221125	20221125
% ];

volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end

rawDataSim = rawData;

% for idxhh=2:size(rawData,1)
%     for idxjj=2:size(rawData,2)
%         rawDataSim(idxhh,idxjj) = rawDataSim(idxhh,idxjj) + 0.01; 
%     end
% end

rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

    
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);


pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);

callablePrice(idx2) = pricingOutMC.priceOption.npv;
aaa = 1.0;
 

end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_deltaGamma_theta.m

clc
clear

% valueDate = '2020-12-18';

    
alphaTrace = cell(100,1);
volIdxTrace = cell(100,1);

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;

idxMax = 366;
for idx2= 1:idxMax
%     
tempDateHong = AddDate(H_Date('2022-10-17'),idx2-1,'day');
valueDateH = tempDateHong;
valueDate = StrDate(valueDateH,'str');

% valueDate = '2022-10-14';
% valueDate = '2022-10-17';
% 
% valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
% volTargetInfo.multiplier = 1.0;

volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;
% volTargetInfo.volPeriodParticipationRate = 0.5;
volTargetInfo.volPeriodParticipationRate = 0.5;

% volTargetInfo.volPeriodParticipationRate = 1.0;


volTargetInfo.leverageCap = 1.0;
% volTargetInfo.leverageCap = 0.5;



% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];

% volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
% 20221017	20221017	20221018	20221018
% 20221018	20221018	20221019	20221019
% 20221019	20221019	20221020	20221020
% 20221020	20221020	20221021	20221021
% 20221021	20221021	20221024	20221024
% 20221024	20221024	20221025	20221025
% 20221025	20221025	20221026	20221026
% 20221026	20221026	20221027	20221027
% 20221027	20221027	20221028	20221028
% 20221028	20221028	20221031	20221031
% 20221031	20221031	20221101	20221101
% 20221101	20221101	20221102	20221102
% 20221102	20221102	20221103	20221103
% 20221103	20221103	20221104	20221104
% 20221104	20221104	20221107	20221107
% 20221107	20221107	20221108	20221108
% 20221108	20221108	20221109	20221109
% 20221109	20221109	20221110	20221110
% 20221110	20221110	20221111	20221111
% 20221111	20221111	20221114	20221114
% 20221114	20221114	20221115	20221115
% 20221115	20221115	20221116	20221116
% 20221116	20221116	20221117	20221117
% 20221117	20221117	20221118	20221118
% 20221118	20221118	20221121	20221121
% 20221121	20221121	20221122	20221122
% 20221122	20221122	20221123	20221123
% 20221123	20221123	20221124	20221124
% 20221124	20221124	20221125	20221125
% 20221125	20221125	20221128	20221128
% 20221128	20221128	20221129	20221129
% 20221129	20221129	20221130	20221130
% 20221130	20221130	20221201	20221201
% 20221201	20221201	20221202	20221202
% 20221202	20221202	20221205	20221205
% 20221205	20221205	20221206	20221206
% 20221206	20221206	20221207	20221207
% 20221207	20221207	20221208	20221208
% 20221208	20221208	20221209	20221209
% 20221209	20221209	20221212	20221212
% 20221212	20221212	20221213	20221213
% 20221213	20221213	20221214	20221214
% 20221214	20221214	20221215	20221215
% 20221215	20221215	20221216	20221216
% 20221216	20221216	20221219	20221219
% 20221219	20221219	20221220	20221220
% 20221220	20221220	20221221	20221221
% 20221221	20221221	20221222	20221222
% 20221222	20221222	20221223	20221223
% 20221223	20221223	20221226	20221226
% 20221226	20221226	20221227	20221227
% 20221227	20221227	20221228	20221228
% 20221228	20221228	20221229	20221229
% 20221229	20221229	20221230	20221230
% 20221230	20221230	20230102	20230102
% 20230102	20230102	20230103	20230103
% 20230103	20230103	20230104	20230104
% 20230104	20230104	20230105	20230105
% 20230105	20230105	20230106	20230106
% 20230106	20230106	20230109	20230109
% 20230109	20230109	20230110	20230110
% 20230110	20230110	20230111	20230111
% 20230111	20230111	20230112	20230112
% 20230112	20230112	20230113	20230113
% 20230113	20230113	20230116	20230116
% 20230116	20230116	20230117	20230117
% 20230117	20230117	20230118	20230118
% 20230118	20230118	20230119	20230119
% 20230119	20230119	20230120	20230120
% 20230120	20230120	20230125	20230125
% 20230125	20230125	20230126	20230126
% 20230126	20230126	20230127	20230127
% 20230127	20230127	20230130	20230130
% 20230130	20230130	20230131	20230131
% 20230131	20230131	20230201	20230201
% 20230201	20230201	20230202	20230202
% 20230202	20230202	20230203	20230203
% 20230203	20230203	20230206	20230206
% 20230206	20230206	20230207	20230207
% 20230207	20230207	20230208	20230208
% 20230208	20230208	20230209	20230209
% 20230209	20230209	20230210	20230210
% 20230210	20230210	20230213	20230213
% 20230213	20230213	20230214	20230214
% 20230214	20230214	20230215	20230215
% 20230215	20230215	20230216	20230216
% 20230216	20230216	20230217	20230217
% 20230217	20230217	20230220	20230220
% 20230220	20230220	20230221	20230221
% 20230221	20230221	20230222	20230222
% 20230222	20230222	20230223	20230223
% 20230223	20230223	20230224	20230224
% 20230224	20230224	20230227	20230227
% 20230227	20230227	20230228	20230228
% 20230228	20230228	20230302	20230302
% 20230302	20230302	20230303	20230303
% 20230303	20230303	20230306	20230306
% 20230306	20230306	20230307	20230307
% 20230307	20230307	20230308	20230308
% 20230308	20230308	20230309	20230309
% 20230309	20230309	20230310	20230310
% 20230310	20230310	20230313	20230313
% 20230313	20230313	20230314	20230314
% 20230314	20230314	20230315	20230315
% 20230315	20230315	20230316	20230316
% 20230316	20230316	20230317	20230317
% 20230317	20230317	20230320	20230320
% 20230320	20230320	20230321	20230321
% 20230321	20230321	20230322	20230322
% 20230322	20230322	20230323	20230323
% 20230323	20230323	20230324	20230324
% 20230324	20230324	20230327	20230327
% 20230327	20230327	20230328	20230328
% 20230328	20230328	20230329	20230329
% 20230329	20230329	20230330	20230330
% 20230330	20230330	20230331	20230331
% 20230331	20230331	20230403	20230403
% 20230403	20230403	20230404	20230404
% 20230404	20230404	20230405	20230405
% 20230405	20230405	20230406	20230406
% 20230406	20230406	20230407	20230407
% 20230407	20230407	20230410	20230410
% 20230410	20230410	20230411	20230411
% 20230411	20230411	20230412	20230412
% 20230412	20230412	20230413	20230413
% 20230413	20230413	20230414	20230414
% 20230414	20230414	20230417	20230417
% 20230417	20230417	20230418	20230418
% 20230418	20230418	20230419	20230419
% 20230419	20230419	20230420	20230420
% 20230420	20230420	20230421	20230421
% 20230421	20230421	20230424	20230424
% 20230424	20230424	20230425	20230425
% 20230425	20230425	20230426	20230426
% 20230426	20230426	20230427	20230427
% 20230427	20230427	20230428	20230428
% 20230428	20230428	20230502	20230502
% 20230502	20230502	20230503	20230503
% 20230503	20230503	20230504	20230504
% 20230504	20230504	20230508	20230508
% 20230508	20230508	20230509	20230509
% 20230509	20230509	20230510	20230510
% 20230510	20230510	20230511	20230511
% 20230511	20230511	20230512	20230512
% 20230512	20230512	20230515	20230515
% 20230515	20230515	20230516	20230516
% 20230516	20230516	20230517	20230517
% 20230517	20230517	20230518	20230518
% 20230518	20230518	20230519	20230519
% 20230519	20230519	20230522	20230522
% 20230522	20230522	20230523	20230523
% 20230523	20230523	20230524	20230524
% 20230524	20230524	20230525	20230525
% 20230525	20230525	20230526	20230526
% 20230526	20230526	20230529	20230529
% 20230529	20230529	20230530	20230530
% 20230530	20230530	20230531	20230531
% 20230531	20230531	20230601	20230601
% 20230601	20230601	20230602	20230602
% 20230602	20230602	20230605	20230605
% 20230605	20230605	20230607	20230607
% 20230607	20230607	20230608	20230608
% 20230608	20230608	20230609	20230609
% 20230609	20230609	20230612	20230612
% 20230612	20230612	20230613	20230613
% 20230613	20230613	20230614	20230614
% 20230614	20230614	20230615	20230615
% 20230615	20230615	20230616	20230616
% 20230616	20230616	20230619	20230619
% 20230619	20230619	20230620	20230620
% 20230620	20230620	20230621	20230621
% 20230621	20230621	20230622	20230622
% 20230622	20230622	20230623	20230623
% 20230623	20230623	20230626	20230626
% 20230626	20230626	20230627	20230627
% 20230627	20230627	20230628	20230628
% 20230628	20230628	20230629	20230629
% 20230629	20230629	20230630	20230630
% 20230630	20230630	20230703	20230703
% 20230703	20230703	20230704	20230704
% 20230704	20230704	20230705	20230705
% 20230705	20230705	20230706	20230706
% 20230706	20230706	20230707	20230707
% 20230707	20230707	20230710	20230710
% 20230710	20230710	20230711	20230711
% 20230711	20230711	20230712	20230712
% 20230712	20230712	20230713	20230713
% 20230713	20230713	20230714	20230714
% 20230714	20230714	20230717	20230717
% 20230717	20230717	20230718	20230718
% 20230718	20230718	20230719	20230719
% 20230719	20230719	20230720	20230720
% 20230720	20230720	20230721	20230721
% 20230721	20230721	20230724	20230724
% 20230724	20230724	20230725	20230725
% 20230725	20230725	20230726	20230726
% 20230726	20230726	20230727	20230727
% 20230727	20230727	20230728	20230728
% 20230728	20230728	20230731	20230731
% 20230731	20230731	20230801	20230801
% 20230801	20230801	20230802	20230802
% 20230802	20230802	20230803	20230803
% 20230803	20230803	20230804	20230804
% 20230804	20230804	20230807	20230807
% 20230807	20230807	20230808	20230808
% 20230808	20230808	20230809	20230809
% 20230809	20230809	20230810	20230810
% 20230810	20230810	20230811	20230811
% 20230811	20230811	20230814	20230814
% 20230814	20230814	20230816	20230816
% 20230816	20230816	20230817	20230817
% 20230817	20230817	20230818	20230818
% 20230818	20230818	20230821	20230821
% 20230821	20230821	20230822	20230822
% 20230822	20230822	20230823	20230823
% 20230823	20230823	20230824	20230824
% 20230824	20230824	20230825	20230825
% 20230825	20230825	20230828	20230828
% 20230828	20230828	20230829	20230829
% 20230829	20230829	20230830	20230830
% 20230830	20230830	20230831	20230831
% 20230831	20230831	20230901	20230901
% 20230901	20230901	20230904	20230904
% 20230904	20230904	20230905	20230905
% 20230905	20230905	20230906	20230906
% 20230906	20230906	20230907	20230907
% 20230907	20230907	20230908	20230908
% 20230908	20230908	20230911	20230911
% 20230911	20230911	20230912	20230912
% 20230912	20230912	20230913	20230913
% 20230913	20230913	20230914	20230914
% 20230914	20230914	20230915	20230915
% 20230915	20230915	20230918	20230918
% 20230918	20230918	20230919	20230919
% 20230919	20230919	20230920	20230920
% 20230920	20230920	20230921	20230921
% 20230921	20230921	20230922	20230922
% 20230922	20230922	20230925	20230925
% 20230925	20230925	20230926	20230926
% 20230926	20230926	20230927	20230927
% 20230927	20230927	20231002	20231002
% 20231002	20231002	20231004	20231004
% 20231004	20231004	20231005	20231005
% 20231005	20231005	20231006	20231006
% 20231006	20231006	20231010	20231010
% 20231010	20231010	20231011	20231011
% 20231011	20231011	20231012	20231012
% 20231012	20231012	20231013	20231013
% 20231013	20231013	20231016	20231016
% 20231016	20231016	20231017	20231017
% ];

% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

% rawDataSim = rawData;
% rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
% rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
% rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
% 20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
% 20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
% 20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
% 20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
% 20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
% 20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
% 20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
% 20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
% 20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
% ];
% 
% for idxhong=2:size(rawData,1)
%     rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
% end


rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
52	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
87	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
143	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
234	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
332	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
423	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
605	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
787	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
969	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
1151	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];


rawDataSim = rawData;

% for idxhh=2:size(rawData,1)
%     for idxjj=2:size(rawData,2)
%         rawDataSim(idxhh,idxjj) = rawDataSim(idxhh,idxjj) + 0.01; 
%     end
% end

rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

%     dummyfixing  = [389.59
% 393.19
% 392.02
% 395.41
% 393.74
% 393.18
% 393.34
% 395.01
% 399.07
% 395.48
% 393.89
% 388.47
% 389.81
% 394.82
% 389.48
% 391.54
% 389.86
% 389.59
% 390.36
% 386.24
% 385.97
% 390.38
% 394.36
% 394.07
% 389.44
% 387.44
% 390.53
% 399.07
% 396.44
% 395.4
% 391.93
% 385.07
% 381.63
% 373.24
% 383.13
% 389.64
% 391.96
% 392.85
% 396.31
% 397.27
% 401.13
% 397.97
% 397.27
% 395.41
% 395.66
% 397.68
% 398.96
% 391.37
% 393.95
% 396.07
% 398.15
% 400.53
% 398.61
% 401.21
% 396.72
% 394.19
% 395.51
% 395.4
% 390.39
% 386.22
% 390.85
% 387.52
% 388.97
% 394.66
% 393
% 387.45
% 383.81
% 381.01
% 378.21
% 380.39
% 376.08
% 371.3
% 362.39
% 360.83
% 348.58
% 357.98
% 361.6
% 366.34
% 362.92
% 363.48
% 368.65
% 371.57
% 368.61
% 362.98
% 360.43
% 367.3
% 369.55
% 368.55
% 368.29
% 363.51
% 364.44
% 355.04
% 358.44
% 361.54
% 360.59
% 367.06
% 361.69
% 353.03
% 349.38
% 358.01
% 355.31
% 354.61
% 351.03
% 356.09
% 361.46
% 362.33
% 358.77
% 361.66
% 364.84
% 363.38
% 362.06
% 362.06
% 363.77
% 364.5
% 365.61
% 363.19
% 364.94
% 364.74
% 361.01
% 355.73
% 356.01
% 355.78
% 352.32
% 359
% 357.38
% 353.84
% 353.63
% 357.36
% 357.52
% 359.02
% 355.43
% 348.98
% 350.67
% 346.21
% 350.16
% 355.08
% 354.02
% 353.32
% 353.85
% 349
% 345.41
% 343.67
% 343.44
% 337.94
% 345.5
% 343.94
% 347.53
% 348.34
% 343.24
% 349.29
% 350.34
% 344.08
% 345.6
% 344.26
% 348.04
% 352.49
% 354.54
% 350.51
% 352.01
% 345.67
% 346.22
% 346.62
% 341.89
% 329.88
% 328.23
% 322.21
% 322.25
% 320.09
% 314.44
% 316.38
% 308.08
% 305.45
% 312.07
% 316.2
% 319.11
% 312.86
% 307.2
% 304.02
% 304.01
% 309.03
% 302.28
% 308.31
% 310.15
% 308.29
% 305.03
% 306.59
% 305.35
% 307.91
% 314.87
% 313.9
% 316.02
% 319.68
% 317.33
% 318.34
% 319.61
% 319.74
% 321.62
% 323.31
% 322.97
% 321.64
% 323.8
% 325.47
% 327.5
% 327.23
% 328.11
% 324.35
% 329.92
% 330.67
% 331.78
% 329.91
% 328.8
% 326.79
% 322.77
% 318.69
% 319.86
% 323.63
% 323.91
% 316.35
% 319.49
% 322.96
% 315.02
% 312.92
% 312.47
% 313.01
% 308.16
% 309.12
% 318.56
% 312.74
% 310.63
% 308.69
% 306.49
% 307.54
% 304.67
% 301.65
% 297.74
% 289.77
% 290.2
% 283.19
% 282.65
% 281.36
% 288.27
% 289.05
% 291.26
% 290.41
% 284.96
% 287.09
% 282.57
% 289.36
% 289.57
% 293.59
% 291.29
% 288.61
% 288.57
% 291.47
% ];
% dummysize = length(dummyfixing);
% 
%     for idxhong=1:10000
%         
%         tempDate = AddDate(tempDate,1,'day');
%         if idxhong<= dummysize
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(idxhong);
%         else
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(end);
%         end
%     end
    
    
     for idxhong=1:10000
        
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
        
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;
% numOfRandom = 100;


numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);

if idx2 == 51
   abcd = 1.0; 
end

% pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);
pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC2(valueDateH,volTargetOptionParams,numMethodMC);


callablePrice(idx2) = pricingOutMC.priceOption.npv;

% alphaTrace{idx2,1} = pricingOutMC.alphaTrace;
% volIdxTrace{idx2,1} = pricingOutMC.volIdxTrace;

aaa = 1.0;
 
end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_deltaGamma_fixed.m

clc
clear

% valueDate = '2020-12-18';

    
alphaTrace = cell(100,1);
volIdxTrace = cell(100,1);

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;

% idxMax = 366;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2022-10-17'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = StrDate(valueDateH,'str');

% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
% volTargetInfo.multiplier = 0.7;
volTargetInfo.multiplier = 1.0;

volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;
% volTargetInfo.volPeriodParticipationRate = 0.5;
% volTargetInfo.volPeriodParticipationRate = 0.5;

volTargetInfo.volPeriodParticipationRate = 1.0;


volTargetInfo.leverageCap = 1.0;
% volTargetInfo.leverageCap = 0.5;



% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

% volTargetInfo.lastSchedule_eventType = 'END';
volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
% volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
% 20221017	20221017	20221018	20221018
% 20221018	20221018	20221019	20221019
% 20221019	20221019	20221020	20221020
% 20221020	20221020	20221021	20221021
% 20221021	20221021	20221024	20221024
% 20221024	20221024	20221025	20221025
% 20221025	20221025	20221026	20221026
% 20221026	20221026	20221027	20221027
% 20221027	20221027	20221028	20221028
% 20221028	20221028	20221031	20221031
% 20221031	20221031	20221101	20221101
% 20221101	20221101	20221102	20221102
% 20221102	20221102	20221103	20221103
% 20221103	20221103	20221104	20221104
% 20221104	20221104	20221107	20221107
% 20221107	20221107	20221108	20221108
% 20221108	20221108	20221109	20221109
% 20221109	20221109	20221110	20221110
% 20221110	20221110	20221111	20221111
% 20221111	20221111	20221114	20221114
% 20221114	20221114	20221115	20221115
% 20221115	20221115	20221116	20221116
% 20221116	20221116	20221117	20221117
% 20221117	20221117	20221118	20221118
% 20221118	20221118	20221121	20221121
% 20221121	20221121	20221122	20221122
% 20221122	20221122	20221123	20221123
% 20221123	20221123	20221124	20221124
% 20221124	20221124	20221125	20221125
% ];

volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
% rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
% 20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
% 20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
% 20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
% 20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
% 20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
% 20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
% 20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
% 20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
% 20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
% ];
% 
% for idxhong=2:size(rawData,1)
%     rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
% end


rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
52	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
87	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
143	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
234	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
332	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
423	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
605	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
787	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
969	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
1151	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];


rawDataSim = rawData;

for idxhh=2:size(rawData,1)
    for idxjj=2:size(rawData,2)
        rawDataSim(idxhh,idxjj) = rawDataSim(idxhh,idxjj) + 0.01; 
    end
end

rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

%     dummyfixing  = [389.59
% 393.19
% 392.02
% 395.41
% 393.74
% 393.18
% 393.34
% 395.01
% 399.07
% 395.48
% 393.89
% 388.47
% 389.81
% 394.82
% 389.48
% 391.54
% 389.86
% 389.59
% 390.36
% 386.24
% 385.97
% 390.38
% 394.36
% 394.07
% 389.44
% 387.44
% 390.53
% 399.07
% 396.44
% 395.4
% 391.93
% 385.07
% 381.63
% 373.24
% 383.13
% 389.64
% 391.96
% 392.85
% 396.31
% 397.27
% 401.13
% 397.97
% 397.27
% 395.41
% 395.66
% 397.68
% 398.96
% 391.37
% 393.95
% 396.07
% 398.15
% 400.53
% 398.61
% 401.21
% 396.72
% 394.19
% 395.51
% 395.4
% 390.39
% 386.22
% 390.85
% 387.52
% 388.97
% 394.66
% 393
% 387.45
% 383.81
% 381.01
% 378.21
% 380.39
% 376.08
% 371.3
% 362.39
% 360.83
% 348.58
% 357.98
% 361.6
% 366.34
% 362.92
% 363.48
% 368.65
% 371.57
% 368.61
% 362.98
% 360.43
% 367.3
% 369.55
% 368.55
% 368.29
% 363.51
% 364.44
% 355.04
% 358.44
% 361.54
% 360.59
% 367.06
% 361.69
% 353.03
% 349.38
% 358.01
% 355.31
% 354.61
% 351.03
% 356.09
% 361.46
% 362.33
% 358.77
% 361.66
% 364.84
% 363.38
% 362.06
% 362.06
% 363.77
% 364.5
% 365.61
% 363.19
% 364.94
% 364.74
% 361.01
% 355.73
% 356.01
% 355.78
% 352.32
% 359
% 357.38
% 353.84
% 353.63
% 357.36
% 357.52
% 359.02
% 355.43
% 348.98
% 350.67
% 346.21
% 350.16
% 355.08
% 354.02
% 353.32
% 353.85
% 349
% 345.41
% 343.67
% 343.44
% 337.94
% 345.5
% 343.94
% 347.53
% 348.34
% 343.24
% 349.29
% 350.34
% 344.08
% 345.6
% 344.26
% 348.04
% 352.49
% 354.54
% 350.51
% 352.01
% 345.67
% 346.22
% 346.62
% 341.89
% 329.88
% 328.23
% 322.21
% 322.25
% 320.09
% 314.44
% 316.38
% 308.08
% 305.45
% 312.07
% 316.2
% 319.11
% 312.86
% 307.2
% 304.02
% 304.01
% 309.03
% 302.28
% 308.31
% 310.15
% 308.29
% 305.03
% 306.59
% 305.35
% 307.91
% 314.87
% 313.9
% 316.02
% 319.68
% 317.33
% 318.34
% 319.61
% 319.74
% 321.62
% 323.31
% 322.97
% 321.64
% 323.8
% 325.47
% 327.5
% 327.23
% 328.11
% 324.35
% 329.92
% 330.67
% 331.78
% 329.91
% 328.8
% 326.79
% 322.77
% 318.69
% 319.86
% 323.63
% 323.91
% 316.35
% 319.49
% 322.96
% 315.02
% 312.92
% 312.47
% 313.01
% 308.16
% 309.12
% 318.56
% 312.74
% 310.63
% 308.69
% 306.49
% 307.54
% 304.67
% 301.65
% 297.74
% 289.77
% 290.2
% 283.19
% 282.65
% 281.36
% 288.27
% 289.05
% 291.26
% 290.41
% 284.96
% 287.09
% 282.57
% 289.36
% 289.57
% 293.59
% 291.29
% 288.61
% 288.57
% 291.47
% ];
% dummysize = length(dummyfixing);
% 
%     for idxhong=1:10000
%         
%         tempDate = AddDate(tempDate,1,'day');
%         if idxhong<= dummysize
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(idxhong);
%         else
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(end);
%         end
%     end
    
    
     for idxhong=1:10000
        
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
        
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;
% numOfRandom = 100;


numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);

if idx2 == 51
   abcd = 1.0; 
end

% pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);
pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC2(valueDateH,volTargetOptionParams,numMethodMC);


callablePrice(idx2) = pricingOutMC.priceOption.npv;

% alphaTrace{idx2,1} = pricingOutMC.alphaTrace;
% volIdxTrace{idx2,1} = pricingOutMC.volIdxTrace;

aaa = 1.0;
 
end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul_deltaGamma.m


clc
clear

% valueDate = '2020-12-18';

    
alphaTrace = cell(100,1);
volIdxTrace = cell(100,1);

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;

% idxMax = 366;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2022-10-17'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = StrDate(valueDateH,'str');

% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;
% volTargetInfo.volPeriodParticipationRate = 0.5;
volTargetInfo.volPeriodParticipationRate = 0.5;
% volTargetInfo.volPeriodParticipationRate = 1.0;


volTargetInfo.leverageCap = 1.0;
% volTargetInfo.leverageCap = 0.5;



% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
% rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
% 20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
% 20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
% 20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
% 20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
% 20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
% 20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
% 20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
% 20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
% 20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
% ];
% 
% for idxhong=2:size(rawData,1)
%     rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
% end


rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
52	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
87	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
143	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
234	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
332	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
423	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
605	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
787	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
969	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
1151	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];


rawDataSim = rawData;

for idxhh=2:size(rawData,1)
    for idxjj=2:size(rawData,2)
        rawDataSim(idxhh,idxjj) = rawDataSim(idxhh,idxjj) + 0.01; 
    end
end

rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

%     dummyfixing  = [389.59
% 393.19
% 392.02
% 395.41
% 393.74
% 393.18
% 393.34
% 395.01
% 399.07
% 395.48
% 393.89
% 388.47
% 389.81
% 394.82
% 389.48
% 391.54
% 389.86
% 389.59
% 390.36
% 386.24
% 385.97
% 390.38
% 394.36
% 394.07
% 389.44
% 387.44
% 390.53
% 399.07
% 396.44
% 395.4
% 391.93
% 385.07
% 381.63
% 373.24
% 383.13
% 389.64
% 391.96
% 392.85
% 396.31
% 397.27
% 401.13
% 397.97
% 397.27
% 395.41
% 395.66
% 397.68
% 398.96
% 391.37
% 393.95
% 396.07
% 398.15
% 400.53
% 398.61
% 401.21
% 396.72
% 394.19
% 395.51
% 395.4
% 390.39
% 386.22
% 390.85
% 387.52
% 388.97
% 394.66
% 393
% 387.45
% 383.81
% 381.01
% 378.21
% 380.39
% 376.08
% 371.3
% 362.39
% 360.83
% 348.58
% 357.98
% 361.6
% 366.34
% 362.92
% 363.48
% 368.65
% 371.57
% 368.61
% 362.98
% 360.43
% 367.3
% 369.55
% 368.55
% 368.29
% 363.51
% 364.44
% 355.04
% 358.44
% 361.54
% 360.59
% 367.06
% 361.69
% 353.03
% 349.38
% 358.01
% 355.31
% 354.61
% 351.03
% 356.09
% 361.46
% 362.33
% 358.77
% 361.66
% 364.84
% 363.38
% 362.06
% 362.06
% 363.77
% 364.5
% 365.61
% 363.19
% 364.94
% 364.74
% 361.01
% 355.73
% 356.01
% 355.78
% 352.32
% 359
% 357.38
% 353.84
% 353.63
% 357.36
% 357.52
% 359.02
% 355.43
% 348.98
% 350.67
% 346.21
% 350.16
% 355.08
% 354.02
% 353.32
% 353.85
% 349
% 345.41
% 343.67
% 343.44
% 337.94
% 345.5
% 343.94
% 347.53
% 348.34
% 343.24
% 349.29
% 350.34
% 344.08
% 345.6
% 344.26
% 348.04
% 352.49
% 354.54
% 350.51
% 352.01
% 345.67
% 346.22
% 346.62
% 341.89
% 329.88
% 328.23
% 322.21
% 322.25
% 320.09
% 314.44
% 316.38
% 308.08
% 305.45
% 312.07
% 316.2
% 319.11
% 312.86
% 307.2
% 304.02
% 304.01
% 309.03
% 302.28
% 308.31
% 310.15
% 308.29
% 305.03
% 306.59
% 305.35
% 307.91
% 314.87
% 313.9
% 316.02
% 319.68
% 317.33
% 318.34
% 319.61
% 319.74
% 321.62
% 323.31
% 322.97
% 321.64
% 323.8
% 325.47
% 327.5
% 327.23
% 328.11
% 324.35
% 329.92
% 330.67
% 331.78
% 329.91
% 328.8
% 326.79
% 322.77
% 318.69
% 319.86
% 323.63
% 323.91
% 316.35
% 319.49
% 322.96
% 315.02
% 312.92
% 312.47
% 313.01
% 308.16
% 309.12
% 318.56
% 312.74
% 310.63
% 308.69
% 306.49
% 307.54
% 304.67
% 301.65
% 297.74
% 289.77
% 290.2
% 283.19
% 282.65
% 281.36
% 288.27
% 289.05
% 291.26
% 290.41
% 284.96
% 287.09
% 282.57
% 289.36
% 289.57
% 293.59
% 291.29
% 288.61
% 288.57
% 291.47
% ];
% dummysize = length(dummyfixing);
% 
%     for idxhong=1:10000
%         
%         tempDate = AddDate(tempDate,1,'day');
%         if idxhong<= dummysize
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(idxhong);
%         else
%             equityFixing1(StrDate(tempDate,'str')) = dummyfixing(end);
%         end
%     end
    
    
     for idxhong=1:10000
        
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
        
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;
% numOfRandom = 100;


numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);

if idx2 == 51
   abcd = 1.0; 
end

% pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);
pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC2(valueDateH,volTargetOptionParams,numMethodMC);


callablePrice(idx2) = pricingOutMC.priceOption.npv;

% alphaTrace{idx2,1} = pricingOutMC.alphaTrace;
% volIdxTrace{idx2,1} = pricingOutMC.volIdxTrace;

aaa = 1.0;
 
end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021_simul.m

clc
clear

% valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2020-12-18'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
% volTargetInfo.multiplier = 0.7;
volTargetInfo.multiplier = 1.0;

volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;

% volTargetInfo.volPeriodParticipationRate = 0.5;
volTargetInfo.volPeriodParticipationRate = 1.0;

volTargetInfo.leverageCap = 1.0;


% volTargetInfo.eventType = 'RESET';
% volTargetInfo.eventType = 'END';

volTargetInfo.firstSchedule_eventType = 'END';

volTargetInfo.lastSchedule_eventType = 'END';
% volTargetInfo.lastSchedule_eventType = 'RESET';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
% rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
% 20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
% 20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
% 20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
% 20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
% 20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
% 20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
% 20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
% 20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
% 20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
% ];

% for idxhong=2:size(rawData,1)
%     rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
% end

rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
52	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
87	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
143	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
234	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
332	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
423	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
605	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
787	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
969	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
1151	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];


    

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

    
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));

out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);


pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);

callablePrice(idx2) = pricingOutMC.priceOption.npv;
aaa = 1.0;
 

end

aaa =1.0;


In [ ]:
test_VolTargetOption_kospi200_20221021.m

clc
clear

% valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% volTargetOption Pricing start
idxMax = 365 + 10;
maxIdx = 3;

% idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 21;
maxIdx = 10;
for idx2= 1:idxMax
%     
% tempDateHong = AddDate(H_Date('2020-12-18'),idx2-1,'day');
% valueDateH = tempDateHong;
% valueDate = '2022-10-14';
valueDate = '2022-10-17';

valueDateH = H_Date(valueDate);

%% security info start
% valueDateH = H_Date(valueDate);
volTargetOptionParams.payCcy = 'KRW';
volTargetOptionParams.startDate = H_Date('2022-10-14');
volTargetOptionParams.endDate = H_Date('2023-10-17');

nominal = 10000;
basePrice = 100.0;

%% volTarget Info Start

volTargetInfo = struct;
volTargetInfo.multiplier = 0.7;
volTargetInfo.volTarget = 0.1;
volTargetInfo.volPeriod = 30;
volTargetInfo.volPeriodParticipationRate = 0.5;
volTargetInfo.leverageCap = 1.0;


% volTargetInfo.eventType = 'RESET';
volTargetInfo.eventType = 'END';


% first 30 fixing schedule with fixed Participation  reset start end pay 
volTargetInfo.firstScheduleInt=[20221014	20221014	20221017	20221017
20221017	20221017	20221018	20221018
20221018	20221018	20221019	20221019
20221019	20221019	20221020	20221020
20221020	20221020	20221021	20221021
20221021	20221021	20221024	20221024
20221024	20221024	20221025	20221025
20221025	20221025	20221026	20221026
20221026	20221026	20221027	20221027
20221027	20221027	20221028	20221028
20221028	20221028	20221031	20221031
20221031	20221031	20221101	20221101
20221101	20221101	20221102	20221102
20221102	20221102	20221103	20221103
20221103	20221103	20221104	20221104
20221104	20221104	20221107	20221107
20221107	20221107	20221108	20221108
20221108	20221108	20221109	20221109
20221109	20221109	20221110	20221110
20221110	20221110	20221111	20221111
20221111	20221111	20221114	20221114
20221114	20221114	20221115	20221115
20221115	20221115	20221116	20221116
20221116	20221116	20221117	20221117
20221117	20221117	20221118	20221118
20221118	20221118	20221121	20221121
20221121	20221121	20221122	20221122
20221122	20221122	20221123	20221123
20221123	20221123	20221124	20221124
20221124	20221124	20221125	20221125
];


% next remaining fixing schedule with dynamic Participation  reset start end pay 
volTargetInfo.lastScheduleInt=[20221125	20221125	20221128	20221128
20221128	20221128	20221129	20221129
20221129	20221129	20221130	20221130
20221130	20221130	20221201	20221201
20221201	20221201	20221202	20221202
20221202	20221202	20221205	20221205
20221205	20221205	20221206	20221206
20221206	20221206	20221207	20221207
20221207	20221207	20221208	20221208
20221208	20221208	20221209	20221209
20221209	20221209	20221212	20221212
20221212	20221212	20221213	20221213
20221213	20221213	20221214	20221214
20221214	20221214	20221215	20221215
20221215	20221215	20221216	20221216
20221216	20221216	20221219	20221219
20221219	20221219	20221220	20221220
20221220	20221220	20221221	20221221
20221221	20221221	20221222	20221222
20221222	20221222	20221223	20221223
20221223	20221223	20221226	20221226
20221226	20221226	20221227	20221227
20221227	20221227	20221228	20221228
20221228	20221228	20221229	20221229
20221229	20221229	20221230	20221230
20221230	20221230	20230102	20230102
20230102	20230102	20230103	20230103
20230103	20230103	20230104	20230104
20230104	20230104	20230105	20230105
20230105	20230105	20230106	20230106
20230106	20230106	20230109	20230109
20230109	20230109	20230110	20230110
20230110	20230110	20230111	20230111
20230111	20230111	20230112	20230112
20230112	20230112	20230113	20230113
20230113	20230113	20230116	20230116
20230116	20230116	20230117	20230117
20230117	20230117	20230118	20230118
20230118	20230118	20230119	20230119
20230119	20230119	20230120	20230120
20230120	20230120	20230125	20230125
20230125	20230125	20230126	20230126
20230126	20230126	20230127	20230127
20230127	20230127	20230130	20230130
20230130	20230130	20230131	20230131
20230131	20230131	20230201	20230201
20230201	20230201	20230202	20230202
20230202	20230202	20230203	20230203
20230203	20230203	20230206	20230206
20230206	20230206	20230207	20230207
20230207	20230207	20230208	20230208
20230208	20230208	20230209	20230209
20230209	20230209	20230210	20230210
20230210	20230210	20230213	20230213
20230213	20230213	20230214	20230214
20230214	20230214	20230215	20230215
20230215	20230215	20230216	20230216
20230216	20230216	20230217	20230217
20230217	20230217	20230220	20230220
20230220	20230220	20230221	20230221
20230221	20230221	20230222	20230222
20230222	20230222	20230223	20230223
20230223	20230223	20230224	20230224
20230224	20230224	20230227	20230227
20230227	20230227	20230228	20230228
20230228	20230228	20230302	20230302
20230302	20230302	20230303	20230303
20230303	20230303	20230306	20230306
20230306	20230306	20230307	20230307
20230307	20230307	20230308	20230308
20230308	20230308	20230309	20230309
20230309	20230309	20230310	20230310
20230310	20230310	20230313	20230313
20230313	20230313	20230314	20230314
20230314	20230314	20230315	20230315
20230315	20230315	20230316	20230316
20230316	20230316	20230317	20230317
20230317	20230317	20230320	20230320
20230320	20230320	20230321	20230321
20230321	20230321	20230322	20230322
20230322	20230322	20230323	20230323
20230323	20230323	20230324	20230324
20230324	20230324	20230327	20230327
20230327	20230327	20230328	20230328
20230328	20230328	20230329	20230329
20230329	20230329	20230330	20230330
20230330	20230330	20230331	20230331
20230331	20230331	20230403	20230403
20230403	20230403	20230404	20230404
20230404	20230404	20230405	20230405
20230405	20230405	20230406	20230406
20230406	20230406	20230407	20230407
20230407	20230407	20230410	20230410
20230410	20230410	20230411	20230411
20230411	20230411	20230412	20230412
20230412	20230412	20230413	20230413
20230413	20230413	20230414	20230414
20230414	20230414	20230417	20230417
20230417	20230417	20230418	20230418
20230418	20230418	20230419	20230419
20230419	20230419	20230420	20230420
20230420	20230420	20230421	20230421
20230421	20230421	20230424	20230424
20230424	20230424	20230425	20230425
20230425	20230425	20230426	20230426
20230426	20230426	20230427	20230427
20230427	20230427	20230428	20230428
20230428	20230428	20230502	20230502
20230502	20230502	20230503	20230503
20230503	20230503	20230504	20230504
20230504	20230504	20230508	20230508
20230508	20230508	20230509	20230509
20230509	20230509	20230510	20230510
20230510	20230510	20230511	20230511
20230511	20230511	20230512	20230512
20230512	20230512	20230515	20230515
20230515	20230515	20230516	20230516
20230516	20230516	20230517	20230517
20230517	20230517	20230518	20230518
20230518	20230518	20230519	20230519
20230519	20230519	20230522	20230522
20230522	20230522	20230523	20230523
20230523	20230523	20230524	20230524
20230524	20230524	20230525	20230525
20230525	20230525	20230526	20230526
20230526	20230526	20230529	20230529
20230529	20230529	20230530	20230530
20230530	20230530	20230531	20230531
20230531	20230531	20230601	20230601
20230601	20230601	20230602	20230602
20230602	20230602	20230605	20230605
20230605	20230605	20230607	20230607
20230607	20230607	20230608	20230608
20230608	20230608	20230609	20230609
20230609	20230609	20230612	20230612
20230612	20230612	20230613	20230613
20230613	20230613	20230614	20230614
20230614	20230614	20230615	20230615
20230615	20230615	20230616	20230616
20230616	20230616	20230619	20230619
20230619	20230619	20230620	20230620
20230620	20230620	20230621	20230621
20230621	20230621	20230622	20230622
20230622	20230622	20230623	20230623
20230623	20230623	20230626	20230626
20230626	20230626	20230627	20230627
20230627	20230627	20230628	20230628
20230628	20230628	20230629	20230629
20230629	20230629	20230630	20230630
20230630	20230630	20230703	20230703
20230703	20230703	20230704	20230704
20230704	20230704	20230705	20230705
20230705	20230705	20230706	20230706
20230706	20230706	20230707	20230707
20230707	20230707	20230710	20230710
20230710	20230710	20230711	20230711
20230711	20230711	20230712	20230712
20230712	20230712	20230713	20230713
20230713	20230713	20230714	20230714
20230714	20230714	20230717	20230717
20230717	20230717	20230718	20230718
20230718	20230718	20230719	20230719
20230719	20230719	20230720	20230720
20230720	20230720	20230721	20230721
20230721	20230721	20230724	20230724
20230724	20230724	20230725	20230725
20230725	20230725	20230726	20230726
20230726	20230726	20230727	20230727
20230727	20230727	20230728	20230728
20230728	20230728	20230731	20230731
20230731	20230731	20230801	20230801
20230801	20230801	20230802	20230802
20230802	20230802	20230803	20230803
20230803	20230803	20230804	20230804
20230804	20230804	20230807	20230807
20230807	20230807	20230808	20230808
20230808	20230808	20230809	20230809
20230809	20230809	20230810	20230810
20230810	20230810	20230811	20230811
20230811	20230811	20230814	20230814
20230814	20230814	20230816	20230816
20230816	20230816	20230817	20230817
20230817	20230817	20230818	20230818
20230818	20230818	20230821	20230821
20230821	20230821	20230822	20230822
20230822	20230822	20230823	20230823
20230823	20230823	20230824	20230824
20230824	20230824	20230825	20230825
20230825	20230825	20230828	20230828
20230828	20230828	20230829	20230829
20230829	20230829	20230830	20230830
20230830	20230830	20230831	20230831
20230831	20230831	20230901	20230901
20230901	20230901	20230904	20230904
20230904	20230904	20230905	20230905
20230905	20230905	20230906	20230906
20230906	20230906	20230907	20230907
20230907	20230907	20230908	20230908
20230908	20230908	20230911	20230911
20230911	20230911	20230912	20230912
20230912	20230912	20230913	20230913
20230913	20230913	20230914	20230914
20230914	20230914	20230915	20230915
20230915	20230915	20230918	20230918
20230918	20230918	20230919	20230919
20230919	20230919	20230920	20230920
20230920	20230920	20230921	20230921
20230921	20230921	20230922	20230922
20230922	20230922	20230925	20230925
20230925	20230925	20230926	20230926
20230926	20230926	20230927	20230927
20230927	20230927	20231002	20231002
20231002	20231002	20231004	20231004
20231004	20231004	20231005	20231005
20231005	20231005	20231006	20231006
20231006	20231006	20231010	20231010
20231010	20231010	20231011	20231011
20231011	20231011	20231012	20231012
20231012	20231012	20231013	20231013
20231013	20231013	20231016	20231016
20231016	20231016	20231017	20231017
];

volTargetOptionInfo = struct('nominal',nominal,'basePrice',basePrice,'volTargetInfo',volTargetInfo);
                        
%% volTarget Info end

%EODFlag

% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = false;
% volTargetOptionInfo.volTargetInfo.EODFlag_Cashflow = true;


volTargetOptionInfo.volTargetInfo.EODFlag_Exercise = true;

volTargetOptionParams.params = volTargetOptionInfo;  


% rmseSimul = zeros(100,1);

    %% KRW Model building
        
  %% import and construct a zeroCurve


rawData = [0.002739726	0.031268661
0.25	0.029902964
0.5	0.032345864
0.75	0.033804006
1	0.035090319
1.5	0.038149151
2	0.041216226
2.5	0.041448706
3	0.041617666
3.5	0.04162547
4	0.041629661
4.5	0.041633853
5	0.041638045
5.5	0.041607359
6	0.041575691
6.5	0.041544023
7	0.041512355
7.5	0.041480687
8	0.041449019
8.5	0.041417351
9	0.041385683
9.5	0.041354015
10	0.041322347
10.5	0.041293445
11	0.041264654
11.5	0.041235862
12	0.041207071
12.5	0.041178279
13	0.041149487
13.5	0.041120696
14	0.041091904
14.5	0.041063113
15	0.041034321
15.5	0.041005529
16	0.040976738
16.5	0.040947946
17	0.040919154
17.5	0.040890363
18	0.040861571
18.5	0.04083278
19	0.040803988
19.5	0.040775196
20	0.040746405
20.5	0.040645134
21	0.04054054
21.5	0.040435946
22	0.040331351
22.5	0.040226757
23	0.040122163
23.5	0.040017569
24	0.039912975
24.5	0.039808381
25	0.039703787
25.5	0.039599193
26	0.039494599
26.5	0.039390005
27	0.039285411
27.5	0.039180817
28	0.039076223
28.5	0.038971629
29	0.038867035
29.5	0.038762441
30	0.038657847
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    AllMktData =containers.Map('zeroCurve',zeroCurve);
    
%% import and construct a spot For KOSPI200
rawData = 289.36;

rawDataSim = rawData;
rawDataSim = rawDataSim*(1.0 - maxIdx*0.01 + (idx2-1)*0.01);
rawData = rawDataSim;

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20221208	0.5969	0.5072	0.4268	0.3526	0.2826	0.2179	0.1751	0.1776	0.1992	0.2235	0.2471
20230112	0.5261	0.4503	0.3825	0.3202	0.2622	0.21	0.1753	0.1726	0.1869	0.2053	0.2239
20230309	0.4667	0.4043	0.3489	0.2983	0.2519	0.2105	0.1798	0.1687	0.1738	0.1853	0.1987
20230608	0.4132	0.363	0.3187	0.2788	0.2425	0.2104	0.1848	0.1697	0.1665	0.1708	0.1786
20230914	0.3793	0.3369	0.2996	0.2662	0.2362	0.2097	0.1878	0.1725	0.1654	0.1649	0.1686
20231214	0.3582	0.3206	0.2877	0.2584	0.2322	0.2091	0.1896	0.175	0.1663	0.1632	0.1641
20240613	0.3291	0.2979	0.2707	0.2466	0.2253	0.2065	0.1905	0.1777	0.1686	0.1632	0.1612
20241212	0.3123	0.2852	0.2617	0.2411	0.2228	0.2066	0.1927	0.1811	0.172	0.1656	0.1617
20250612	0.2989	0.2747	0.2539	0.2356	0.2194	0.2052	0.1928	0.1823	0.1738	0.1674	0.1629
20251211	0.2902	0.2682	0.2493	0.2328	0.2182	0.2054	0.1941	0.1845	0.1764	0.1699	0.165
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end
    

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0    0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

%% allFixings
    %% KOSPI200
    tempDate = volTargetOptionParams.startDate;

    % KOSPI200
    equityFixing1 = containers.Map({StrDate(tempDate,'str')},{289.36});

    
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        equityFixing1(StrDate(tempDate,'str')) = 289.36;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'equityIndex'},{equityFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic



out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

%% NumMethod Init Start
numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                        {'AMC', 1, numOfRandom});

numMethodMC = NumMethod(numMethodInfo);


pricingOutMC = KOSPI200Dupire.computeVolTargetOption1SMC(valueDateH,volTargetOptionParams,numMethodMC);

callablePrice(idx2) = pricingOutMC.priceOption.npv;
aaa = 1.0;
 
%     %% import and construct a discountCurve
% %         rawData = [0.00274	0.004799968
% % 0.25	0.004019196
% % 0.5	0.004772386
% % 0.75	0.005368417
% % 1	0.005819943
% % 2	0.00885494
% % 3	0.009626873
% % 5	0.013470451
% % 10	0.017206717
% % 20	0.018218138
% % 30	0.018261571
% %     ];
% 
% rawData = [0.00274	0.004799968
% 0.25	0.004019196
% 0.5	0.004772386
% 0.75	0.005368417
% 1	0.00581667
% 2	0.008853178
% 3	0.009625741
% 5	0.013468644
% 10	0.017205609
% 20	0.018217501
% 30	0.018261113
%     ];
% 
%     mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                             {rawData,valueDate,'KRW','discountZero'});
%     discountCurveMktData = MktData(mktData);
%     discountCurveMktData.params('convention') = {};
%     discountCurve =  ZeroCurve(discountCurveMktData);
% 
%     KRWAllMktData('discountCurve') = discountCurve;
% 
%     %% import and construct a swaptionVolSurfaceNormal
% 
% 
% 
% % % GIROPremiumToNormalVol From LIB
% % rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% % 1	0.0035	0.0038	0.0041	0.0044	0.0047	0.0049	0.0052	0.0053	0.0050	0.0044
% % 2	0.0042	0.0043	0.0044	0.0045	0.0046	0.0047	0.0049	0.0049	0.0046	0.0040
% % 3	0.0044	0.0044	0.0044	0.0044	0.0044	0.0046	0.0048	0.0049	0.0044	0.0039
% % 4	0.0045	0.0045	0.0045	0.0045	0.0044	0.0046	0.0048	0.0048	0.0042	0.0039
% % 5	0.0046	0.0046	0.0045	0.0045	0.0045	0.0046	0.0047	0.0047	0.0041	0.0039
% % 7	0.0046	0.0046	0.0046	0.0046	0.0045	0.0046	0.0048	0.0048	0.0044	0.0043
% % 10	0.0047	0.0047	0.0047	0.0047	0.0046	0.0047	0.0042	0.0043	0.0042	0.0043
% % 15	0.0032	0.0031	0.0029	0.0028	0.0026	0.0031	0.0033	0.0035	0.0037	0.0040
% % 20	0.0037	0.0037	0.0037	0.0037	0.0037	0.0038	0.0038	0.0039	0.0040	0.0041
% %     ];
% 
% % SwaptionVol From FrontSystem 
% % 20 x 20
% rawDataSwaption = [0	1	2	3	4	5	7		10	12	15	20
% 1	0.00313868	0.00345381	0.00376428	0.00408151	0.00440039	0.00454026		0.00475148	0.00489589	0.00489589	0.00489589
% 2	0.00372804	0.00387327	0.00400155	0.00414319	0.00428737	0.00444983		0.00469545	0.00476574	0.00476574	0.00476574
% 3	0.00390354	0.00401313	0.00409429	0.00418996	0.0042849	0.0044097		0.00459433	0.00464883	0.00464883	0.00464883
% 4	0.00408523	0.00411002	0.00416392	0.00421762	0.00426983	0.00436359		0.00449447	0.00453408	0.00453408	0.00453408
% 5	0.00418943	0.00421751	0.00423044	0.00424642	0.0042546	0.00430589		0.0043945	0.0044123	0.0044123	0.0044123
% 7	0.00421476	0.00425097	0.00425793	0.00428002	0.00430127	0.00436624		0.00446172	0.00448601	0.00448601	0.00448601
% 10	0.0042714	0.0042711	0.00431105	0.00434579	0.00438057	0.00447166		0.00461303	0.00463415	0.00463415	0.00463415
% 15	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022		0.00463187	0.00464424	0.00464424	0.00464424
% 20	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022		0.00463187	0.00464424	0.00464424	0.00464424
%     ];
% 
% % % 15 x 15
% % rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
% % 1	0.00313868	0.00345381	0.00376428	0.00408151	0.00440039	0.00454026	0.00475148	0.00489589	0.00489589
% % 2	0.00372804	0.00387327	0.00400155	0.00414319	0.00428737	0.00444983	0.00469545	0.00476574	0.00476574
% % 3	0.00390354	0.00401313	0.00409429	0.00418996	0.0042849	0.0044097	0.00459433	0.00464883	0.00464883
% % 4	0.00408523	0.00411002	0.00416392	0.00421762	0.00426983	0.00436359	0.00449447	0.00453408	0.00453408
% % 5	0.00418943	0.00421751	0.00423044	0.00424642	0.0042546	0.00430589	0.0043945	0.0044123	0.0044123
% % 7	0.00421476	0.00425097	0.00425793	0.00428002	0.00430127	0.00436624	0.00446172	0.00448601	0.00448601
% % 10	0.0042714	0.0042711	0.00431105	0.00434579	0.00438057	0.00447166	0.00461303	0.00463415	0.00463415
% % 15	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022	0.00463187	0.00464424	0.00464424
% %     ];
% 
%     mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                             {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
%     swaptionVolMktData = MktData(mktData);
%     KOSPI200AllMktData('swaptionVol') = swaptionVolMktData;
% 
%     %% create  a KRWBlackModel to calculate market Swaption
%     modelParams= containers.Map({'modelName'},{'Black'});
%     KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 
% 
%     expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
%     tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
%     swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
%     swaptionPriceSurf.expiry= expiry;
%     swaptionPriceSurf.tenor = tenor;
%     
%     fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
% 
%     tempdHTenor = zeros(size(expiry,1),size(tenor,1));
%     for i=1:size(expiry,1)
%         for j=1:size(tenor,1)
%             fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
%             swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
%             swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
%             tempdHTenor(i,j) = expiry(i) + tenor(j);
%         end
%     end
%    
% %% create a CK1FRisky Model with initial modelParams
% %     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
%     
%     vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
%     meanR_r = 0.03;
% %     meanR_r = 0.0;
%     
%     
%     joint_swap_tenor = 1;
% %     targetSize = 10;
% 
%     targetSize = 20;
%     
%     freq = 4;
%     
%     
%     matu = DateDiff(volTargetOptionParams.endDate,valueDateH)/365.0;
%     
%     modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
%                             {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
% 
%     %% allFixings
%     %% CMS10Y & CMS2Y
%     tempDate = volTargetOptionParams.startDate;
% 
%     %CMS10Y
%     irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});
% 
%     %CMS2Y
%     irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});
% 
%     %LIBOR3M
%     irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.006466038});
% 
%     for idxhong=1:10000
%         tempDate = AddDate(tempDate,1,'day');
%         irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
%         irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
%         irIndexFixing3(StrDate(tempDate,'str')) = 0.006466038;
%     end
% 
%     %% fixing info -> let it be the modelParams !
% 
%     allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
%     allFixings('CMS2Y') = irIndexFixing2;
%     allFixings('LIBOR3M') = irIndexFixing3;
% 
%     modelParams('allFixings') = allFixings;
%     
%     KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
%     
% %     tic
% 
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
%     
%     for idx3 = 1:length(out.vol_r.quote)
%         KRWVol_r(1,idx3) = out.vol_r.quote(idx3);
%         KRWRe(1,idx3) = out.re(idx3);
%     end
%     KRWRMSE(1) = out.rmseTotal;
%     
% %     toc
%     %% CK1FRisky model building end
%     %% KRW model building end
%     
% %   %% create a LGM2FModel with initial modelParams
% % % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
% % tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
% % dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
% % dH2.tenor = dH1.tenor;
% % 
% % %swaption only initial parameters
% % % dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
% % 
% % %Samsung
% % % dHTest = [0.058507682	0.500210296
% % % 0.06145709	0.515096302
% % % 0.057841075	0.632558982
% % % 0.069442883	0.632845436
% % % 0.081752868	0.607322125
% % % 0.087574376	0.586095839
% % % 0.103284643	0.52688632
% % % 0.113936628	0.552598585
% % % 0.131046999	0.498409755
% % % 0.153553385	0.437472215
% % % 0.173381122	0.423805445
% % % 0.180703372	0.380344606
% % % 0.207484315	0.305927652
% % % 0.218329282	0.268192317
% % % 0.224885432	0.227191192
% % % 0.221244544	0.231204789
% % % 0.262502203	0.212209604
% % % 0.264579505	0.135257044
% % % 0.318821555	0.118179799
% % % 0.271515922	0.107957745
% % % 0.358024755	0.088834433
% % % 0.33143161	0.069384916
% % % 0.457059078	0.033745097
% % % 0.374067262	0.006573072
% % % ];
% % 
% % %Daewoo
% % % dHTest = [0.037286763	0.525314433
% % % 0.037286763	0.525314433
% % % 0.054571525	0.586928199
% % % 0.069289617	0.618362181
% % % 0.099111958	0.605543022
% % % 0.1123501	0.589459841
% % % 0.139006516	0.523225356
% % % 0.175987354	0.505500158
% % % 0.184616726	0.463242812
% % % 0.204164537	0.412151488
% % % 0.263444475	0.364947643
% % % 0.251615512	0.358337407
% % % 0.259708458	0.315375986
% % % 0.269854863	0.270026782
% % % 0.337623406	0.253701114
% % % 0.258270519	0.179914465
% % % 0.321985902	0.191343978
% % % 0.347503907	0.154371663
% % % 0.36490148	0.127052474
% % % 0.36706888	0.095165077
% % % 0.480469769	0.05084228
% % % 0.423787026	0.033262976
% % % 0.531944266	0.006463726
% % % 0.333342463	0.000100001
% % % ];
% % 
% % % %Sungwoo
% % % dHTest = [0.060654929	0.585843811
% % % 0.065181321	0.621357434
% % % 0.059213061	0.705433024
% % % 0.070356009	0.697274699
% % % 0.084854535	0.641491331
% % % 0.078986949	0.60204605
% % % 0.103979263	0.504724505
% % % 0.128905871	0.522635178
% % % 0.142868097	0.436948669
% % % 0.167927649	0.341253642
% % % 0.171414266	0.384418469
% % % 0.173137707	0.332946905
% % % 0.205465936	0.252922264
% % % 0.190811162	0.233849244
% % % 0.212134317	0.235149902
% % % 0.208305697	0.261176412
% % % 0.267225845	0.213982892
% % % 0.244169602	0.131637885
% % % 0.293167102	0.110338057
% % % 0.302235653	0.107810085
% % % 0.341199539	0.095697121
% % % 0.353976	0.072468098
% % % 0.421517994	0.033547027
% % % 0.357938773	0.006568033
% % % ];
% % 
% % %JaeYeol
% % dHTest = [0.058982775	0.543001377
% % 0.061234997	0.56742444
% % 0.062961086	0.622386628
% % 0.067490351	0.649674014
% % 0.075410597	0.63934757
% % 0.087663903	0.593591285
% % 0.112669112	0.501377655
% % 0.117220704	0.537782529
% % 0.140692951	0.444873509
% % 0.168027009	0.390384507
% % 0.182609655	0.410559905
% % 0.184911175	0.327958872
% % 0.214621342	0.297359459
% % 0.213082379	0.268233279
% % 0.228595305	0.243594185
% % 0.220204578	0.241085585
% % 0.27611566	0.202850389
% % 0.244581841	0.134958255
% % 0.314853582	0.120816977
% % 0.317684576	0.109773131
% % 0.325368543	0.100229783
% % 0.348723657	0.073457815
% % 0.31747133	0.033017077
% % 0.359049924	0.006573421
% % ];
% % 
% % dH1.quote = dHTest(:,1);
% % dH2.quote = dHTest(:,2);
% % 
% % 
% % dH1Orig = dH1;
% % dH2Orig = dH2;
% % 
% % % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
% % Alpha1.tenor =  expiry;
% % Alpha2.tenor =  Alpha1.tenor;
% % correl.tenor = Alpha1.tenor;
% % correl.quote = zeros(length(expiry),1);
% % 
% % %swaption only initial parameters
% % 
% % %Samsung
% % % AlphaTest = [0.0289581	0.006633509
% % % 0.021585088	0.007003076
% % % 0.019812596	0.007056061
% % % 0.015014178	0.007801684
% % % 0.008683651	0.008360616
% % % 0.011075137	0.008248663
% % % 0.008794173	0.011226664
% % % 0.000329846	0.023982837
% % % ];
% % 
% % %Daewoo
% % % AlphaTest = [0.029142908	0.007983918
% % % 0.016018045	0.007219568
% % % 0.00912537	0.007233927
% % % 0.008101129	0.007774553
% % % 0.012850209	0.008916743
% % % 0.011065789	0.009477842
% % % 0.008930619	0.011214209
% % % 0.0001	0.032096513
% % % ];
% % 
% % % %sungwoo
% % % AlphaTest = [0.032526561	0.005901286
% % % 0.021888486	0.006159185
% % % 0.015500653	0.006183779
% % % 0.011483817	0.007702683
% % % 0.008112119	0.008910715
% % % 0.010323471	0.008310996
% % % 0.006966815	0.012340985
% % % 0.000302754	0.020086129
% % % ];
% % 
% % %jaeweol
% % AlphaTest = [0.027941111	0.006258382
% % 0.021327126	0.006691243
% % 0.015009253	0.006502172
% % 0.012188678	0.007718057
% % 0.010231971	0.009065476
% % 0.006437447	0.009477921
% % 0.004646654	0.012890028
% % 0.004203719	0.024248438
% % ];
% % 
% % 
% % Alpha1.quote = AlphaTest(:,1);
% % Alpha2.quote = AlphaTest(:,2);
% % 
% % smoothdH = 1.0;
% % smoothAlpha = 1.0;
% % smoothTermCorr =0.1;
% % targetTermCorr = 0.92;
% % 
% % freq = 4;
% % 
% % modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
% %                         {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% % % import irModels.*;
% % % import targetFunctions.*;
% %                     
% % KRWLGM = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));
% % 
% % 
% % tic
% % % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
% % out = KRWLGM.CalibrateToATMSwaptionSurfaceNormal(KRWBlack,'global',200);
% % % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
% % 
% % dH1Calib = KRWLGM.dH1;
% % dH2Calib = KRWLGM.dH2;
% % Alpha1Calib = KRWLGM.Alpha1;
% % Alpha2Calib = KRWLGM.Alpha2;
% % 
% % modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
% %                         {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});
% %                     
% %                     
% % %% allFixings
% % %% CMS10Y & CMS2Y
% % tempDate = volTargetOptionParams.startDate;
% % 
% % %CMS10Y
% % irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});
% % 
% % %CMS2Y
% % irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});
% % 
% % %LIBOR3M
% % irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.0063});
% % 
% % for idxhong=1:10000
% %     tempDate = AddDate(tempDate,1,'day');
% %     irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
% %     irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
% %     irIndexFixing3(StrDate(tempDate,'str')) = 0.0063;
% % end
% % 
% % %% fixing info -> let it be the modelParams !
% % 
% % allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
% % allFixings('CMS2Y') = irIndexFixing2;
% % allFixings('LIBOR3M') = irIndexFixing3;
% % 
% % modelParamsCalib('allFixings') = allFixings;
% % 
% % KRWLGM2FRisky = LGM2FRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
% %     
% % 
% % toc
% %     %% LGM2FRisky model building end
% %     %% KRW model building end
% 
%     %% NumMethod Init Start
%     numOfRandom = 40000;
%     
%     numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
%                             {'AMC', 7, numOfRandom});
% 
%     numMethodAMC = NumMethod(numMethodInfo);
%     %% NumMethod Init End
%     tic
%     pricingOutMC1 = KRWCK1FRisky.computeInverseFloaterSwapGeneric2(valueDateH,volTargetOptionParams,numMethodAMC);
% %     pricingOutMC1 = KRWCK1FRisky.computeInverseFloaterSwapGeneric2_LSMC(valueDateH,volTargetOptionParams,numMethodAMC);
% %     pricingOutMC1 = KRWCK1FRisky.computeInverseFloaterSwapGeneric3_LSMC(valueDateH,volTargetOptionParams,numMethodAMC);
%     toc
%     aaa = 1.0;
% 
%     
%     callablePrice(idx2) = pricingOutMC1.callable.npv;
%     nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
%     nonCallBPrice(idx2) = pricingOutMC1.nonCallB.npv;
%     eventTimeStepT= pricingOutMC1.eventTimeStep;
%     eventTimeStepTSize = length(eventTimeStepT);
%     eventTimeStep(1:eventTimeStepTSize,idx2) = pricingOutMC1.eventTimeStep(:);
end

aaa =1.0;


In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
